# 시작


In [ ]:
!pip install sentence_transformers
!pip install konlpy

In [2]:
import pandas as pd
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

from tqdm import tqdm
from collections import Counter
import re

# 데이터 로드

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
filtered = pd.read_csv('/content/gdrive/MyDrive/DATA/filtered_drop.csv', encoding='utf-8-sig')
remained = pd.read_csv('/content/gdrive/MyDrive/DATA/remained_drop.csv', encoding='utf-8-sig')

In [5]:
filtered

,기업명,Title,Content,c_index,c_name
0,(사)교회정보기술연구원,"이동현 교회정보기술연구원장, 국무총리 표창 수상",과학기술정보통신부는 (사)교회정보기술연구원 원장 이동현(사진) 목사가 오는 14일 ...,58,정보통신업
1,(사)교회정보기술연구원,[광장] 신원교회，어노인팅 초청 찬양집회 外,"예배·기도회\n\n◇어노인팅 초청 찬양집회=31일 오후 7시 대구 신암동 신원교회,...",58,정보통신업
2,(사)글로벌경쟁력강화포럼,"여성경제정상회의 W20 ""지금이 아니면 성평등 언제 실현하나""",성평등과 여성 역량강화 정책 논의하는\n주요 20개국(G20) 공식 연계그룹 W20...,70,전문과학기술업
3,(사)글로벌경쟁력강화포럼,"여가부-㈜기업가정신, '2019 청년여성 멘토링' 발대식","[머니투데이 대학경제 임홍조 기자] [진선미 여성가족부 장관 ""2030 청년여성이 ...",70,전문과학기술업
4,(사)글로벌경쟁력강화포럼,"[영상] 여가부-㈜기업가정신, '2019 청년여성 멘토링' 발대식",[머니투데이 대학경제 김시현 기자]\n머니투데이 교육법인 ㈜기업가정신이 지난 16일...,70,전문과학기술업
...,...,...,...,...,...
3232,히메시야,카레우동 만드는 법.. 다양한 레시피 비법 '눈길',카레우동 만드는 법.. 다양한 레시피 비법 '눈길'\n\n카레우동 만드는 법이 SN...,56,음식점
3233,히메시야,"일본식 집밥의 달인, ‘덮밥 초밥 카레 우동’…“일본 갈 필요 없네”",[헤럴드경제]] ‘생활의 달인’에서 소개된 일본식 집밥의 달인이 시청자들을 사로 잡...,56,음식점
3234,히메시야,"'생활의달인' 일본식 집밥의 달인, 어디서 만나나",[TV리포트=김명석 기자] '생활의 달인'에서 소개된 일본식 집밥의 달인이 화제다....,56,음식점
3235,힐링필링협동조합,"힐링 필링(healing feeling)협동조합, 핸드메이드 공예 체험으로 나만의 ...",힐링필링협동조합(대표 윤유식)은 공예 작가들이 모여 새로운 핸드메이드 공예 상품을 ...,45,도소매업


In [6]:
remained

,기업명,Title,Content,c_index,c_name
0,(사)글로벌발전연구원,"줄리아 길라드 GPE 의장, 국제교육개발 NGO들과 간담회 가져…","- “대한민국의 교육 역량, 세계에 나누어 달라”\n\n줄리아 길라드(Julia ...",70,전문과학기술업
1,(사)글로벌발전연구원,"줄리아 길라드 GPE 의장, 국제교육 NGO들과 협력모색","[머니투데이 B&C 임수정 기자]“대한민국의 교육 역량, 세계에 나누어 달라”\n\...",70,전문과학기술업
2,(사)글로벌발전연구원,"길라드 전 총리 “대한민국의 교육 역량, 세계에 나누어 달라”",13일 서울 마포구 성미산로 더나은세상에서 줄리아 길라드 GPE 의장과 국내 국제교...,70,전문과학기술업
3,(사)글로벌발전연구원,"줄리아 길라드 GPE 의장, 국제교육개발 NGO와 협력 방안 모색",줄리아 길라드 GPE(Global Partnership for Education) ...,70,전문과학기술업
4,(사)누리복지회,"(사)누리복지회, 법무보호대상자 지원에 협력",[파이낸셜뉴스] 한국법무보호복지공단 부산지부(지부장 이흥수)와 (사)누리복지회(회장...,41,건설업
...,...,...,...,...,...
53969,힐링정보통신,[알림] 2013년 전자신문 9대 기획,"2013년, 새 정부 출범과 함께 산업도 변화의 한가운데 섰습니다. 전자신문은 지난...",45,도소매업
53970,힐링필링협동조합,"힐링이 필요한 연말연시, 나와 사회가 ""가치 선물“",[머니투데이 이경숙 기자] [편집자주] <편집자주>소비는 ‘나’를 표현하는 도구다....,45,도소매업
53971,힐링필링협동조합,배움 의지 충만한 새해 버킷 리스트…매력적인 문화 강좌들은 어떠세요,길가에 줄지어선 멋들어진 공방 앞을 지나며 까치발로 그 안을 기웃거리는가? 아니면 ...,45,도소매업
53972,힐링필링협동조합,"“좋은 제품,소비자가 지켜요” 역발상소비 캠페인",[머니투데이 이경숙 기자] [서울시사경센터 16일까지...공정여행권·공정무역화장품·...,45,도소매업


# 함수 설계

In [7]:
# 전처리 함수 설계
def preprocess_nounlist(column, stop_words='', remove_stopwords=False):
  
  noun_list = []
  for doc in tqdm(column):
    doc_index = doc.find('Copyright')
    if doc_index == -1: pass
    else: doc = doc[:doc_index]

    okt = Okt()

    tokenized_doc = okt.pos(doc)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])
      
    # 불용어 제거
    if remove_stopwords:
      tokens = ' '.join(word for word in tokenized_nouns.split() if not word in stop_words if len(word) > 1)
    # 불용어 미제거
    else:
      tokens = ' '.join(word for word in tokenized_nouns.split() if len(word) > 1)

    noun_list.append(tokens)
  noun_full_list = ' '.join(noun_list)
  noun_full_list = noun_full_list.split()

  return noun_full_list

In [8]:
# 명사 빈도수 계산 함수
def noun_counter(doc, num=10):
  from collections import Counter

  result = Counter(doc)
  most_word = result.most_common(num)

  return most_word

def get_word_list(noun, n):
  most_word = noun_counter(noun, n)
  word_list = []
  for i in range(n):
    word = most_word[i][0]
    word_list.append(word)
  return word_list

In [9]:
# 전처리 함수 설계
def preprocess_noun(doc, stop_words='', remove_stopwords=False):

  doc_index = doc.find('Copyright')
  if doc_index == -1: pass
  else: doc = doc[:doc_index]
  doc_index = doc.find('관련기사')
  if doc_index == -1: pass
  else: doc = doc[:doc_index]
  doc_index = doc.find('다른기사')
  if doc_index == -1: pass
  else: doc = doc[:doc_index]

  okt = Okt()
  tokenized_doc = okt.pos(doc)
  tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])
      
  noun_list = []
  # 불용어 제거
  if remove_stopwords:
    tokens = ' '.join(word for word in tokenized_nouns.split() if not word in stop_words if len(word) > 1)
  # 불용어 미제거
  else:
    tokens = ' '.join(word for word in tokenized_nouns.split() if len(word) > 1)
  noun_list.append(tokens)

  return noun_list

In [10]:
# 전처리 함수 설계
def preprocess_sentence(sentence, stop_words='', remove_stopwords=False):

  sentence_index = sentence.find('Copyright')
  if sentence_index == -1: pass
  else: sentence = sentence[:sentence_index]

  sentence_index = sentence.find('관련기사')
  if sentence_index == -1: pass
  else: sentence = sentence[:sentence_index]

  sentence_index = sentence.find('다른기사')
  if sentence_index == -1: pass
  else: sentence = sentence[:sentence_index]


  sentence = re.sub(r'\[[^)]*\]', '', sentence)
  sentence = re.sub(r'\【[^)]*\】', '', sentence)
  sentence = re.sub(r'\<[^)]*\>', '', sentence)
  sentence = re.sub('\n', '', sentence)
  sentence = re.sub('"','', sentence)
  sentence = re.sub("'","", sentence)
  sentence = re.sub("[^가-힣a-zA-Z0-9]", " ", sentence)

  okt = Okt()

  tokenized_doc = okt.pos(sentence)
  tokenized = ' '.join([word[0] for word in tokenized_doc])

  # 불용어 제거
  if remove_stopwords:
    sentence = ' '.join(word for word in tokenized.split() if not word in stop_words)
  # 불용어 미제거
  else:
    sentence = ' '.join(word for word in tokenized.split())

  return sentence

In [11]:
#1 정보통신업 불용어
stop_words1=["대표","한국","빌딩","통해","제공","시장","교육","국내","서울","대한","서울특별시","서울","강남구","기반","지난","에스","판사","사진","코리아","아이","사용","대상","부장","시작","번길","참여","대표이사","병역","최근","타워","구독","교수","행사","생각","올해","부산","회장","관련","부문","무단","기존","의원","독자","영등포구","케이","과정","지방법원","모든","서초구","그룹","진흥","정당인","가장","확인","출판사","현장","에이","이후","중구","역삼동","지난해","한편","확대","미국","노력","이야기","우리","이상","서구","응원","여의도동","매매","자신","광주","사장","자체","마련","발표","라며","지속","마포구","학생","비롯","하나","처음","작가","강서구","최고","여행","서초동","전국","부산광역시","내용","국가","상무","송파구","오전","에이치","통한","중국","가지","저작권","중이","코로나","최초","자문","블록","오후","삼성동","주요","사례","일본","의료","공개","의미","단지","국민","원장","이하","대구광역시","비즈","향후","실제","사랑","대전","인기","인천광역시","본보기","광역시","세상","국내외","자리","신문","개선","논현동","신규","금천구","도시","양성","오피스텔","대구","일보","가운데","자재","북구","대통령","동구","장관","스타","테헤란로","구로구","기념","제이","공연","이름","가산동","바로가기","계속","에프","경기","구로동","표창","로서","전체","밸리","대학교","맞춤","무엇","시민","인천","경우","크게","담당","중소","파악","단체","계기","이유","이번","울산","주관","특징","개월","이제","기대","강남대로","종로구","발인","관계","도전","그동안","성동구","선박","우리나라","시행","친구","은행","상가","버스","대전광역시","현실","마음","오마이","로부터","시스","출신","사실","왼쪽","유치","사항","차별","역시","선도","수도","이사장","먼저","매우","이전","산학","서로","저자","레이","서울시","상호","경기도","자사","여기","용산구","이슬아","고려","별도","최종","반영","공단","어려움","스스로","대신","성수동","한경닷컴","흐름","아트","머니투데이","지음","대치동","헌트","측은","연수구","앞서","여개","수원지방법원","거나","사이","이데일리","신사동","용역","시상식","서점","수업","정말","독서","초청","시기","문정동","사의","공부","양재동","할인","프레","사무실","주년","이기","그것","동대문구","하이","어디","서적","광진구","노컷뉴스","정보","금융","안방","재테크","객원","이데일리","기자","저작권","뉴스","무단","금지","배포"]
stop_words_ic = ' '.join(stop_words1)

#2 전문과학기술업 불용어
stop_words2=["대표","한국","통해","빌딩","서울","대한","지난","계획","교수","올해","생각","아이","최근","가장","저작권","이후","프레","자신","모든","한편","기존","강조","지금","코리아","통한","사례","개선","처음","라며","에스","오후","우수","에이","때문","부분","이상","관련","대학","증여","포럼","역활","서울시","노조","사장","감사","독일","서초동","비롯","돼지","누구","우리","테이","무엇","로서","일본","역삼동","중이","우리나라","최초","바로가기","케이","오늘","부장","뉴시스","매년","김포시","이하","이제","로부터","이크","대부분","김포","마포구","안나","향후","인정","왼쪽","그동안","웍스","먼저","어디","역시","그대로","측은","인지","연대","여기","교회","이번","시도","양재동","거나","여개","그것","라면","이데일리","논의","파이낸셜뉴스","강동완","서초","이기","아주","노컷뉴스","정보","금융","안방","재테크","객원","기자","저작권","뉴스","무단","금지","배포"]
stop_words_sci = ' '.join(stop_words2)

#3 도소매업 불용어
stop_words3=["대표","대표이사","서울특별시","빌딩","기여","국가","후보자","포상","모범","한국","강남구","산업","서울","사람","제조","통해","국내","에스","코리아","대한","부산광역시","번길","아이","영등포구","뉴스","서초구","오전","마포구","부장","대구광역시","인천광역시","등반","지난","가장","케이","서초동","여의도동","삼성동","매장","에이","구독","대전광역시","미국","인기","서구","강서구","이사","구로구","베트남","송파구","이후","과정","자동차","우리","에이치","처음","금천구","스포츠","개척","논현동","유지","중국","테헤란로","모든","가지","자신","활동","에프","교수","구로동","가산동","때문","오후","울산","치과","지점","일본","경북","동구","이상","중이","역시","종로구","대학교","한편","최대","광진구","북구","이유","도움","계속","친구","미정","최초","독일","이하","추가","동물","포함","통한","요즘","무엇","용산구","크게","지리산","회원","왼쪽","관악구","보기","명의","오늘","그대로","처리","가죽","논현동","송도동","로서","추천","양재동","행복","여기","트리","달서구","향후","고려","해운대구","바로가기","대행","서초","진자","서로","인쇄","한의원","강동완","주요","거나","대신","종로","서교동","도산대로","헤나","이번","위원","발표","동대문구","오버","경제범죄","마지막","중인","방배동","보통","유럽","시스","매일","별로","서울역","수상","라면","출발","통합","라이프","김정식","펠트","한경닷컴","박재홍","일이","최선","남부","양천구","광산구","위드","양천구","경인로","영동대로","봔트길","여개","전혀","노컷뉴스","정보","금융","안방","재테크","객원","이데일리","기자","저작권","뉴스","무단","금지","배포"]
stop_words_re = ' '.join(stop_words3)

#4 건설업 불용어
stop_words4=["대표","기여","국가","대표이사","의원","주식회사","빌딩","정당인","한국","천원","에스","광주","코리아","서울","센터","군수","건수","횟수","순위","평가","통해","졸업","실천","전라남도","전라북도","교육","지난","대한","광역시","국내","민주당","대통령","이사장","대학원","서초구","공헌","무직","보고서","미래","역삼동","송파구","에프","대학","제일모직","영등포구","뉴스","김대중","오전","실내","여의도동","최고","제주","추진","구독","강남","직원","공동","마포구","최근","전북","청장","관련","이웃","강서구","단체","주거","서초동","우리","경기","기탁","일보","구로구","유지","산학","전남","에이치","상황","중구","논현동","노원구","연구원","통합","바이오","배포","진행","생각","경남","이후","상장","내용","인터","올해","지속","사장","관악구","감사","독자","비롯","가격","도시","은평구","대전","충남","연대","회사원","새누리당","강원","사무","소장","사무국","회원","고려","보고","이용","서대문구","당시","대호","성동구","통한","오피스텔","신사동","바탕","겸임","일반","양천구","제이","적극","서초","강동구","가장","월드컵","광진구","부장","기업인","전주시","홍콩","확대","종로구","중랑구","용산구","경기도","학과","국민의당","통일","기관","오후","기반","작성","한편","대치동","강북구","판매업","지난해","삼성동","김포대","동대문구","동부","강조","구로동","울산","시작","전남대학교","서울시","동작구","제주도","테헤란로","도봉구","대신","모든","무엇","로서","실제","로부터","충북","라며","부의","시스","주요","달라","감시","기획재정부장관","부분","마을","방배동","전주","경북","서구","부문","금호","부사","향후","한강로","그동안","학생회장","우성","광주시","저작권","의혹","광산구","진천","뉴시스","연구개발","가지","서교동","양사","송산","금회","고등학교","이기","목포","김종","강릉","사비","사례","웅진","두산","그동안","학생회장","본부장","광주시","위혹","연합뉴스","서교동","대비","알선","하이","구로","유공","가산동","초크","미전","순천시","최소","청담동","차관","신우","해당","번지","여건","번은","남동","제주시","번길","김현","정도","떄문","이야기","방배","논현","이재용","협회장","김석","부안군","군산","정진","창호","이름","바로가기","법학과","행정학","대한민국","연세","남경","스틸","선진","표현","가락동","닉스","수도","주관","보기","시상식","전자상거래","반포동","용산","지시","관장","재단","노컷뉴스","정보","금융","안방","재테크","객원","이데일리","기자","저작권","뉴스","무단","금지","배포"]
stop_words_con = ' '.join(stop_words4)

#5 기기류 제조업 불용어
stop_words5=["대표이사","서울특별시","대표","서울","빌딩","시장","통해","국내","한국","회사","중소기업","에스","사용","부산광역시","분야","강남구","번길","경기","해외","대한","광주","인천광역시","대구광역시","에이","광역시","아이","적용","부산","뉴스","기존","무단","기자","행사","지난","임대","영등포구","최근","최초","대전광역시","출시","참여","서구","구독","중구","일본","올해","서초구","한편","체결","오전","대상","최고","여의도동","통한","동구","대구","관련","비롯","서울시","한일전","강서구","시작","울산","최대","저작권","가장","확보","역삼동","서초동","대전","구로구","중앙회","남구","과정","오후","중이","부문","선도","확인","북구","적극","연합뉴스","시간","실시간","향후","방문","하이","시스","로부터","우산","바로가기","국내외","가산동","일링","정공","연수구","중동","송도동","이후","주요","해운대구","달서구","성동구","테헤란로","수성구","독일","달성","가운데","일반","노즐","여유","테헤란로","수성구","양산시","왼쪽","계기","독일","도전","로서","블루","닉스","비즈","추가","대부분","확산","우리나라","감사","나눔","의미","강남대로","남동구","전자신문","계속","매우","통화","우리","기대","중개업","요구","자신감","가능성","복합","대화","서로","이번","아이오","이노","유성구","세리","이재수","때문","코엑스","무료","본부장","현지","가구","충남","특수","속기","더블유","고유","경험","바람","판로","이글","서초","성수동","이전","수상자","그동안","프리","별세","광진구","홀딩스","중인","전남","개사","대통령","신사동","절대그이","신사동","종로구","광산구","이하","맞이","이데일리","경복대","도산대로","영동대로","청담동","우동","부평구","필수","초기","레이","유치","협의","앞서","공간","역시","어린이","엄마","주로","기금","부산진구","방배동","양재동","문정동","용산구","주거","폴리텍","필요","이중","크게","이기","조성","한경닷컴","무선통신","차지","다수","주최","유수","종로","사이","이재","도시","오스","러브","꼬임","경기도","크기","주년","인기","완화","인수","세원","허가","노컷뉴스","정보","금융","안방","재테크","객원","이데일리","기자","저작권","뉴스","무단","금지","배포"]
stop_words_ma = ' '.join(stop_words5)

#6 음식점 불용어
stop_words6=["대표","서울","책방","사람","두리반","시작","마포구","한국","생각","사장","문화","직접","이름","가장","직원","통해","연남동","작가","추리소설","개발","사이","마포","대한","지금","지난","모든","서점","하나","처음","서울시","경제","뉴스","제공","마음","오후","우리","지난해","전문점","최근","이후","기타","자신","망원동","때문","프랑스","설명","배포","현재","가지","칩스","독자","소비자","방식","무단","금지","이씨","월향","이야기","과정","사랑","정도","이제","의미","구독","활용","느낌","모습","회사","당시","이용","활동","방법","올해","중국","특징","이자","대표이사","대부분","그대로","부부","추천","통해","연남동","위치","사용","소개","작가","사이","일본","대한","지금","지난","진행","거리","모든","서점","처음","제공","마음","오후","우리","지난해","최근","이후","기타","자신","망원동","때문","설명","배포","현재","독자","무단","이씨","월향","이야기","이제","의미","구독","느낌","당시","영화","이용","활동","방법","친구","중국","특징","누구","대표이사","대부분","그대로","수백","부부","자체","작업","여기","보고","선택","오늘","이유","계속","먼저","완성","명동","유씨","오전","국악","예정","대신","별로","추소영","한국인","기본","부담","비롯","라며","학교","상호","이상","경우","구성","고민","마련","연남","관리","출신","기존","그림","저작권","부각","철거","스페인","출시","유지","주로","표현","차례","더다듬","도움","대상","명의","소리","관심","서로","라면","구름","개월","역시","요즘","바삭","전공","공개","채림","독립","정광수","대한민국","자극","중이","찾기","무엇","제대로","중심","서교동","일대","규모","등장","블러","시래기","책장","자주","만들기","덕분","내부","한편","시절","배우","상황","수도","자랑","호이안","북한","카라","지기","바탕","주인공","노력","최고","수제","해당","타이","우리나라","대중","예전","시선","중앙","안종","크게","랜드","전시","테마","유니온","협회","미디어","담당","이전","확인","전혀","기억","과거","아침","내용","비결","합정동","최윤","도서","재판","레이","관련","가운데","상태","기술","분석","거나","기록","추구","보기","로서","일이","박종일","콘서트","형태","입자","새벽","기분","트럭","기도","진열","수준","철학","보통","리브","국제","산업","나무","정체","만난","미국","그동안","기회","본격","촬영","용역","그것","최대","면허","발걸음","바이","강연","도전","아내","미리","연구","그때","남편","철거민","가슴","낭독","일부","혐의","용산","김진용","연희동","차지","동생","평소","이화","바다","할머니","한쪽","사슴","포함","방향","시스템","목표","필요","정말","벽면","작년","이번","한번","질문","강남","결정","가면","이란","로부터","아시아","눈길","비교","상승","이강일","가을","승리","출구","백승","공부","달라","당일","후반","월요일","조찬","에반스","해소","문재인","창비","발굴","국가","분야","집중","디지털","발견","확장","어디","이기","중인","벌써","음반","최초","알리","비밀","진출","아예","아들","예술가","위기","타임","그냥","간다","곳곳","살짝","사건","장구","가스","이동수","김선","나눔","활성화","강동완","공급","겨울","산경","공인","유미","오른쪽","공항","탁주","마무리","반면","타고","가까이","아버지","졸업","초반","동교동","언론","신인호","미가","초반","언론","신인호","미가","사례","노소","사실","미소","대학로","번지","실내","포기","눈물","주가","북아현동","규칙","부인","규칙","점점","공덕","공덕역","싱가포르","라스트","배합","한정","로우","카포나타","강조","악기","연주자","동포","동경","정식","조각가","우렁이","이병학","미숙","진선","재능","통한","회장","지속","특성","안정","특산","근처","마찬가지","이면","저희","전체","빼곡","고스","청장","구청","사고","조명","고등학교","영역","중장","얼굴","최씨","지적","대기","예방","노컷뉴스","정보","금융","안방","재테크","객원","이데일리","기자","저작권","뉴스","무단","금지","배포"]
stop_words_res = ' '.join(stop_words6)

# 모델 정의

In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

# 데이터 선택

In [ ]:
filtered[filtered['기업명']=='닭날다']

,기업명,Title,Content,c_index,c_name
1848,닭날다,홍대앞 동네잡지 '스트리트 H' 창간 3주년 맞아,(서울=연합뉴스) 정천기 기자 = 홍대 앞 동네잡지 '스트리트 H'가 3주년을 맞아...,56,음식점


In [ ]:
remained[remained['기업명']=='닭날다']

,기업명,Title,Content,c_index,c_name
22282,닭날다,기발한 발상의 사이드 메뉴개발로 매장 인지도 업! <닭날다>,[이데일리 EFN 황보경 객원기자] 홍익대학교 정문에서 극동방송국 방향으로 내려오는...,56,음식점
22283,닭날다,개에게 이단옆차기 날리는 '용맹한 닭' 눈길,[노컷뉴스 편집팀] 개에게 하이킥을 날리는 용맹한 닭이 누리꾼들에게 인기다.\n\n...,56,음식점
22284,닭날다,[경향NIE] 고등학생용 - 전시행정과 홍대문화,[경향NIE] 고등학생용 - 전시행정과 홍대문화\n【자료 1】 상업자본·부동산·권리...,56,음식점
22285,닭날다,홍대 앞 치킨 맛집 상륙 작전,"치킨 요리를 주문하기 전 ‘프라이드를 먹을까, 양념을 먹을까’ 하는 고민은 이제 거...",56,음식점
22286,닭날다,상업자본·부동산·권리금·지자체 전시행정이 홍대 앞 망쳐,"홍대 앞 거리의 상징이라고 할 수 있는 아트숍, 공연장, 갤러리 등 예술이 숨쉬는 ...",56,음식점
22287,닭날다,홍대 앞은 ‘쩐의 전쟁’,"젊음과 예술, 문화의 거리로 통하는 홍대 앞이 건물주들의 임대료 인상에 위기를 맞고...",56,음식점
22288,닭날다,[Ki-Z 인디人 No.5] ‘갤럭시 익스프레스’,[쿠키 문화] ‘한국은 좁다’ 일본에 이어 미국 진출. 세계로 뻗어 나가는 갤럭시 ...,56,음식점
22289,닭날다,“힘내세요” 노사모 격려글 잇따라,“당당하게 다녀오십시오.” “‘위대한 대통령이었다’고 기록되시리라 굳게 믿고 있습니...,56,음식점
22290,닭날다,[SNS 맛 감정단] 치킨을 사랑하는 이들의 필수지침서 '이색 치킨집',여기 사람들이 이구동성으로 추천하는 진짜 맛집이 있다. 한국경제신문 온라인미디어 [...,56,음식점


In [13]:
real_data = pd.DataFrame({'Content':[filtered['Content'][1848], remained['Content'][22282], remained['Content'][22284], remained['Content'][22285], remained['Content'][22286], remained['Content'][22287], remained['Content'][22290]]})
fake_data = pd.DataFrame({'Content':[remained['Content'][22283], remained['Content'][22288], remained['Content'][22289]]})

In [14]:
real_data

,Content
0,(서울=연합뉴스) 정천기 기자 = 홍대 앞 동네잡지 '스트리트 H'가 3주년을 맞아...
1,[이데일리 EFN 황보경 객원기자] 홍익대학교 정문에서 극동방송국 방향으로 내려오는...
2,[경향NIE] 고등학생용 - 전시행정과 홍대문화\n【자료 1】 상업자본·부동산·권리...
3,"치킨 요리를 주문하기 전 ‘프라이드를 먹을까, 양념을 먹을까’ 하는 고민은 이제 거..."
4,"홍대 앞 거리의 상징이라고 할 수 있는 아트숍, 공연장, 갤러리 등 예술이 숨쉬는 ..."
5,"젊음과 예술, 문화의 거리로 통하는 홍대 앞이 건물주들의 임대료 인상에 위기를 맞고..."
6,여기 사람들이 이구동성으로 추천하는 진짜 맛집이 있다. 한국경제신문 온라인미디어 [...


index 1번이 '닭날다'의 정보 데이터, 3번, 6번은 치킨 데이터 안에 닭날다 정보도 들어있음

나머지는 애매

In [15]:
fake_data

,Content
0,[노컷뉴스 편집팀] 개에게 하이킥을 날리는 용맹한 닭이 누리꾼들에게 인기다.\n\n...
1,[쿠키 문화] ‘한국은 좁다’ 일본에 이어 미국 진출. 세계로 뻗어 나가는 갤럭시 ...
2,“당당하게 다녀오십시오.” “‘위대한 대통령이었다’고 기록되시리라 굳게 믿고 있습니...


In [25]:
real = real_data['Content'][1]

In [26]:
real

'[이데일리 EFN 황보경 객원기자] 홍익대학교 정문에서 극동방송국 방향으로 내려오는 큰길 뒷골목, 이곳은 아지트 같은 카페, 음식점들이 길 양옆에 옹기종기 모여 있는 또 다른 맛집 골목이다.\n\n독특한 상호만큼이나 독특한 콘셉트로 유명한 <닭날다>는 이 골목에서 가장 잘 알려진 음식점 중 하나. 2001년 문은 연 이곳은 미군용 식기와 수통에 마시는 맥주, 군대 막사와 같은 밀리터리 콘셉트로 주목을 끌며 꾸준한 인기를 얻고 있다.\n\n그러나 이러한 분위기 보다 더 인기를 끌고 있는 것이 매운 닭볶음인 ‘오리지날 철판(1만4000원)’이다. 알싸한 맛의 중독성 때문인지 <닭날다>를 대표하는 메뉴로 자리 잡고 있다.\n\n오리지날 철판과 더불어 눈길을 끄는 것이 ‘알밥(2000원)’. 날치알이 들어간 알밥이 아니라 말 그대로 밥 안에 알이 들어있다.\n\n잘 뭉친 밥 위에 김가루와 깨가 송송 뿌려져 제공되는 이 메뉴는 삶은 달걀이 통째로 밥 안에 들어간 이색적인 사이드 메뉴다.\n\n감칠맛 나는 오리지널 철판의 매운 소스는 밥은 물론 삶은 달걀과도 좋은 궁합을 이루며 2% 부족한 고객들의 배를 채우는데 일조하고 있다.\n\n\n‘알밥’은 사이드 메뉴지만 임팩트 있는 메뉴개발로 인해<닭날다> 최고의 성공 키워드로 자리매김하고 있다.\n\n[ 도움말 : 월간 외식경영 ]\n\n▶ 관련기사 ◀\n☞6가지 허브와 어우러지는 이색 알밥 <해장금>\n☞미식가의 주머니를 터는 황금알! ‘알밥’\n\n[ ⓒ 프랜차이즈 창업 체인 가맹 사업 네트워크 " 이데일리 EFN "]\n\n\n▶ 돈이 보이는 이데일리 모바일ㆍ실시간 해외지수/SMS <3993+show/nate/ez-i>\n▶ 가장 빠른 글로벌 경제뉴스ㆍ금융정보 터미널, 이데일리 MARKETPOINT\n＜ⓒ종합 경제정보 미디어 이데일리 - 무단전재&재배포 금지＞\n＜안방에서 만나는 가장 빠른 경제뉴스ㆍ돈이 되는 재테크정보 - 이데일리TV＞\n\n객원 (adevent@)\n객원 (adevent@)\nCopyright ⓒ 

# 불용어 제거 문장 데이터

단어 리스트는 불용어 제거 명사만 50개

## 변수 정의

In [27]:
#진짜 기사 문서
real_sen_doc = preprocess_sentence(real, stop_words=stop_words_res, remove_stopwords=True)
real_sen_doc

'홍익대학교 정문 에서 극동방송국 으로 내려오는 큰길 뒷골목 은 아지트 같은 카페 음식점 양옆 옹기종기 모여 있는 또 다른 맛집 골목 이다 독특한 만큼이나 독특한 콘셉트 유명한 는 골목 에서 잘 알려진 음식점 2001년 은 은 미군 식기 와 수통 마시는 맥주 군대 막사 와 같은 밀리터리 콘셉트 주목 끌며 꾸준한 인기 를 얻고 있다 그러나 이러한 분위기 보다 인기 를 끌 있는 매운 닭 볶음 오리 지날 철판 1만 4000원 이다 알싸한 맛 중독 인지 를 하는 메뉴 자리 잡고 있다 오리 지날 철판 더불어 끄는 알밥 2000원 날치 들어간 알밥 아니라 밥 들어있다 잘 뭉친 밥 가루 와 깨 송송 뿌려져 되는 메뉴 는 삶은 달걀 통째 밥 들어간 이색 사이드 메뉴 감칠맛 는 리지 널 철판 매운 소스 는 밥 은 물론 삶은 달걀 과도 좋은 궁합 이루며 2 부족한 고객 를 채우는데 일조 하고 있다 알밥 은 사이드 메뉴 지만 임팩트 있는 메뉴 인해 성공 키 워드 자리매 하고 있다'

In [28]:
#진짜 기사 단어
real_noun_doc = preprocess_noun(real, stop_words=stop_words_res, remove_stopwords=True)
real_noun_doc = real_noun_doc[0]
real_noun_word = get_word_list(real_noun_doc.split(), 50)
real_noun_word

['메뉴',
 '철판',
 '알밥',
 '음식점',
 '골목',
 '콘셉트',
 '인기',
 '오리',
 '달걀',
 '사이드',
 '황보',
 '홍익대학교',
 '정문',
 '극동방송국',
 '큰길',
 '뒷골목',
 '아지트',
 '카페',
 '양옆',
 '옹기종기',
 '다른',
 '맛집',
 '미군',
 '식기',
 '수통',
 '맥주',
 '군대',
 '막사',
 '밀리터리',
 '주목',
 '분위기',
 '볶음',
 '중독',
 '자리',
 '잡고',
 '날치',
 '가루',
 '송송',
 '통째',
 '이색',
 '감칠맛',
 '리지',
 '소스',
 '궁합',
 '고객',
 '일조',
 '임팩트',
 '성공',
 '워드',
 '도움말']

## 임베딩 (불용어 처리, 명사)

In [29]:
real_sen_doc_emb = model.encode([real_sen_doc])
print(f'real_sen_doc_emb.shape : {real_sen_doc_emb.shape}')
real_noun_word_emb = model.encode(real_noun_word)
print(f'real_noun_word_emb.shape : {real_noun_word_emb.shape}')

real_sen_doc_emb.shape : (1, 768)
real_noun_word_emb.shape : (50, 768)


In [ ]:
# real_sen_doc_emb
# real_noun_word_emb

## 코사인 유사도 (불용어 처리, 문장)

### real + fake

In [ ]:
fake = fake_data['Content'][1]
#가짜 기사 문서
fake_sen_doc = preprocess_sentence(fake, stop_words=stop_words_res, remove_stopwords=True)
#가짜 기사 단어
fake_noun_doc = preprocess_noun(fake, stop_words=stop_words_res, remove_stopwords=True)
fake_noun_doc = fake_noun_doc[0]
fake_noun_word = get_word_list(fake_noun_doc.split(), 50)
#임베딩
fake_sen_doc_emb = model.encode([fake_sen_doc])
print(f'fake_sen_doc_emb.shape : {fake_sen_doc_emb.shape}')
fake_noun_word_emb = model.encode(fake_noun_word)
print(f'fake_noun_word_emb.shape : {fake_noun_word_emb.shape}')

fake_sen_doc_emb.shape : (1, 768)
fake_noun_word_emb.shape : (50, 768)


In [ ]:
fake_sen_doc

'은 좁다 이어 세계 뻗어 나가는 갤럭시 익스 프레 갤럭시 익스 프레 는 우주 를 쾌속 으로 질주 한다는 음악 우주 뻗어 나가는 전파 되어 개개인 소우주 를 여행 있다 개인 세상 만물 만날 는 없을지라도 음악 은 언젠가 당신 만나게 될지도 데뷔 2006년 EP 투 갤럭시 To the galaxy 소속 사러 브락 컴퍼니 Loverock Company 장르 포스트 펑크 와 사이키 델리 멤버 성현 멤버 이주현 베이스 보컬 김희 드럼 코러스 보컬 박종현 보컬 멤버 윤홍구 드럼 탈퇴 앨범 2006 EP 투 갤럭시 To the galaxy 2007 EP 램블 라운드 Ramble around 2008 정규 1 노이즈 파이어 Noise on fire 2009 EP 컴온 겟업 Come on Get up 2010 정규 2 와일드 데이즈 Wild days 2011 EP 개구쟁이 with 크라잉 넛 곡정 글더 블랙 Jungle the black 진짜 너 를 원해 오예 지나고 나면 언제나 좋았어 개인 이력 이주현 35 자살 매미 와 럭스 RUX 게토밤즈 Ghetto bombs 재주 있어 아트워크 김희 31 양구 초등학교 양구 중학교 양구 종합 박종현 31 5 사단 육군 병장 수상 2008 Mnet KM 뮤직페스티벌 수상 2009년 대중음악 최우수 노이즈 파이어 Noise on fire 2011년 대중음악 음악인 Music Alliance Pacts 선정 2012년 1월 각국 아티스트 아티스트 선정 취미 특기 이주현 그리기 감상 명상 사색 운전면허증 김희 담배 게임 욕 하기 박종현 먹고 진상 부리기 홍대 출몰 지역 은 이주현 성산 초등학교 앞 커피 플 리즈 등촌 칼국수 홍대 상암 CGV 상암 홈플러스 상암 하늘 공원 한강 김희 홍대 여러 술집 박종현 닭 날다 에게 갤럭시 익스 프레 이주현 나를 사람과 섞이게 하는 나를 세상 녹 하는 직장 이며 놀이터 이며 이며 동료 김희 하고 있는 밴드 있는 하는 밴드 박종현 우주선 탑승 하고 항해 한다 애칭 이주현 갤럭시 베이시스트 베이스 이라서 김

In [ ]:
# real 문서 + fake 문서
distances = cosine_similarity(real_sen_doc_emb, fake_sen_doc_emb)
distances

array([[0.23681122]], dtype=float32)

In [ ]:
# real 문서 + fake 단어
distances = cosine_similarity(real_sen_doc_emb, fake_noun_word_emb)
# print(distances)
print(np.mean(distances))

-0.048069406


In [ ]:
# real 문서 + fake 단어평균
fake_noun_word_emb_mean = np.mean(fake_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, fake_noun_word_emb_mean)
distances

array([[-0.06292852]], dtype=float32)

In [ ]:
# real 단어 + fake 문서
distances = cosine_similarity(real_noun_word_emb, fake_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.2948992


In [ ]:
# real 단어 + fake 단어
distances = cosine_similarity(real_noun_word_emb, fake_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.5972948  0.43616983 0.45565605 ... 0.48576665 0.50948596 0.2621813 ]
 [0.6236847  0.5433911  0.462256   ... 0.43141812 0.51718175 0.32967314]
 [0.8366065  0.75025225 0.65857625 ... 0.5748139  0.70239294 0.4087414 ]
 ...
 [0.6295496  0.49643505 0.46876863 ... 0.3336129  0.5156909  0.24603473]
 [0.8746172  0.7526971  0.6673922  ... 0.5915314  0.70799124 0.42351407]
 [0.6377727  0.54896575 0.54767907 ... 0.4638309  0.5162641  0.31390324]]
(50, 50)
0.6414043


In [ ]:
# real 단어평균 + fake 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, fake_sen_doc_emb)
distances

array([[0.34581983]], dtype=float32)

### real + real2

In [30]:
real2 = real_data['Content'][3]
#진짜 기사 문서
real2_sen_doc = preprocess_sentence(real2, stop_words=stop_words_res, remove_stopwords=True)
#진짜 기사 단어
real2_noun_doc = preprocess_noun(real2, stop_words=stop_words_res, remove_stopwords=True)
real2_noun_doc = real2_noun_doc[0]
real2_noun_word = get_word_list(real2_noun_doc.split(), 50)
#임베딩
real2_sen_doc_emb = model.encode([real2_sen_doc])
print(f'real2_sen_doc_emb.shape : {real2_sen_doc_emb.shape}')
real2_noun_word_emb = model.encode(real2_noun_word)
print(f'real2_noun_word_emb.shape : {real2_noun_word_emb.shape}')

real2_sen_doc_emb.shape : (1, 768)
real2_noun_word_emb.shape : (50, 768)


In [31]:
real2_sen_doc

'치킨 요리 를 주문 하기 프라이드 를 먹을까 양념 먹을까 하는 은 거둘 홍대 앞 에는 크림소스 재운 치킨 매콤 철판 치킨 다양한 치킨 메뉴 무장 맛집 성황 이루고 있다 치르 치르 여자 좋아할 다양한 치킨 메뉴 있는 일반 치킨 있는 양파 소스 를 내어주는데 양파 토마토 채소 와 레몬주스 를 섞어 만든 으로 치킨 맛 돋워 주는 치르 치르만의 자랑거리 치킨 쉽게 잡 있도록 디자인 된 완소 집게 와 치킨 에서 막걸리 를 파는 의외 또한 손님 즐겁게 해주는 요소 DATA 상상마당 건너편 골목 영업 시간 3시 다음 날 5시 문의 02 332 3187 둥지 파닭 파 를 둥지 처럼 둥그런 모양 으로 만든 뒤 닭 캐슈 너트 마늘 간장 소스 를 올린 요리 1만 6천 9 백원 몸짱 치킨 기름 쏙 뺀 전기 구이 치킨 요리 담백하면서도 부드러운 맛 일품 다이어트 민감한 여성 손님 에게 인기 많다 1만 3천 9 백원 뿔레 치킨 해외여행 다니다 둘 모은 소품 으로 꾸민 국적 인테리어 와 이색 메뉴 여성 손님 80 90 를 하는 아이 와 함께 가족 단위 방문 하는 손님 많다 3시 5시 까지는 치킨 요리 와 함께 샐러드 포테이토칩 어니언 링 함께 나오는 세트 메뉴 를 품 가격 판매 한다 DATA 홍대 놀이터 옆 오르막 골목 영업 시간 3시 다음 날 1시 30분 문의 02 322 4870 스위트 칠리 뿔레 매콤 하면서도 콤 달콤한 스위트 칠리소스 치킨 풍 미를 더하는 메뉴 소스 강하지 않아 매운 음식 즐기지 않는 에게도 권할 만하 1만 6천원 까르보나라 뿔레 뿔레치킨 홍대 맛집 으로 만들어 일등 공신 으로 크림소스 푹 재운 보들보들한 순 살이 별미 1만 6천원 닭 날다 매콤 치킨 요리 생각날 방문 하면 좋은 음식 주문 하고 기다리는 건빵 나눠주고 생맥주 를 군용 수통 담아주는 밀리터리 컨셉트 재미나다 넓은 공간 직장인 회식 장소 사랑받으며 양 푸 짐해 일부러 저녁 시간 맞춰 방문 하는 손님 많다 DATA 삼거리 포차 에서 홍대 으로 직진 영업 시간 4시 다음 날 4시 문의 02 322 45

In [ ]:
# real 문서 + real2 문서
distances = cosine_similarity(real_sen_doc_emb, real2_sen_doc_emb)
distances

array([[0.71098506]], dtype=float32)

In [ ]:
# real 문서 + real2 단어
distances = cosine_similarity(real_sen_doc_emb, real2_noun_word_emb)
# print(distances)
print(np.mean(distances))

0.004988513


In [ ]:
# real 문서 + real2 단어평균
real2_noun_word_emb_mean = np.mean(real2_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, real2_noun_word_emb_mean)
distances

array([[0.01121997]], dtype=float32)

In [ ]:
# real 단어 + real2 문서
distances = cosine_similarity(real_noun_word_emb, real2_sen_doc_emb)
# print(distances)
print(np.mean(distances))

-0.08961272


In [ ]:
# real 단어 + real2 단어
distances = cosine_similarity(real_noun_word_emb, real2_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.38801894 1.0000002  0.6034899  ... 0.37387735 0.39919356 0.39063752]
 [0.35138983 0.42752993 0.4433522  ... 0.46267188 0.5112967  0.5003484 ]
 [0.5794052  0.5357519  0.43698254 ... 0.597479   0.48117152 0.7547186 ]
 ...
 [0.36071032 0.35069203 0.36615738 ... 0.86102074 0.32716727 0.5541719 ]
 [0.53705555 0.5331935  0.45526278 ... 0.6218757  0.4859876  0.7742833 ]
 [0.37648278 0.38964963 0.3742798  ... 0.56141406 0.4251297  0.5398612 ]]
(50, 50)
0.6356899


In [ ]:
# real 단어평균 + real2 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, real2_sen_doc_emb)
distances

array([[-0.10697618]], dtype=float32)

### real + real3

In [32]:
real3 = real_data['Content'][6]
#진짜 기사 문서
real3_sen_doc = preprocess_sentence(real3, stop_words=stop_words_res, remove_stopwords=True)
#진짜 기사 단어
real3_noun_doc = preprocess_noun(real3, stop_words=stop_words_res, remove_stopwords=True)
real3_noun_doc = real3_noun_doc[0]
real3_noun_word = get_word_list(real3_noun_doc.split(), 50)
#임베딩
real3_sen_doc_emb = model.encode([real3_sen_doc])
print(f'real3_sen_doc_emb.shape : {real3_sen_doc_emb.shape}')
real3_noun_word_emb = model.encode(real3_noun_word)
print(f'real3_noun_word_emb.shape : {real3_noun_word_emb.shape}')

real3_sen_doc_emb.shape : (1, 768)
real3_noun_word_emb.shape : (50, 768)


In [33]:
real3_sen_doc

'동성 으로 하는 진짜 맛집 있다 신문 온라인 은 기반 소셜네트워크서비스 SNS 기업 씨온 SeeOn 손잡고 SNS 에서 인기 있는 맛집 엄선 한다 특정 지역 또는 특정 에서 상위 몇 맛집 추려내는 택했다 SNS 맛 감정 은 매주 수요일 연재 된다 어렸을 퇴근길 손꼽아 기다렸던 는 가끔 씩 사오시던 통닭 이었다 종이 포장 지에 돌돌 말린 전기 구이 통닭 꺼내 살점 발라 깨소금 찍어 먹던 맛 은 잊을 수가 없는 추억 으로 남아있다 어느새 식감 튀김 닭 파는 치킨 동네 어귀 마다 둘 씩 생겨나기 하더니 는 독특한 양념 이나 파 양파 얹은 다양한 맛 으로 진화 하고 있다 SNS 맛 감정 에서는 치킨 하는 치키 니스 를 자처 하는 SNS 이용자 맛집 카레 가득한 광화문 치킨 대명사 거성 호프 광화문 주변 있는 직장인 모르는 없는 거성 호프 수수한 외관 호프 는 광화문 에서 20년 사랑받아 치킨 으로 퇴근길 무렵 양복 입은 직장인 문전 시를 이룬다 으로 들어서면 왁자지껄한 분위기 와 함께 치킨 튀겨지는 고소한 냄새 를 만날 있다 에서 유명한 메뉴 는 튀김 옷 카레 가루 를 넣어 튀겨 낸 카레 치킨 1만 5000원 이다 후라이드 치킨 에서 카레 조금 는 굽는다 함께 내주는 카레 가루 를 찍어 먹으면 카레 돌면서 튀김 옷 어울리는 고소한 맛 일품 이다 동료 와 함께 치킨 생맥주 한잔 으로 하루 피로 를 풀 좋다 남미 닭 요리 를 맛보다 경리 단길 핫 플 레이스 투칸 Tucan 이태원 경리 들어서면 국적 물씬 풍기는 가게 즐비하지만 에서도 단연 가는 은 남미 치킨 맛 볼 있는 투칸 이다 노랑 초록 빨강 색 외부 인테리어 를 있노라면 자메이카 물씬 난다 메뉴 는 캐리비안 저크 치킨 2만 2000원 오렌지 치킨 2만 2000원 이다 저크 치킨 은 자메이카 길거리 음식 으로 드럼통 에서 구워 기름기 쏙 빠진 치킨 맵 짜고 맛 어우러진 양념 맛 독특한 치킨 이다 튀긴 마늘 콤 오렌지 소스 어우러지는 오렌지 치킨 다른 서는 느낄 없는 매력 맛 낸다 여자 만을 있는 치킨 카페 소스 와

In [34]:
# real 문서 + real3 문서
distances = cosine_similarity(real_sen_doc_emb, real3_sen_doc_emb)
distances

array([[0.19507524]], dtype=float32)

In [ ]:
# real 문서 + real3 단어
distances = cosine_similarity(real_sen_doc_emb, real3_noun_word_emb)
# print(distances)
print(np.mean(distances))

-0.0006145464


In [ ]:
# real 문서 + real3 단어평균
real3_noun_word_emb_mean = np.mean(real3_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, real3_noun_word_emb_mean)
distances

array([[0.00201382]], dtype=float32)

In [ ]:
# real 단어 + real3 문서
distances = cosine_similarity(real_noun_word_emb, real3_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.2798932


In [ ]:
# real 단어 + real3 단어
distances = cosine_similarity(real_noun_word_emb, real3_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.38801914 0.6136117  0.56163776 ... 0.58192396 0.4793254  0.5440504 ]
 [0.35139003 0.62129027 0.6479703  ... 0.6214087  0.5341975  0.66815484]
 [0.5794055  0.9238865  0.96155924 ... 0.8354741  0.7554332  0.8938943 ]
 ...
 [0.3607105  0.6705898  0.6943643  ... 0.5132244  0.5108589  0.5972169 ]
 [0.53705597 0.9306347  0.96924573 ... 0.8453681  0.73975897 0.9064038 ]
 [0.3764831  0.7250231  0.74968684 ... 0.6022258  0.548749   0.70283115]]
(50, 50)
0.6933787


In [ ]:
# real 단어평균 + real3 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, real3_sen_doc_emb)
distances

array([[0.32789484]], dtype=float32)

### real + fake2

In [ ]:
fake2 = fake_data['Content'][2]
#가짜 기사 문서
fake2_sen_doc = preprocess_sentence(fake2, stop_words=stop_words_res, remove_stopwords=True)
#가짜 기사 단어
fake2_noun_doc = preprocess_noun(fake2, stop_words=stop_words_res, remove_stopwords=True)
fake2_noun_doc = fake2_noun_doc[0]
fake2_noun_word = get_word_list(fake2_noun_doc.split(), 50)
#임베딩
fake2_sen_doc_emb = model.encode([fake2_sen_doc])
print(f'fake2_sen_doc_emb.shape : {fake2_sen_doc_emb.shape}')
fake2_noun_word_emb = model.encode(fake2_noun_word)
print(f'fake2_noun_word_emb.shape : {fake2_noun_word_emb.shape}')

fake2_sen_doc_emb.shape : (1, 768)
fake2_noun_word_emb.shape : (50, 768)


In [ ]:
fake2_sen_doc

'당당하게 다녀오십시오 위대한 대통령 이었다 되시리라 굳게 믿고 있습니다 14년 만에 전직 대통령 검찰 소환 되는 불 상사 에서도 노무현 대통령 지지자 충심 은 흔들림이 없었다 오히려 조사 를 계기 은 똘똘 뭉치 있었다 지지자 은 대통령 개인 홈페이지 람사 는 세상 www knowhow or kr 팬클럽 노사모 홈페이지 www nosamo org 대통령 지지 사이트 진보 계열 홈페이지 힘내세요 잘 되실 겁니다 격려 글 잇 올렸다 지지자 은 홈페이지 대검찰청 앞 모여 인간 띠 잇기 촛불집회 퍼포먼스 를 벌이 는 주장 까지 펼치며 상경 하는 대통령 맞이 준비 까지 하고 있었다 람사 는 세상 은 30일 자정 8시 까지 200 글 올라왔다 람사 는 세상 에는 하늘 이여 땅 이여 바람 이여 노무현 대통령 님 지켜주소서 당당하신 으로 안녕히 다녀 오십시요 대통령 무사 귀가 를 바라는 들의 지지자 글 속속 올라오고 있다 ID 연등 엄마 는 영국 토마스 무어 는 진실하고 충직 신하 는 세상 보다도 영혼 양심 존중 해야 의무 있다 했습니다 흔들리지 않는 신념 처럼 노공 님 진실 믿고 있습니다 몸 은 영국 있지만 으로 응원 하겠다 했다 30일 경남 김해시 봉하마을 노무현 대통령 앞 에서 노사모 회원 마을 주민 검찰 조사 항의 하는 집회 를 열고 있다 정희 checho 노사모 홈페이지 였다 ID 남녀평등 은 는 에게 허물 있다하더라도 허물 이다 최후 지지자 남고 싶다 그런데 만이 아니라서 기쁘다 했다 퍼포먼스 집단 행동 제안 하는 게시 꾸준히 올라왔다 지지자 은 대통령 마중 나가겠다며 옷 따 뜻 입고 서초동 으로 오십 시오 는 현수막 준비 했습니다 30일 하루 휴가 냈습니다 짱 뵈려 글 올렸다 특히 다른 없이 30일 1시 서초동 대검찰청 이라고만 적힌 글 상당수 있었고 어떻게 해야 되나요 지침 묻는 글 있어 지지자 끼리 암 묵적 행동 준비 하고 있다는 해석 나오고 있다 ID average 는 김해 에서 서초동 까지 짱 가는 노란 풍선 대통령 상징물 날리고 꽃가루 를 뿌려주자 했다

In [ ]:
# real 문서 + fake2 문서
distances = cosine_similarity(real_sen_doc_emb, fake2_sen_doc_emb)
distances

array([[0.17376286]], dtype=float32)

In [ ]:
# real 문서 + fake2 단어
distances = cosine_similarity(real_sen_doc_emb, fake2_noun_word_emb)
print(distances)
print(np.mean(distances))

[[-0.08310185 -0.0351185  -0.04733943 -0.10416343  0.00614846 -0.0782207
  -0.04537811 -0.11483335 -0.04039231 -0.11964837 -0.12976694 -0.04096713
  -0.02520191 -0.09836115  0.02616425 -0.04146113 -0.06150378 -0.06262823
  -0.05823022 -0.02217892 -0.09984235 -0.12253711 -0.0576148   0.00939622
  -0.06965636 -0.1173337  -0.04618426 -0.0505487  -0.07403804  0.0008467
  -0.03770257 -0.07201992 -0.17136462 -0.10013543  0.02445152 -0.09279731
   0.04051017 -0.07745772 -0.10249421 -0.07948956 -0.10494762 -0.02234918
  -0.0515094  -0.02457418 -0.09341893 -0.07993718 -0.06618746 -0.08883347
  -0.07630937 -0.11237746]]
-0.06385278


In [ ]:
# real 문서 + fake2 단어평균
fake2_noun_word_emb_mean = np.mean(fake2_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, fake2_noun_word_emb_mean)
distances

array([[-0.07943976]], dtype=float32)

In [ ]:
# real 단어 + fake2 문서
distances = cosine_similarity(real_noun_word_emb, fake2_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.20705435


In [ ]:
# real 단어 + fake2 단어
distances = cosine_similarity(real_noun_word_emb, fake2_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.39693317 0.33456367 0.5558324  ... 0.49338192 0.4400061  0.5024327 ]
 [0.4769457  0.37739033 0.48473686 ... 0.578802   0.5952736  0.6252861 ]
 [0.6058428  0.5112804  0.552337   ... 0.91737336 0.8105808  0.950546  ]
 ...
 [0.4223467  0.6646833  0.3494315  ... 0.67996645 0.5009949  0.67471987]
 [0.5875702  0.5541171  0.56292284 ... 0.936882   0.806992   0.9667089 ]
 [0.35191974 0.6436748  0.45197088 ... 0.7229068  0.6500691  0.7015877 ]]
(50, 50)
0.6748712


In [ ]:
# real 단어평균 + fake2 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, fake2_sen_doc_emb)
distances

array([[0.24578302]], dtype=float32)

# 문장 데이터 (불용어 제거 X)

단어 리스트는 불용어 제거 명사만 50개

## 변수 정의

In [35]:
#진짜 기사 문서
real_sen_doc = preprocess_sentence(real)
real_sen_doc

'홍익대학교 정문 에서 극동방송국 방향 으로 내려오는 큰길 뒷골목 이 곳 은 아지트 같은 카페 음식점 들 이 길 양옆 에 옹기종기 모여 있는 또 다른 맛집 골목 이다 독특한 상호 만큼이나 독특한 콘셉트 로 유명한 는 이 골목 에서 가장 잘 알려진 음식점 중 하나 2001년 문 은 연 이 곳 은 미군 용 식기 와 수통 에 마시는 맥주 군대 막사 와 같은 밀리터리 콘셉트 로 주목 을 끌며 꾸준한 인기 를 얻고 있다 그러나 이러한 분위기 보다 더 인기 를 끌 고 있는 것 이 매운 닭 볶음 인 오리 지날 철판 1만 4000원 이다 알싸한 맛 의 중독 성 때문 인지 를 대표 하는 메뉴 로 자리 잡고 있다 오리 지날 철판 과 더불어 눈길 을 끄는 것 이 알밥 2000원 날치 알 이 들어간 알밥 이 아니라 말 그대로 밥 안 에 알 이 들어있다 잘 뭉친 밥 위 에 김 가루 와 깨 가 송송 뿌려져 제공 되는 이 메뉴 는 삶은 달걀 이 통째 로 밥 안 에 들어간 이색 적 인 사이드 메뉴 다 감칠맛 나 는 오 리지 널 철판 의 매운 소스 는 밥 은 물론 삶은 달걀 과도 좋은 궁합 을 이루며 2 부족한 고객 들 의 배 를 채우는데 일조 하고 있다 알밥 은 사이드 메뉴 지만 임팩트 있는 메뉴 개발 로 인해 최고 의 성공 키 워드 로 자리매 김 하고 있다'

In [ ]:
#진짜 기사 단어
real_noun_doc = preprocess_noun(real, stop_words=stop_words_res, remove_stopwords=True)
real_noun_doc = real_noun_doc[0]
real_noun_word = get_word_list(real_noun_doc.split(), 50)

## 임베딩 (불용어 처리, 명사)

In [ ]:
real_sen_doc_emb = model.encode([real_sen_doc])
print(f'real_sen_doc_emb.shape : {real_sen_doc_emb.shape}')
real_noun_word_emb = model.encode(real_noun_word)
print(f'real_noun_word_emb.shape : {real_noun_word_emb.shape}')

real_sen_doc_emb.shape : (1, 768)
real_noun_word_emb.shape : (50, 768)


## 코사인 유사도 (불용어 처리, 문장)

### real + fake

In [ ]:
fake = fake_data['Content'][1]
#가짜 기사 문서
fake_sen_doc = preprocess_sentence(fake)
#가짜 기사 단어
fake_noun_doc = preprocess_noun(fake, stop_words=stop_words_res, remove_stopwords=True)
fake_noun_doc = fake_noun_doc[0]
fake_noun_word = get_word_list(fake_noun_doc.split(), 50)
#임베딩
fake_sen_doc_emb = model.encode([fake_sen_doc])
print(f'fake_sen_doc_emb.shape : {fake_sen_doc_emb.shape}')
fake_noun_word_emb = model.encode(fake_noun_word)
print(f'fake_noun_word_emb.shape : {fake_noun_word_emb.shape}')

fake_sen_doc_emb.shape : (1, 768)
fake_noun_word_emb.shape : (50, 768)


In [ ]:
fake_sen_doc

'한국 은 좁다 일본 에 이어 미국 진출 세계 로 뻗어 나가는 갤럭시 익스 프레 스 갤럭시 익스 프레 스 는 우주 를 쾌속 으로 질주 한다는 의미 우리 의 음악 이 우주 로 뻗어 나가는 전파 가 되어 개개인 의 소우주 를 여행 할 수도 있다 우리 개인 이 세상 의 만물 을 만날 수 는 없을지라도 우리 음악 은 언젠가 당신 을 만나게 될지도 데뷔 2006년 EP 투 더 갤럭시 To the galaxy 소속 사러 브락 컴퍼니 Loverock Company 장르 포스트 펑크 와 사이키 델리 아 멤버 구 성현 멤버 이주현 베이스 보컬 김희 권 드럼 코러스 보컬 박종현 기타 보컬 전 멤버 윤홍구 드럼 탈퇴 앨범 2006 EP 투 더 갤럭시 To the galaxy 2007 EP 램블 어 라운드 Ramble around 2008 정규 1 집 노이즈 온 파이어 Noise on fire 2009 EP 컴온 겟업 Come on Get up 2010 정규 2 집 와일드 데이즈 Wild days 2011 EP 개구쟁이 with 크라잉 넛 대표 곡정 글더 블랙 Jungle the black 난 어디 로 진짜 너 를 원해 오예 지나고 나면 언제나 좋았어 개인 이력 이주현 35 자살 매미 와 럭스 RUX 게토밤즈 Ghetto bombs 활동 그림 에 재주 가 있어 아트워크 담당 김희 권 31 양구 초등학교 졸업 양구 중학교 졸업 양구 종합 고등학교 졸업 박종현 31 5 사단 육군 병장 제대 수상 내 역 2008 Mnet KM 뮤직페스티벌 올해 의 발견 상 수상 2009년 대중음악 상 최우수 음반 노이즈 온 파이어 Noise on fire 2011년 대중음악 상 올해 의 음악인 상 Music Alliance Pacts 가 선정 한 2012년 1월 각국 대표 아티스트 중 한국 의 아티스트 로 선정 취미 특기 이주현 그림 그리기 영화 감상 명상 사색 운전면허증 김희 권 술 담배 게임 욕 하기 박종현 술 먹고 진상 부리기 홍대 근처 의 출몰 지역 은 이주현 망원동 성산 초등학교 앞 에 커피 

In [ ]:
# real 문서 + fake 문서
distances = cosine_similarity(real_sen_doc_emb, fake_sen_doc_emb)
distances

array([[0.24130021]], dtype=float32)

In [ ]:
# real 문서 + fake 단어
distances = cosine_similarity(real_sen_doc_emb, fake_noun_word_emb)
print(distances)
print(np.mean(distances))

[[ 0.12049864  0.00395992 -0.0494884   0.03970939  0.03123921  0.01038969
  -0.02076998 -0.00663808  0.05416141 -0.0464213   0.00039423  0.05188087
  -0.00287432  0.08438845 -0.00032584 -0.04350725  0.01077064  0.01682213
   0.04455601  0.0205803  -0.01367706 -0.02130404  0.00286268 -0.02361947
   0.02764408  0.02533836  0.01416451  0.07203214  0.16290969  0.00312584
   0.0027932  -0.02429359 -0.05089107 -0.02245137 -0.03405081  0.01105261
  -0.08749247  0.00365541  0.06259353  0.15378505 -0.02781199 -0.06271064
   0.01320038  0.02166385 -0.0067621   0.06226701  0.04039805  0.00092434
   0.02637848  0.11673979]]
0.015355801


In [ ]:
# real 문서 + fake 단어평균
fake_noun_word_emb_mean = np.mean(fake_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, fake_noun_word_emb_mean)
distances

array([[0.0195419]], dtype=float32)

In [ ]:
# real 단어 + fake 문서
distances = cosine_similarity(real_noun_word_emb, fake_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.19508074


In [ ]:
# real 단어 + fake 단어
distances = cosine_similarity(real_noun_word_emb, fake_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.5972948  0.43616983 0.45565605 ... 0.48576665 0.50948596 0.2621813 ]
 [0.6236847  0.5433911  0.462256   ... 0.43141812 0.51718175 0.32967314]
 [0.8366065  0.75025225 0.65857625 ... 0.5748139  0.70239294 0.4087414 ]
 ...
 [0.6295496  0.49643505 0.46876863 ... 0.3336129  0.5156909  0.24603473]
 [0.8746172  0.7526971  0.6673922  ... 0.5915314  0.70799124 0.42351407]
 [0.6377727  0.54896575 0.54767907 ... 0.4638309  0.5162641  0.31390324]]
(50, 50)
0.6414043


In [ ]:
# real 단어평균 + fake 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, fake_sen_doc_emb)
distances

array([[0.23053908]], dtype=float32)

### real + real2

In [36]:
real2 = real_data['Content'][3]
#진짜 기사 문서
real2_sen_doc = preprocess_sentence(real2)
#진짜 기사 단어
real2_noun_doc = preprocess_noun(real2, stop_words=stop_words_res, remove_stopwords=True)
real2_noun_doc = real2_noun_doc[0]
real2_noun_word = get_word_list(real2_noun_doc.split(), 50)
#임베딩
real2_sen_doc_emb = model.encode([real2_sen_doc])
print(f'real2_sen_doc_emb.shape : {real2_sen_doc_emb.shape}')
real2_noun_word_emb = model.encode(real2_noun_word)
print(f'real2_noun_word_emb.shape : {real2_noun_word_emb.shape}')

real2_sen_doc_emb.shape : (1, 768)
real2_noun_word_emb.shape : (50, 768)


In [37]:
real2_sen_doc

'치킨 요리 를 주문 하기 전 프라이드 를 먹을까 양념 을 먹을까 하는 고민 은 이제 거둘 것 지금 홍대 앞 에는 크림소스 에 재운 치킨 매콤 한 철판 치킨 등 다양한 치킨 메뉴 로 무장 한 맛집 이 성황 을 이루고 있다 치르 치르 여자 들 이 좋아할 만 한 다양한 치킨 메뉴 가 있는 곳 일반 치킨 집 에 있는 무 대신 자체 개발 한 양파 소스 를 내어주는데 양파 토마토 등 의 채소 와 레몬주스 를 섞어 만든 것 으로 치킨 맛 을 돋워 주는 치르 치르만의 자랑거리 치킨 을 쉽게 잡 을 수 있도록 디자인 된 완소 집게 와 치킨 집 에서 막걸리 를 파는 의외 성 또한 손님 을 즐겁게 해주는 요소 다 DATA 위치 상상마당 건너편 골목 안 영업 시간 오후 3시 다음 날 오전 5시 문의 02 332 3187 둥지 철 파닭 파 를 둥지 처럼 둥그런 모양 으로 만든 뒤 그 위 에 닭 가슴 살 캐슈 너트 마늘 간장 소스 를 올린 요리 1만 6천 9 백원 몸짱 치킨 기름 을 쏙 뺀 전기 구이 치킨 요리 로 담백하면서도 부드러운 맛 이 일품 다이어트 에 민감한 여성 손님 에게 인기 가 많다 1만 3천 9 백원 뿔레 치킨 주인 이 해외여행 을 다니다 하나 둘 모은 소품 으로 꾸민 이 국적 인 인테리어 와 이색 적 인 메뉴 로 여성 손님 이 80 90 를 차지 하는 곳 아이 와 함께 가족 단위 로 방문 하는 손님 들 도 많다 오후 3시 부터 5시 까지는 치킨 요리 와 함께 샐러드 포테이토칩 어니언 링 이 함께 나오는 세트 메뉴 를 단 품 가격 에 판매 한다 DATA 위치 홍대 놀이터 옆 오르막 길 골목 영업 시간 오후 3시 다음 날 오전 1시 30분 문의 02 322 4870 스위트 칠리 뿔레 매콤 하면서도 새 콤 달콤한 스위트 칠리소스 가 치킨 의 풍 미를 더하는 메뉴 소스 가 강하지 않아 매운 음식 을 즐기지 않는 사람 에게도 권할 만하 다 1만 6천원 까르보나라 뿔레 뿔레치킨 을 홍대 맛집 으로 만들어 준 일등 공신 으로 크림소스 에 푹 재운 보들보들한 순 살이 별미 다 1

In [ ]:
# real 문서 + real2 문서
distances = cosine_similarity(real_sen_doc_emb, real2_sen_doc_emb)
distances

array([[0.44269645]], dtype=float32)

In [ ]:
# real 문서 + real2 단어
distances = cosine_similarity(real_sen_doc_emb, real2_noun_word_emb)
print(distances)
print(np.mean(distances))

[[ 0.11226086  0.31245655  0.26957607  0.10699594 -0.02637366  0.02764407
  -0.01706921  0.06444563 -0.05092401  0.11000259 -0.05235226 -0.04776013
  -0.00073952  0.0564411   0.04282162 -0.04849252  0.01980093  0.00255069
  -0.01755132  0.0820688   0.03346486  0.2580522   0.12552926  0.02117665
   0.13295004  0.07023036 -0.05691622  0.13006923  0.11272582 -0.00492412
   0.01099346  0.27845615 -0.01587938  0.06798634  0.16556318 -0.00988286
   0.05897494 -0.0677949   0.02957354  0.18889247  0.00981443 -0.04985036
  -0.06822044 -0.00511479 -0.03335257  0.06127655  0.12459795  0.16403669
  -0.04424261 -0.04713179]]
0.051737126


In [ ]:
# real 문서 + real2 단어평균
real2_noun_word_emb_mean = np.mean(real2_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, real2_noun_word_emb_mean)
distances

array([[0.07118239]], dtype=float32)

In [ ]:
# real 단어 + real2 문서
distances = cosine_similarity(real_noun_word_emb, real2_sen_doc_emb)
# print(distances)
print(np.mean(distances))

-0.18967023


In [ ]:
# real 단어 + real2 단어
distances = cosine_similarity(real_noun_word_emb, real2_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.38801894 1.0000002  0.6034899  ... 0.37387735 0.39919356 0.39063752]
 [0.35138983 0.42752993 0.4433522  ... 0.46267188 0.5112967  0.5003484 ]
 [0.5794052  0.5357519  0.43698254 ... 0.597479   0.48117152 0.7547186 ]
 ...
 [0.36071032 0.35069203 0.36615738 ... 0.86102074 0.32716727 0.5541719 ]
 [0.53705555 0.5331935  0.45526278 ... 0.6218757  0.4859876  0.7742833 ]
 [0.37648278 0.38964963 0.3742798  ... 0.56141406 0.4251297  0.5398612 ]]
(50, 50)
0.6356899


In [ ]:
# real 단어평균 + real2 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, real2_sen_doc_emb)
distances

array([[-0.22541168]], dtype=float32)

### real + real3

In [38]:
real3 = real_data['Content'][6]
#진짜 기사 문서
real3_sen_doc = preprocess_sentence(real3)
#진짜 기사 단어
real3_noun_doc = preprocess_noun(real3, stop_words=stop_words_res, remove_stopwords=True)
real3_noun_doc = real3_noun_doc[0]
real3_noun_word = get_word_list(real3_noun_doc.split(), 50)
#임베딩
real3_sen_doc_emb = model.encode([real3_sen_doc])
print(f'real3_sen_doc_emb.shape : {real3_sen_doc_emb.shape}')
real3_noun_word_emb = model.encode(real3_noun_word)
print(f'real3_noun_word_emb.shape : {real3_noun_word_emb.shape}')

real3_sen_doc_emb.shape : (1, 768)
real3_noun_word_emb.shape : (50, 768)


In [39]:
real3_sen_doc

'여기 사람 들 이 이 구 동성 으로 추천 하는 진짜 맛집 이 있다 한국 경제 신문 온라인 미디어 은 위치 기반 소셜네트워크서비스 SNS 전문 기업 씨온 SeeOn 과 손잡고 SNS 에서 가장 인기 있는 맛집 을 엄선 한다 특정 지역 또는 특정 테마 에서 상위 몇 개 맛집 을 추려내는 작업 을 택했다 SNS 맛 감정 단 은 매주 수요일 연재 된다 어렸을 때 아버지 의 퇴근길 을 손꼽아 기다렸던 이유 중 하나 는 가끔 씩 사오시던 통닭 때문 이었다 종이 포장 지에 돌돌 말린 전기 구이 통닭 을 꺼내 살점 을 발라 깨소금 에 찍어 먹던 그 맛 은 잊을 수가 없는 추억 으로 남아있다 어느새 바삭 한 식감 의 튀김 닭 을 파는 치킨 집 이 동네 어귀 마다 하나 둘 씩 생겨나기 시작 하더니 이제 는 독특한 양념 이나 파 양파 등 을 얹은 다양한 맛 으로 진화 하고 있다 이번 주 SNS 맛 감정 단 에서는 치킨 을 사랑 하는 치키 니스 트 를 자처 하는 SNS 이용자 들 이 직접 추천 한 맛집 을 소개 한 다 카레 향 가득한 광화문 치킨 의 대명사 거성 호프 광화문 주변 에 있는 직장인 이 라면 모르는 이 가 없는 거성 호프 수수한 외관 의 이 호프 는 광화문 에서 20년 간 사랑받아 온 치킨 집 으로 퇴근길 무렵 이면 양복 을 입은 직장인 들 로 문전 성 시를 이룬다 안 으로 들어서면 왁자지껄한 분위기 와 함께 치킨 이 튀겨지는 고소한 냄새 를 만날 수 있다 이 집 에서 가장 유명한 메뉴 는 튀김 옷 에 카레 가루 를 넣어 튀겨 낸 카레 치킨 1만 5000원 이다 후라이드 치킨 에서 카레 향 이 조금 나 는 정도 로 굽는다 함께 내주는 카레 가루 를 찍어 먹으면 살짝 카레 향 이 돌면서 튀김 옷 과 어울리는 고소한 맛 이 일품 이다 동료 와 함께 치킨 과 생맥주 한잔 으로 하루 의 피로 를 풀 면 좋다 남미 식 닭 요리 를 맛보다 경리 단길 핫 플 레이스 투칸 Tucan 이태원 경리 단 길 을 들어서면 이 국적 인 느낌 이 물씬 풍기는 가게 들 이 즐비하지만 그 중 에

In [ ]:
# real 문서 + real3 문서
distances = cosine_similarity(real_sen_doc_emb, real3_sen_doc_emb)
distances

array([[0.5510278]], dtype=float32)

In [ ]:
# real 문서 + real3 단어
distances = cosine_similarity(real_sen_doc_emb, real3_noun_word_emb)
print(distances)
print(np.mean(distances))

[[ 0.11226091  0.10699594  0.02503694  0.258052   -0.01311647  0.04098339
   0.0564411   0.03978263  0.00336235  0.03303282  0.13006923  0.21550503
   0.18889247  0.09925801 -0.02788546  0.31245655  0.01371204  0.02892463
  -0.03176348 -0.02129524 -0.03583917  0.01073527  0.05982652 -0.0265923
   0.02557372  0.17883572 -0.00286904  0.02134237  0.05622531  0.26957607
  -0.05195207  0.00328101 -0.00782393  0.11272582  0.00964114 -0.02098157
   0.07280927  0.1233884  -0.05691622  0.00882453  0.17824961  0.06798634
  -0.00100639  0.04317921  0.02238085 -0.00306908  0.01980095  0.04282175
   0.32981455  0.03322939]]
0.06107806


In [ ]:
# real 문서 + real3 단어평균
real3_noun_word_emb_mean = np.mean(real3_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, real3_noun_word_emb_mean)
distances

array([[0.07545798]], dtype=float32)

In [ ]:
# real 단어 + real3 문서
distances = cosine_similarity(real_noun_word_emb, real3_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.06283762


In [ ]:
# real 단어 + real3 단어
distances = cosine_similarity(real_noun_word_emb, real3_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.38801914 0.6136117  0.56163776 ... 0.58192396 0.4793254  0.5440504 ]
 [0.35139003 0.62129027 0.6479703  ... 0.6214087  0.5341975  0.66815484]
 [0.5794055  0.9238865  0.96155924 ... 0.8354741  0.7554332  0.8938943 ]
 ...
 [0.3607105  0.6705898  0.6943643  ... 0.5132244  0.5108589  0.5972169 ]
 [0.53705597 0.9306347  0.96924573 ... 0.8453681  0.73975897 0.9064038 ]
 [0.3764831  0.7250231  0.74968684 ... 0.6022258  0.548749   0.70283115]]
(50, 50)
0.6933787


In [ ]:
# real 단어평균 + real3 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, real3_sen_doc_emb)
distances

array([[0.07546151]], dtype=float32)

### real + fake2

In [ ]:
fake2 = fake_data['Content'][2]
#가짜 기사 문서
fake2_sen_doc = preprocess_sentence(fake2)
#가짜 기사 단어
fake2_noun_doc = preprocess_noun(fake2, stop_words=stop_words_res, remove_stopwords=True)
fake2_noun_doc = fake2_noun_doc[0]
fake2_noun_word = get_word_list(fake2_noun_doc.split(), 50)
#임베딩
fake2_sen_doc_emb = model.encode([fake2_sen_doc])
print(f'fake2_sen_doc_emb.shape : {fake2_sen_doc_emb.shape}')
fake2_noun_word_emb = model.encode(fake2_noun_word)
print(f'fake2_noun_word_emb.shape : {fake2_noun_word_emb.shape}')

fake2_sen_doc_emb.shape : (1, 768)
fake2_noun_word_emb.shape : (50, 768)


In [ ]:
fake2_sen_doc

'당당하게 다녀오십시오 위대한 대통령 이었다 고 기록 되시리라 굳게 믿고 있습니다 14년 만에 전직 대통령 이 검찰 로 소환 되는 불 상사 속 에서도 노무현 전 대통령 지지자 들 의 충심 은 흔들림이 없었다 오히려 이번 조사 를 계기 로 이 들 은 더 똘똘 뭉치 고 있었다 지지자 들 은 노 전 대통령 의 개인 홈페이지 사 람사 는 세상 www knowhow or kr 팬클럽 노사모 홈페이지 www nosamo org 등 노 대통령 지지 사이트 나 진보 계열 홈페이지 에 힘내세요 잘 되실 겁니다 등 의 격려 글 을 잇 달 아 올렸다 일부 지지자 들 은 홈페이지 에 대검찰청 앞 에 모여 인간 띠 잇기 촛불집회 등 의 퍼포먼스 를 벌이 자 는 주장 까지 펼치며 상경 하는 노 전 대통령 을 맞이 할 준비 까지 하고 있었다 사 람사 는 세상 은 30일 자정 부터 오전 8시 까지 200 개 의 지 지 글 이 올라왔다 사 람사 는 세상 에는 하늘 이여 땅 이여 바람 이여 노무현 대통령 님 을 지켜주소서 당당하신 모습 으로 안녕히 다녀 오십시요 등 노 전 대통령 의 무사 귀가 를 바라는 들의 지지자 들 의 글 이 속속 올라오고 있다 ID 연등 엄마 는 영국 의 토마스 무어 는 진실하고 충직 한 신하 는 이 세상 무엇 보다도 자신 의 영혼 과 양심 을 존중 해야 할 의무 가 있다 고 했습니다 흔들리지 않는 그 의 신념 처럼 노공 님 의 진실 을 믿고 있습니다 몸 은 영국 에 있지만 마음 으로 응원 하겠다 고 했다 30일 오전 경남 김해시 봉하마을 노무현 전 대통령 사 저 앞 에서 노사모 회원 과 마을 주민 이 검찰 조사 에 항의 하는 집회 를 열고 있다 정희 조 기자 checho 노사모 홈페이지 도 마찬가지 였다 ID 남녀평등 사랑 은 나 는 그 분 에게 허물 이 있다하더라도 그 허물 도 사랑 할 것 이다 최후 의 지지자 로 남고 싶다 그런데 나 만이 아니라서 기쁘다 고 했다 퍼포먼스 등 집단 행동 을 제안 하는 게시 물 도 꾸준히 올라왔다 일부 지지자 들 은 노 전 대통령 

In [ ]:
# real 문서 + fake2 문서
distances = cosine_similarity(real_sen_doc_emb, fake2_sen_doc_emb)
distances

array([[0.17501403]], dtype=float32)

In [ ]:
# real 문서 + fake2 단어
distances = cosine_similarity(real_sen_doc_emb, fake2_noun_word_emb)
print(distances)
print(np.mean(distances))

[[ 0.02758376  0.01497631  0.0584936  -0.05089107  0.04507381 -0.02428807
  -0.0089454  -0.02628295  0.05495142 -0.0687627  -0.05950172  0.03349109
   0.05272583 -0.01109832  0.10116701  0.07232007 -0.02889671 -0.04256152
   0.01902038  0.0153268  -0.00512215 -0.03701708  0.02188087  0.06206141
  -0.02335873 -0.03281186  0.01149105  0.03874969 -0.02429365  0.0813647
   0.03271072  0.02081029 -0.08665273 -0.0202909   0.05781415 -0.0424256
   0.13191254 -0.04279641 -0.02435432  0.01073128 -0.02499141  0.05598513
   0.02816861  0.02927162 -0.02003243 -0.0495671   0.06288093 -0.0294447
  -0.04334538 -0.04605011]]
0.0053436016


In [ ]:
# real 문서 + fake2 단어평균
fake2_noun_word_emb_mean = np.mean(fake2_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, fake2_noun_word_emb_mean)
distances

array([[0.00691704]], dtype=float32)

In [ ]:
# real 단어 + fake2 문서
distances = cosine_similarity(real_noun_word_emb, fake2_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.001194699


In [ ]:
# real 단어 + fake2 단어
distances = cosine_similarity(real_noun_word_emb, fake2_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.39693317 0.33456367 0.5558324  ... 0.49338192 0.4400061  0.5024327 ]
 [0.4769457  0.37739033 0.48473686 ... 0.578802   0.5952736  0.6252861 ]
 [0.6058428  0.5112804  0.552337   ... 0.91737336 0.8105808  0.950546  ]
 ...
 [0.4223467  0.6646833  0.3494315  ... 0.67996645 0.5009949  0.67471987]
 [0.5875702  0.5541171  0.56292284 ... 0.936882   0.806992   0.9667089 ]
 [0.35191974 0.6436748  0.45197088 ... 0.7229068  0.6500691  0.7015877 ]]
(50, 50)
0.6748712


In [ ]:
# real 단어평균 + fake2 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, fake2_sen_doc_emb)
distances

array([[0.00851253]], dtype=float32)

# -----다른 기업1 고래서점-----

In [ ]:
# filtered.groupby('기업명')['Content'].count().sort_values(ascending=False).to_csv('filtered.txt')
# remained.groupby('기업명')['Content'].count().sort_values(ascending=False).to_csv('remained.txt')
# full = pd.concat([filtered, remained], axis=0)
# full.groupby('기업명')['Content'].count().sort_values(ascending=False).to_csv('full.txt')

In [ ]:
filtered[filtered['기업명']=='고래서점']

,기업명,Title,Content,c_index,c_name
1683,고래서점,헌책방 강자 된 인터넷서점…중고책 사들여 재판매,지난 7월 3일 수요일 오후 3시 알라딘 중고서점 종로점. 일반 서점으로 치면 가장...,45,도소매업
1684,고래서점,오래된 책의 향기 온라인에서도 지식의 내음을…,[머니투데이 강동완 기자][중고 서적 전문 쇼핑몰 고래서점]\n“최고가 될 자신이 ...,45,도소매업


In [ ]:
remained[remained['기업명']=='고래서점']

,기업명,Title,Content,c_index,c_name
18993,고래서점,[취재] 사진책방이 살아난다,"단순히 사진책 파는 그 이상, 북큐레이팅 북토크 워크숍…\n\n\n이색적인 사진 전...",45,도소매업
18994,고래서점,<뒤늦게 깨달은 가족의 따뜻함>,연극 '대빵 큰 고래의 꿈'\n\n(서울=연합뉴스) 현윤경 기자 = 연말을 맞아 가...,45,도소매업
18995,고래서점,10년앞 보며 온라인으로 걸음마,[한겨레] 헌책방 순례/고래서점\n\n고래서점은 세 군데다. 용산구 보광동 오산학교...,45,도소매업
18996,고래서점,[피플&피플]“우리 새로운 경력은 강릉의 감성 파는 일”,[피플&피플]“우리 새로운 경력은 강릉의 감성 파는 일” ◇강릉 스타트업 `아이오프...,45,도소매업


In [ ]:
filtered['Content'][1684]

In [ ]:
remained['Content'][18994]

In [ ]:
real_data = pd.DataFrame({'Content':[filtered['Content'][1684], remained['Content'][18995]]})
fake_data = pd.DataFrame({'Content':[remained['Content'][18994], remained['Content'][18996]]})

In [ ]:
real_data

,Content
0,[머니투데이 강동완 기자][중고 서적 전문 쇼핑몰 고래서점]\n“최고가 될 자신이 ...
1,[한겨레] 헌책방 순례/고래서점\n\n고래서점은 세 군데다. 용산구 보광동 오산학교...


In [ ]:
fake_data

,Content
0,연극 '대빵 큰 고래의 꿈'\n\n(서울=연합뉴스) 현윤경 기자 = 연말을 맞아 가...
1,[피플&피플]“우리 새로운 경력은 강릉의 감성 파는 일” ◇강릉 스타트업 `아이오프...


In [ ]:
real = real_data['Content'][0]
fake = fake_data['Content'][1]

In [ ]:
real

'[머니투데이 강동완 기자][중고 서적 전문 쇼핑몰 고래서점]\n“최고가 될 자신이 없다면 새로운 분야를 개척해야죠.”\n중고 서적 전문 쇼핑몰 고래서점[www.gorebook.co.kr]은 동화책부터 소설, 인문, 철학, 전문 서적까지 약 10만권의 장서를 보유한 ‘온라인 헌책방’이다.\n고래서점 남동일 대표(39)가 처음 온라인 중고책 서점 창업을 생각한 것은 전 직장에서였다. 대학 때 일어 전공을 했던 남대표의 전직은 의료기기 회사 일본 지사장. 일본 현지에서 중고 서적의 취급 현장을 직접 확인했고, 국내 시장에서 중고책 사업 운영 가능성을 점칠 수 있었다.\n첫 사업을 시작한 것은 2002년 서울, 처음 ‘헌책’을 구할 때 사람들은 거래 상대가 아닌 파지를 모으는 사람으로 오해 한적도 있다고. 한번은 이걸로 돈 벌이나 되냐고 딱하게 쳐다본 사람들도 있었다.\n반대로 남대표의 중고책에 대한 자세에 감동을 받은 사람도 있었다. 판매자에게 비용을 지불하고 포장을 했는데 책을 조심스럽게 다루는 모습을 보고 앞으로 종종 이용 하겠다고 한 사람도 있었다.\n이런 노력으로 고래서점은 양질의 중고책을 확보 할 수 있었다. 중고책을 먼저 팔겠다고 하는 사람들도 늘었고, 보다 관리가 잘된 책을 많이 구할 수 있었다. 고래서점은 중고책을 매입 할 때 택배비를 대신 부담 했고, 가까운 거리는 직접 방문했다.\n‘헌책이라도 더 좋은 상태로 구할 수 있다’란 입소문이 독서광들에게 퍼지면서 오픈마켓에서 ‘고래서점’의 이름을 도용하는 일도 있었다. 이런 과정 속에서 단골들이 늘었다. 현재 고래서점은 절판 도서를 찾는 사람들이 많다.\n특히 다양한 분야의 책을 보유하고 있어 관심 분야의 책이나 혹은 다시 보고 싶은 책, 소장용 책 등을 찾는 사람들이 많다.\n고래서점은 온라인으로 중고책을 구하려는 고객의 불편함을 최소화시키는데 중점을 두고 있다. 보유한 책은 상태에 따라 자체 기준으로 ‘상,중,하’ 등급 별로 분류해 놓고 있다. 훼손 상태가 심한 도서는 취급하지 않고, 세세한 책 상

In [ ]:
fake

"[피플&피플]“우리 새로운 경력은 강릉의 감성 파는 일” ◇강릉 스타트업 `아이오프너'의 최하얀(사진 왼쪽부터)·백은경·임혜정·서성정씨.\n\n\n스타트업 `아이오프너'\n\n주문진해변의 BTS 버스정류장, 하평해변의 핑크뮬리, 안목해변의 커피자판기 등 강릉의 해변을 배경으로 만든 여행상품으로 히트 치고 있는 스타트업 `아이오프너'.\n\n아이오프너는 결혼과 육아 등으로 다니던 직장을 그만두고 강릉에 정착한 경력단절여성(경단녀)들이 만든 스타트업이다.\n\nKBS 콘텐츠 기획PD였지만 아이를 키우기 위해 직장을 그만둔 임혜정(41) 대표와 대학교 조교생활을 하다가 일본으로 유학을 떠났지만 결혼과 함께 강릉에 정착한 백은경(41)씨, 경기 수원에서 학원강사로 일하다 결혼을 하며 강릉에 살게 된 서성정(39)씨, 여기에 웹툰작가로 일러스트를 담당하는 최하얀씨가 의기투합했다.\n\n아이를 키운다는 공통분모를 지닌 이들은 아이들에게 착한 마음과 새로운 눈을 깨워주자는 의미로 2018년 12월 `아이오프너'를 창립했다. 키즈메이커 프로젝트 등 활동을 하다 지난해 11월 안목해변을 배경으로 그린 포스터가 강릉시 SNS콘텐츠 공모전에서 수상하면서 전환점을 맞게 됐다.\n\n포스터로 그린 안목해변과 함께 주문진해변 BTS 버스정류장, 하평해변의 핑크뮬리, 영진해변의 도깨비 방사제 등 감성을 자극하는 그림으로 자석, 머그컵, 노트 등의 여행상품을 만들었고 고래서점에 전시·판매되면서 인기를 얻고 있다.\n\n이들은 다음 달 4일 국립중앙도서관 국제회의장에서 문화체육관광부 주최로 열리는 `저작권 비전 2030 선포식'에서 우수사례를 발표한다.\n\n임혜정 대표는 “강릉시가 관광거점도시로 선정된 것을 계기로 앞으로 감성이 담긴 여행상품 개발뿐만 아니라 강릉을 찾아오는 관광객들에게 감성여행을 안내하는 여행자 플랫폼 사업으로 규모를 확장해갈 계획”이라고 했다.\n\n강릉=조상원기자\n\n\nⓒ 강원일보, 무단 전재 및 재배포 금지\n조상원\n기자 프로필\n구독\n조상원 기자\n구독자

## 불용어 제거 문장 데이터

단어 리스트는 불용어 제거 명사만 50개

### 변수 정의

In [ ]:
#진짜 기사 문서
real_sen_doc = preprocess_sentence(real, stop_words=stop_words, remove_stopwords=True)
real_sen_doc

'은 동화책 인문 서적 까지 약 10만 권의 장서 를 보유 온라인 헌책방 이다 고래 동일 39 온라인 중고 창업 은 직장 에서 였다 일어 했던 전직 은 의료기기 현지 에서 중고 서적 취급 현장 했고 국내 시장 에서 중고 사업 운영 가능성 점칠 있었다 첫 사업 은 2002년 헌 은 거래 상대 아닌 파지 를 모으는 으로 오해 한적 있다고 은 돈 벌이 되냐고 딱하게 쳐다본 있었다 반대 중고 자세 감동 받은 있었다 판매 에게 비용 지불 하고 포장 했는데 조심 스럽게 다루는 앞 으로 종종 하겠다고 있었다 이런 으로 고래 은 양질 중고 확보 있었다 중고 팔겠다고 하는 늘었고 보다 잘 된 많이 있었다 고래 은 중고 매입 택배 를 했고 가까운 는 방문 했다 헌 이라도 좋은 있다 소문 독서광 에게 퍼지면서 오픈마켓 에서 고래 도용 하는 일도 있었다 이런 에서 단골 늘었다 고래 은 절판 를 찾는 많다 특히 다양한 보유 하고 있어 이나 혹은 다시 싶은 장용 찾는 많다 고래 은 온라인 으로 중고 하려는 고객 불편함을 최소 시키는데 중점 두고 있다 보유 은 따라 기준 으로 등급 분류 놓고 있다 훼손 심한 는 취급 하지 않고 세세한 까지 별도 그래서 구입 하려는 고객 온라인 에서 쉽게 를 볼 있게 했다 시중 에서 쉽게 찾을 없는 중고 찾는 은 책벌레 학자 독서광 많이 회원 끼리 공유 를 쉽게 있게 게시판 시켰다 입시 지급 되는 적립금 은 어느 비중 까지 예치 시키지 않고 다음 구매 바로 쓸 있게 했다 동일 는 고래 온라인 헌책방 으로 끝내지 않겠다고 한다 이미 보광동 오프라인 헌책방 운영 10만 권의 보유 서량 2 늘리는 기적 장기 으로는 운영 오프라인 헌책방 오래된 지식 향기 를 느낄 있는 공간 이나 까페 확대 시켜 하는 에게 무료 대여 서비스 준비 는 헌책방 자취 를 감추고 있는 헌 귀하게 쓴다는 소문 나면서 좋은 취지 사업 이라고 가끔 기증 하는 분도 있습니다 하며 증자 에게 감사 뜻 전하 있다 증시 파파라치 없는 고급 증권 서비스 55 층 갇힌 불안 대지진 떠는 초고 층 아파트

In [ ]:
#진짜 기사 단어
real_noun_doc = preprocess_noun(real, stop_words=stop_words, remove_stopwords=True)
real_noun_doc = real_noun_doc[0]
real_noun_word = get_word_list(real_noun_doc.split(), 50)

In [ ]:
#가짜 기사 문서
fake_sen_doc = preprocess_sentence(fake, stop_words=stop_words, remove_stopwords=True)
fake_sen_doc

'새로운 경력 은 강릉 감성 파는 강릉 스타트업 아이 오프너 최하 얀 사진 왼쪽 백은경 임혜정 정씨 스타트업 아이 오프너 주문진 해변 BTS 버스정류장 하평 해변 핑크 뮬리 안목 해변 커피 자판기 강릉 해변 배경 으로 만든 여행 상품 으로 히트 치고 있는 스타트업 아이 오프너 아이 오프너 는 결혼 육아 으로 다니던 직장 그만두고 강릉 정착 경력 단절 여성 녀 들이 만든 스타트업 이다 KBS 콘텐츠 기획 PD 였지만 아이 를 키우기 위해 직장 그만둔 임혜정 41 와 대학교 조교 생활 하다가 으로 유학 떠났지만 결혼 함께 강릉 정착 백은경 41 경기 수원 에서 학원 강사 하다 결혼 하며 강릉 된 39 웹툰작가 일러스트 를 하는 최하 얀 의기 투합 했다 아이 를 키운다는 공통분모 를 지닌 은 아이 에게 착한 새로운 깨워주자는 2018년 12월 아이 오프너 를 창립 했다 키즈 메이커 프로젝트 하다 11월 안목 해변 배경 으로 그린 포스터 강릉시 SNS 콘텐츠 공모전 에서 수상하면서 전환 맞게 됐다 포스터 그린 안목 해변 함께 주문진 해변 BTS 버스정류장 하평 해변 핑크 뮬리 영진 해변 도깨비 방사제 감성 하는 으로 자석 머그컵 노트 여행 상품 만들었고 고래 판매 되면서 인기 를 얻고 있다 은 다음 4일 국립 도서관 회의장 에서 체육관 광부 주최 열리는 비전 2030 선포 에서 우수 를 발표 한다 임혜정 는 강릉시 관광 거점도시 선정 된 계기 앞 으로 감성 담긴 여행 상품 뿐 아니라 강릉 찾아오는 관광객 에게 감성 여행 안내 하는 여행자 플랫폼 사업 으로 를 해갈 계획 이라고 했다 강릉 조상원 강원 일보 및 조상원 프로필 조상원 378 응원 1 055 강원도 와 강릉 와 자연 멋 맛 하는 22년 지역 입니다 세계 피아니스트 온다 산골 클래식 축제 이목 평창 강서 다슬기 잡던 70 실종'

In [ ]:
#가짜 기사 단어
fake_noun_doc = preprocess_noun(fake, stop_words=stop_words, remove_stopwords=True)
fake_noun_doc = fake_noun_doc[0]
fake_noun_word = get_word_list(fake_noun_doc.split(), 50)

### 임베딩 (불용어 처리, 명사)

In [ ]:
real_sen_doc_emb = model.encode([real_sen_doc])
print(f'real_sen_doc_emb.shape : {real_sen_doc_emb.shape}')
real_noun_word_emb = model.encode(real_noun_word)
print(f'real_noun_word_emb.shape : {real_noun_word_emb.shape}')
fake_sen_doc_emb = model.encode([fake_sen_doc])
print(f'fake_sen_doc_emb.shape : {fake_sen_doc_emb.shape}')
fake_noun_word_emb = model.encode(fake_noun_word)
print(f'fake_noun_word_emb.shape : {fake_noun_word_emb.shape}')

real_sen_doc_emb.shape : (1, 768)
real_noun_word_emb.shape : (50, 768)
fake_sen_doc_emb.shape : (1, 768)
fake_noun_word_emb.shape : (50, 768)


### 코사인 유사도 (불용어 처리, 문장)

#### real + fake

In [ ]:
# real 문서 + fake 문서
distances = cosine_similarity(real_sen_doc_emb, fake_sen_doc_emb)
distances

array([[0.20369087]], dtype=float32)

In [ ]:
# real 문서 + fake 단어
distances = cosine_similarity(real_sen_doc_emb, fake_noun_word_emb)
print(distances)
print(np.mean(distances))

[[-0.03174685 -0.21856408 -0.17448643 -0.12947974 -0.12416087  0.11678267
  -0.08663168  0.02482729 -0.15502065  0.02943091 -0.00706027 -0.09696192
  -0.1668745   0.01460709 -0.1549354  -0.15426637 -0.10915466 -0.10121024
  -0.17148468 -0.11842649 -0.14797445 -0.19626106  0.16357377 -0.1081619
  -0.04993096 -0.13451895 -0.11027843  0.01040808 -0.22362737 -0.19074139
  -0.20471627 -0.09596407  0.20231959 -0.14486289 -0.1203817  -0.21159694
  -0.12885687 -0.06574059 -0.16140196 -0.0591438  -0.11325318 -0.07049283
   0.03390549 -0.09044345 -0.12957007 -0.00953552 -0.0777605   0.138006
  -0.14914234 -0.1930928 ]]
-0.089081086


In [ ]:
# real 문서 + fake 단어평균
fake_noun_word_emb_mean = np.mean(fake_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, fake_noun_word_emb_mean)
distances

array([[-0.11177652]], dtype=float32)

In [ ]:
# real 단어 + fake 문서
distances = cosine_similarity(real_noun_word_emb, fake_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.22148687


In [ ]:
# real 단어 + fake 단어
distances = cosine_similarity(real_noun_word_emb, fake_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.7237084  0.5355514  0.97412    ... 0.50240743 0.9284726  0.9463839 ]
 [0.36757118 0.49676818 0.49207127 ... 0.26535556 0.5348764  0.54316664]
 [0.41941318 0.2921177  0.4394375  ... 0.5818777  0.5274011  0.5336913 ]
 ...
 [0.6638793  0.53154683 0.97054243 ... 0.49145663 0.9062006  0.9363649 ]
 [0.5488607  0.5373445  0.8083114  ... 0.4013424  0.79522645 0.8314248 ]
 [0.4099425  0.41427767 0.6679767  ... 0.38022944 0.68509    0.691083  ]]
(50, 50)
0.55389255


In [ ]:
# real 단어평균 + fake 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, fake_sen_doc_emb)
distances

array([[0.29509133]], dtype=float32)

#### real + real2

In [ ]:
real2 = real_data['Content'][1]
#진짜 기사 문서
real2_sen_doc = preprocess_sentence(real2, stop_words=stop_words, remove_stopwords=True)
#진짜 기사 단어
real2_noun_doc = preprocess_noun(real2, stop_words=stop_words, remove_stopwords=True)
real2_noun_doc = real2_noun_doc[0]
real2_noun_word = get_word_list(real2_noun_doc.split(), 50)
#임베딩
real2_sen_doc_emb = model.encode([real2_sen_doc])
print(f'real2_sen_doc_emb.shape : {real2_sen_doc_emb.shape}')
real2_noun_word_emb = model.encode(real2_noun_word)
print(f'real2_noun_word_emb.shape : {real2_noun_word_emb.shape}')

real2_sen_doc_emb.shape : (1, 768)
real2_noun_word_emb.shape : (50, 768)


In [ ]:
# real 문서 + real2 문서
distances = cosine_similarity(real_sen_doc_emb, real2_sen_doc_emb)
distances

array([[0.26183733]], dtype=float32)

In [ ]:
# real 문서 + real2 단어
distances = cosine_similarity(real_sen_doc_emb, real2_noun_word_emb)
print(distances)
print(np.mean(distances))

[[ 0.09496121  0.07260428 -0.17052752  0.10936713 -0.18627596 -0.15012734
  -0.04716346 -0.07572298 -0.16757774 -0.11702128 -0.15729463 -0.09015824
  -0.05365399 -0.02066157 -0.1898542  -0.1393426  -0.05567371 -0.08143725
  -0.18740247 -0.17955333 -0.14557016 -0.09527148 -0.0967768  -0.09745578
  -0.19779627 -0.18122393 -0.21277928  0.0439906   0.06077512  0.01449652
  -0.10063145 -0.10816184 -0.2035545  -0.0264257  -0.09674227 -0.13426676
  -0.06972072  0.01456455 -0.16924837 -0.17448644 -0.14840186 -0.16689013
  -0.13564569 -0.20827396 -0.22261795 -0.14664906  0.12933336 -0.19042096
  -0.08196227 -0.09314872]]
-0.100669555


In [ ]:
# real 문서 + real2 단어평균
real2_noun_word_emb_mean = np.mean(real2_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, real2_noun_word_emb_mean)
distances

array([[-0.12535334]], dtype=float32)

In [ ]:
# real 단어 + real2 문서
distances = cosine_similarity(real_noun_word_emb, real2_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.22218084


In [ ]:
# real 단어 + real2 단어
distances = cosine_similarity(real_noun_word_emb, real2_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.47446108 0.3457206  0.9235866  ... 0.9498672  0.8351218  0.78779894]
 [0.17862615 0.11225205 0.49432492 ... 0.5370233  0.5224673  0.41109782]
 [0.99999994 0.80339783 0.42899275 ... 0.5190548  0.53915304 0.45233625]
 ...
 [0.47552633 0.3214498  0.887397   ... 0.9390179  0.85141146 0.80839217]
 [0.38923222 0.24741746 0.77552927 ... 0.82729423 0.71505344 0.7217636 ]
 [0.41714472 0.2778106  0.6163237  ... 0.69731015 0.6450881  0.6994566 ]]
(50, 50)
0.56617224


In [ ]:
# real 단어평균 + real2 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, real2_sen_doc_emb)
distances

array([[0.29538006]], dtype=float32)

#### real + fake2

In [ ]:
fake2 = fake_data['Content'][0]
#가짜 기사 문서
fake2_sen_doc = preprocess_sentence(fake2, stop_words=stop_words, remove_stopwords=True)
#가짜 기사 단어
fake2_noun_doc = preprocess_noun(fake2, stop_words=stop_words, remove_stopwords=True)
fake2_noun_doc = fake2_noun_doc[0]
fake2_noun_word = get_word_list(fake2_noun_doc.split(), 50)
#임베딩
fake2_sen_doc_emb = model.encode([fake2_sen_doc])
print(f'fake2_sen_doc_emb.shape : {fake2_sen_doc_emb.shape}')
fake2_noun_word_emb = model.encode(fake2_noun_word)
print(f'fake2_noun_word_emb.shape : {fake2_noun_word_emb.shape}')

fake2_sen_doc_emb.shape : (1, 768)
fake2_noun_word_emb.shape : (50, 768)


In [ ]:
# real 문서 + fake2 문서
distances = cosine_similarity(real_sen_doc_emb, fake2_sen_doc_emb)
distances

array([[0.21832174]], dtype=float32)

In [ ]:
# real 문서 + fake2 단어
distances = cosine_similarity(real_sen_doc_emb, fake2_noun_word_emb)
print(distances)
print(np.mean(distances))

[[-0.19481406 -0.18627602  0.01253594 -0.02718959 -0.18558562 -0.12831746
   0.25231162 -0.16140191 -0.10342284 -0.10919465 -0.18882154 -0.10643108
  -0.16699937 -0.08655387 -0.16970494 -0.18356809 -0.15852694  0.03422451
  -0.0125257  -0.17049463  0.0287628  -0.06798141 -0.19626106  0.01775604
  -0.0833918  -0.13934803 -0.16481794 -0.13451906 -0.121034   -0.00677761
  -0.1850281  -0.00776662 -0.10409598 -0.16191494 -0.20258287 -0.09643944
  -0.05422997 -0.11968297 -0.04201572 -0.1409227  -0.04004186 -0.08797998
   0.23737818 -0.20106402  0.00113216 -0.15015268 -0.09840084 -0.19902004
  -0.06560136 -0.18190843]]
-0.096174136


In [ ]:
# real 문서 + fake2 단어평균
fake2_noun_word_emb_mean = np.mean(fake2_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, fake2_noun_word_emb_mean)
distances

array([[-0.120311]], dtype=float32)

In [ ]:
# real 단어 + fake2 문서
distances = cosine_similarity(real_noun_word_emb, fake2_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.22186548


In [ ]:
# real 단어 + fake2 단어
distances = cosine_similarity(real_noun_word_emb, fake2_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.9067644  0.5066105  0.55756027 ... 0.85112846 0.8287846  0.83779246]
 [0.67261624 1.         0.23739049 ... 0.49099237 0.40907973 0.5063595 ]
 [0.49492228 0.17862615 0.4530959  ... 0.5027371  0.34718305 0.4993686 ]
 ...
 [0.8867731  0.47509784 0.56309164 ... 0.8585193  0.80575585 0.83604693]
 [0.78075707 0.42190176 0.43306905 ... 0.7640576  0.6576079  0.77638257]
 [0.6222623  0.3190912  0.4377239  ... 0.7036453  0.5330664  0.59850264]]
(50, 50)
0.56728184


In [ ]:
# real 단어평균 + fake2 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, fake2_sen_doc_emb)
distances

array([[0.296589]], dtype=float32)

## 문장 데이터 (불용어 제거 X)

단어 리스트는 불용어 제거 명사만 50개

### 변수 정의

In [ ]:
#진짜 기사 문서
real_sen_doc = preprocess_sentence(real)
real_sen_doc

'은 동화책 부터 소설 인문 철학 전문 서적 까지 약 10만 권의 장서 를 보유 한 온라인 헌책방 이다 고래 서점 남 동일 대표 39 가 처음 온라인 중고 책 서점 창업 을 생각 한 것 은 전 직장 에서 였다 대학 때 일어 전공 을 했던 남 대표 의 전직 은 의료기기 회사 일본 지 사장 일본 현지 에서 중고 서적 의 취급 현장 을 직접 확인 했고 국내 시장 에서 중고 책 사업 운영 가능성 을 점칠 수 있었다 첫 사업 을 시작 한 것 은 2002년 서울 처음 헌 책 을 구 할 때 사람 들 은 거래 상대 가 아닌 파지 를 모으는 사람 으로 오해 한적 도 있다고 한번 은 이 걸 로 돈 벌이 나 되냐고 딱하게 쳐다본 사람 들 도 있었다 반대 로 남 대표 의 중고 책 에 대한 자세 에 감동 을 받은 사람 도 있었다 판매 자 에게 비용 을 지불 하고 포장 을 했는데 책 을 조심 스럽게 다루는 모습 을 보고 앞 으로 종종 이용 하겠다고 한 사람 도 있었다 이런 노력 으로 고래 서점 은 양질 의 중고 책 을 확보 할 수 있었다 중고 책 을 먼저 팔겠다고 하는 사람 들 도 늘었고 보다 관리 가 잘 된 책 을 많이 구 할 수 있었다 고래 서점 은 중고 책 을 매입 할 때 택배 비 를 대신 부담 했고 가까운 거리 는 직접 방문 했다 헌 책 이라도 더 좋은 상태 로 구 할 수 있다 란 입 소문 이 독서광 들 에게 퍼지면서 오픈마켓 에서 고래 서점 의 이름 을 도용 하는 일도 있었다 이런 과정 속 에서 단골 들 이 늘었다 현재 고래 서점 은 절판 도서 를 찾는 사람 들 이 많다 특히 다양한 분야 의 책 을 보유 하고 있어 관심 분야 의 책 이나 혹은 다시 보고 싶은 책 소 장용 책 등 을 찾는 사람 들 이 많다 고래 서점 은 온라인 으로 중고 책 을 구 하려는 고객 의 불편함을 최소 화 시키는데 중점 을 두고 있다 보유 한 책 은 상태 에 따라 자체 기준 으로 상 중 하 등급 별로 분류 해 놓고 있다 훼손 상태 가 심한 도서 는 취급 하지 않고 세세한 책 상태 까지 별도 정보 로 제

In [ ]:
#진짜 기사 단어
real_noun_doc = preprocess_noun(real, stop_words=stop_words, remove_stopwords=True)
real_noun_doc = real_noun_doc[0]
real_noun_word = get_word_list(real_noun_doc.split(), 50)

In [ ]:
#가짜 기사 문서
fake_sen_doc = preprocess_sentence(fake)
fake_sen_doc

'우리 새로운 경력 은 강릉 의 감성 파는 일 강릉 스타트업 아이 오프너 의 최하 얀 사진 왼쪽 부터 백은경 임혜정 서 성 정씨 스타트업 아이 오프너 주문진 해변 의 BTS 버스정류장 하평 해변 의 핑크 뮬리 안목 해변 의 커피 자판기 등 강릉 의 해변 을 배경 으로 만든 여행 상품 으로 히트 치고 있는 스타트업 아이 오프너 아이 오프너 는 결혼 과 육아 등 으로 다니던 직장 을 그만두고 강릉 에 정착 한 경력 단절 여성 경 단 녀 들이 만든 스타트업 이다 KBS 콘텐츠 기획 PD 였지만 아이 를 키우기 위해 직장 을 그만둔 임혜정 41 대표 와 대학교 조교 생활 을 하다가 일본 으로 유학 을 떠났지만 결혼 과 함께 강릉 에 정착 한 백은경 41 씨 경기 수원 에서 학원 강사 로 일 하다 결혼 을 하며 강릉 에 살 게 된 서 성 정 39 씨 여기 에 웹툰작가 로 일러스트 를 담당 하는 최하 얀 씨 가 의기 투합 했다 아이 를 키운다는 공통분모 를 지닌 이 들 은 아이 들 에게 착한 마음 과 새로운 눈 을 깨워주자는 의미 로 2018년 12월 아이 오프너 를 창립 했다 키즈 메이커 프로젝트 등 활동 을 하다 지난해 11월 안목 해변 을 배경 으로 그린 포스터 가 강릉시 SNS 콘텐츠 공모전 에서 수상하면서 전환 점 을 맞게 됐다 포스터 로 그린 안목 해변 과 함께 주문진 해변 BTS 버스정류장 하평 해변 의 핑크 뮬리 영진 해변 의 도깨비 방사제 등 감성 을 자극 하는 그림 으로 자석 머그컵 노트 등 의 여행 상품 을 만들었고 고래 서점 에 전시 판매 되면서 인기 를 얻고 있다 이 들 은 다음 달 4일 국립 중앙 도서관 국제 회의장 에서 문화 체육관 광부 주최 로 열리는 저작권 비전 2030 선포 식 에서 우수 사례 를 발표 한다 임혜정 대표 는 강릉시 가 관광 거점도시 로 선정 된 것 을 계기 로 앞 으로 감성 이 담긴 여행 상품 개발 뿐 만 아니라 강릉 을 찾아오는 관광객 들 에게 감성 여행 을 안내 하는 여행자 플랫폼 사업 으로 규모 를 확장 해갈 계획 이

In [ ]:
#가짜 기사 단어
fake_noun_doc = preprocess_noun(fake, stop_words=stop_words, remove_stopwords=True)
fake_noun_doc = fake_noun_doc[0]
fake_noun_word = get_word_list(fake_noun_doc.split(), 50)

### 임베딩 (불용어 처리, 명사)

In [ ]:
real_sen_doc_emb = model.encode([real_sen_doc])
print(f'real_sen_doc_emb.shape : {real_sen_doc_emb.shape}')
real_noun_word_emb = model.encode(real_noun_word)
print(f'real_noun_word_emb.shape : {real_noun_word_emb.shape}')
fake_sen_doc_emb = model.encode([fake_sen_doc])
print(f'fake_sen_doc_emb.shape : {fake_sen_doc_emb.shape}')
fake_noun_word_emb = model.encode(fake_noun_word)
print(f'fake_noun_word_emb.shape : {fake_noun_word_emb.shape}')

real_sen_doc_emb.shape : (1, 768)
real_noun_word_emb.shape : (50, 768)
fake_sen_doc_emb.shape : (1, 768)
fake_noun_word_emb.shape : (50, 768)


### 코사인 유사도 (불용어 처리, 문장)

#### real + fake

In [ ]:
# real 문서 + fake 문서
distances = cosine_similarity(real_sen_doc_emb, fake_sen_doc_emb)
distances

array([[0.62348455]], dtype=float32)

In [ ]:
# real 문서 + fake 단어
distances = cosine_similarity(real_sen_doc_emb, fake_noun_word_emb)
print(distances)
print(np.mean(distances))

[[ 0.32329428 -0.08667517  0.0292524   0.09525049  0.04747156  0.22200301
   0.0053687   0.077126    0.04231142  0.10177166  0.08075553  0.04491265
   0.03949721  0.10887505  0.06563133  0.02750469  0.10678988  0.1309008
   0.05929995  0.08103135  0.06896946 -0.01576655  0.13874613  0.0660817
   0.04731899  0.09212548  0.0452529   0.36654192 -0.07046314 -0.08941112
  -0.00134063  0.16106898  0.17779511  0.03015099  0.07036562 -0.09318772
  -0.01715437  0.11380801  0.03158931  0.1419296   0.08873911  0.01167032
   0.24038142  0.00728726  0.10690437  0.13904278  0.11971866  0.2350927
   0.04936802  0.00178017]]
0.07733557


In [ ]:
# real 문서 + fake 단어평균
fake_noun_word_emb_mean = np.mean(fake_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, fake_noun_word_emb_mean)
distances

array([[0.10281295]], dtype=float32)

In [ ]:
# real 단어 + fake 문서
distances = cosine_similarity(real_noun_word_emb, fake_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.124489896


In [ ]:
# real 단어 + fake 단어
distances = cosine_similarity(real_noun_word_emb, fake_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.7237084  0.5355514  0.97412    ... 0.50240743 0.9284726  0.9463839 ]
 [0.36757118 0.49676818 0.49207127 ... 0.26535556 0.5348764  0.54316664]
 [0.41941318 0.2921177  0.4394375  ... 0.5818777  0.5274011  0.5336913 ]
 ...
 [0.6638793  0.53154683 0.97054243 ... 0.49145663 0.9062006  0.9363649 ]
 [0.5488607  0.5373445  0.8083114  ... 0.4013424  0.79522645 0.8314248 ]
 [0.4099425  0.41427767 0.6679767  ... 0.38022944 0.68509    0.691083  ]]
(50, 50)
0.55389255


In [ ]:
# real 단어평균 + fake 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, fake_sen_doc_emb)
distances

array([[0.17095216]], dtype=float32)

#### real + real2

In [ ]:
real2 = real_data['Content'][1]
#진짜 기사 문서
real2_sen_doc = preprocess_sentence(real2)
#진짜 기사 단어
real2_noun_doc = preprocess_noun(real2, stop_words=stop_words, remove_stopwords=True)
real2_noun_doc = real2_noun_doc[0]
real2_noun_word = get_word_list(real2_noun_doc.split(), 50)
#임베딩
real2_sen_doc_emb = model.encode([real2_sen_doc])
print(f'real2_sen_doc_emb.shape : {real2_sen_doc_emb.shape}')
real2_noun_word_emb = model.encode(real2_noun_word)
print(f'real2_noun_word_emb.shape : {real2_noun_word_emb.shape}')

real2_sen_doc_emb.shape : (1, 768)
real2_noun_word_emb.shape : (50, 768)


In [ ]:
# real 문서 + real2 문서
distances = cosine_similarity(real_sen_doc_emb, real2_sen_doc_emb)
distances

array([[0.56188077]], dtype=float32)

In [ ]:
# real 문서 + real2 단어
distances = cosine_similarity(real_sen_doc_emb, real2_noun_word_emb)
print(distances)
print(np.mean(distances))

[[ 0.14953469  0.10582714  0.06904902  0.25225097 -0.00515912  0.12407625
   0.1820553   0.11806365  0.04492919  0.01676628  0.06120777  0.10937542
   0.00182856  0.09796147  0.00090483  0.07940654  0.08614945  0.02987489
   0.00382756  0.00460128 -0.14752394  0.08584759  0.08822967 -0.02363169
  -0.02938713  0.01306701 -0.1502987   0.13982859  0.05199867  0.18203098
   0.08740662  0.0660817  -0.02558088  0.0751114   0.09474501  0.09071264
   0.08416694  0.14478096 -0.00614826  0.02925239  0.04937557  0.02023571
   0.08534627 -0.01465562 -0.13500603  0.08349407  0.2758641   0.02477856
   0.08462983  0.02141395]]
0.057573944


In [ ]:
# real 문서 + real2 단어평균
real2_noun_word_emb_mean = np.mean(real2_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, real2_noun_word_emb_mean)
distances

array([[0.07198645]], dtype=float32)

In [ ]:
# real 단어 + real2 문서
distances = cosine_similarity(real_noun_word_emb, real2_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.22803955


In [ ]:
# real 단어 + real2 단어
distances = cosine_similarity(real_noun_word_emb, real2_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.47446108 0.3457206  0.9235866  ... 0.9498672  0.8351218  0.78779894]
 [0.17862615 0.11225205 0.49432492 ... 0.5370233  0.5224673  0.41109782]
 [0.99999994 0.80339783 0.42899275 ... 0.5190548  0.53915304 0.45233625]
 ...
 [0.47552633 0.3214498  0.887397   ... 0.9390179  0.85141146 0.80839217]
 [0.38923222 0.24741746 0.77552927 ... 0.82729423 0.71505344 0.7217636 ]
 [0.41714472 0.2778106  0.6163237  ... 0.69731015 0.6450881  0.6994566 ]]
(50, 50)
0.56617224


In [ ]:
# real 단어평균 + real2 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, real2_sen_doc_emb)
distances

array([[0.30429608]], dtype=float32)

#### real + fake2

In [ ]:
fake2 = fake_data['Content'][0]
#가짜 기사 문서
fake2_sen_doc = preprocess_sentence(fake2)
#가짜 기사 단어
fake2_noun_doc = preprocess_noun(fake2, stop_words=stop_words, remove_stopwords=True)
fake2_noun_doc = fake2_noun_doc[0]
fake2_noun_word = get_word_list(fake2_noun_doc.split(), 50)
#임베딩
fake2_sen_doc_emb = model.encode([fake2_sen_doc])
print(f'fake2_sen_doc_emb.shape : {fake2_sen_doc_emb.shape}')
fake2_noun_word_emb = model.encode(fake2_noun_word)
print(f'fake2_noun_word_emb.shape : {fake2_noun_word_emb.shape}')

fake2_sen_doc_emb.shape : (1, 768)
fake2_noun_word_emb.shape : (50, 768)


In [ ]:
# real 문서 + fake2 문서
distances = cosine_similarity(real_sen_doc_emb, fake2_sen_doc_emb)
distances

array([[0.43332508]], dtype=float32)

In [ ]:
# real 문서 + fake2 단어
distances = cosine_similarity(real_sen_doc_emb, fake2_noun_word_emb)
print(distances)
print(np.mean(distances))

[[ 0.01745802 -0.00515919  0.09780204  0.14425242 -0.00373353  0.07415423
   0.12970167  0.03158933  0.04160851  0.10152131 -0.07153099 -0.03340154
   0.04135067  0.10841639 -0.01827131  0.02671818  0.02292712  0.12317316
   0.28411502  0.05438893 -0.02692541 -0.10531475 -0.01576655  0.21884727
  -0.07948831  0.08067611  0.02216811  0.09212545  0.0160957   0.11700542
  -0.00997157  0.29665282  0.12061463 -0.00518134  0.00947873  0.10441656
   0.08043439  0.02146507  0.28101078 -0.09220757  0.07608393  0.11705759
   0.33982405 -0.00260622  0.07271059  0.02448079  0.03782412 -0.00435916
   0.1139624  -0.02790016]]
0.06080588


In [ ]:
# real 문서 + fake2 단어평균
fake2_noun_word_emb_mean = np.mean(fake2_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, fake2_noun_word_emb_mean)
distances

array([[0.073884]], dtype=float32)

In [ ]:
# real 단어 + fake2 문서
distances = cosine_similarity(real_noun_word_emb, fake2_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.04458351


In [ ]:
# real 단어 + fake2 단어
distances = cosine_similarity(real_noun_word_emb, fake2_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.9067644  0.5066105  0.55756027 ... 0.85112846 0.8287846  0.83779246]
 [0.67261624 1.         0.23739049 ... 0.49099237 0.40907973 0.5063595 ]
 [0.49492228 0.17862615 0.4530959  ... 0.5027371  0.34718305 0.4993686 ]
 ...
 [0.8867731  0.47509784 0.56309164 ... 0.8585193  0.80575585 0.83604693]
 [0.78075707 0.42190176 0.43306905 ... 0.7640576  0.6576079  0.77638257]
 [0.6222623  0.3190912  0.4377239  ... 0.7036453  0.5330664  0.59850264]]
(50, 50)
0.56728184


In [ ]:
# real 단어평균 + fake2 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, fake2_sen_doc_emb)
distances

array([[0.05858099]], dtype=float32)

# -----다른 기업2 소년식당-----

In [ ]:
# filtered.groupby('기업명')['Content'].count().sort_values(ascending=False).to_csv('filtered.txt')
# remained.groupby('기업명')['Content'].count().sort_values(ascending=False).to_csv('remained.txt')
# full = pd.concat([filtered, remained], axis=0)
# full.groupby('기업명')['Content'].count().sort_values(ascending=False).to_csv('full.txt')

In [ ]:
filtered[filtered['기업명']=='소년식당']

,기업명,Title,Content,c_index,c_name
2294,소년식당,‘생활의 달인’ 간장새우 밥의 달인 맛집···서교동 ‘소년식당’,이미지 원본보기\n\n\n‘생활의 달인’ 간장새우 밥의 달인이 화제로 떠올랐다.\n...,56,음식점
2295,소년식당,"간장새우밥의 달인, 짭쪼름하고 달콤한 간장새우밥 ""그래서 어디죠?""",이미지 원본보기\n사진=SBS 캡처\n\n간장새우밥 달인이 시청자들의 입맛을 자극했...,56,음식점


In [ ]:
remained[remained['기업명']=='소년식당']

,기업명,Title,Content,c_index,c_name
33515,소년식당,'V앱' 소년24 축하 첫방 #해준생일 #하트200만돌파(종합),이미지 원본보기\n소년24 V앱 방송캡처\n\n[헤럴드POP=박수인 기자]소년24의...,56,음식점
33516,소년식당,'소년24'의 팬맞춤형 첫방 '히트다 히트'[V라이브 종합],"이미지 원본보기\n[OSEN=강서정 기자] '소년24'가 요리부터 팔찌 만들기, 편...",56,음식점
33517,소년식당,Find Dining | 성수동 요즘 맛,"몹시 추운 날씨에도, 브레이크타임이 끝나자마자 이미 대기하고 있던 손님들이 우르르 ...",56,음식점
33518,소년식당,느리게 걸으면 골목길 삶의 현장 뚜렷이 다가와,[한겨레] 손관승의 공감재생 골목여행 ③ 요즘 뜨는 동네 성수동\n\n한때 서울에서...,56,음식점
33519,소년식당,2TV '생생정보' 광주 강황머리국밥-종로 영양돌솥밥-서교동 간장새우밥- 청주 삼겹...,"이미지 원본보기\n\n7일 방송되는 KBS 2TV '생생정보'에서는 삼색삼맛, 삼바...",56,음식점


In [ ]:
filtered['Content'][2295]

In [ ]:
remained['Content'][33518]

In [ ]:
real_data = pd.DataFrame({'Content':[filtered['Content'][2294], filtered['Content'][2295], remained['Content'][33517]]})
fake_data = pd.DataFrame({'Content':[remained['Content'][33515], remained['Content'][33516], remained['Content'][33518]]})

In [ ]:
real_data

,Content
0,이미지 원본보기\n\n\n‘생활의 달인’ 간장새우 밥의 달인이 화제로 떠올랐다.\n...
1,이미지 원본보기\n사진=SBS 캡처\n\n간장새우밥 달인이 시청자들의 입맛을 자극했...
2,"몹시 추운 날씨에도, 브레이크타임이 끝나자마자 이미 대기하고 있던 손님들이 우르르 ..."


In [ ]:
fake_data

,Content
0,이미지 원본보기\n소년24 V앱 방송캡처\n\n[헤럴드POP=박수인 기자]소년24의...
1,"이미지 원본보기\n[OSEN=강서정 기자] '소년24'가 요리부터 팔찌 만들기, 편..."
2,[한겨레] 손관승의 공감재생 골목여행 ③ 요즘 뜨는 동네 성수동\n\n한때 서울에서...


In [ ]:
real = real_data['Content'][0]
fake = fake_data['Content'][0]

In [ ]:
real

'이미지 원본보기\n\n\n‘생활의 달인’ 간장새우 밥의 달인이 화제로 떠올랐다.\n\n26일 방송된 ‘생활의 달인’에서는 간장새우 달인이 운영하는 ‘소년식당’이 소개됐다.\n\n갖가지 재료를 넣고 간장을 졸여 새우를 담가 만드는 간장새우. 잘못 조리하면 비린 맛이 나 먹기가 힘들어 만들기 쉽지 않다고 한다. 그만큼 만드는 사람의 노하우가 중요하다고 한다.\n\n여기 그 비린 맛을 싹 없앤 간장새우를 만드는 달인이 있다. 그 주인공은 바로 박중천 (36세, 경력 16년) 달인. 달인의 가게는 간장새우의 비린 맛을 전혀 느낄 수 없어 사람들이 많이 찾는다고 한다.\n\n그 비밀의 시작은 바로 숙성. 숙성 과정의 비법은 오이와 레몬을 갈아 청주를 넣고 밥솥에 찌는 것이라고 한다. 그것을 식히고 새우를 넣어 가다랑어 포의 향을 더해 숙성시킨다고. 간장새우의 핵심이라고 할 수 있는 간장 또한 평범하지 않다.\n\n특별한 비법으로 구운 소고기를 간장에 넣어 끓이고, 여기에 토마토와 구운 대파를 넣어 간장의 짠맛을 잡아주고 은은한 단맛을 내준다고. 쌀밥 위에 올라가는 후리카케 또한 특별하다고 한다.\n\n이곳은 서울시 마포구 와우산로29라길 16에 위치해 있다.\n\n[사진=SBS 제공]\n\n/전종선기자 sestar@sedaily.com\n\n\n[서울경제 바로가기] [인기포토로 보는 오늘]\n서울경제와 친해지는 가장 확실한 방법 [페이스북]\n\n\n저작권자 ⓒ 서울경제, 무단 전재 및 재배포 금지'

In [ ]:
fake

'이미지 원본보기\n소년24 V앱 방송캡처\n\n[헤럴드POP=박수인 기자]소년24의 첫 번째 V앱 방송이 진행됐다.\n\n16일 오후 8시 네이버 V앱에서는 소년24의 \'놀러와 마이 소년하우스\'가 방송됐다. 이날 한 집에 모인 멤버들은 소년 식당, 소년 공방, 소년 놀이방, 소년 벌칙방으로 나뉘어 각자 임무를 수행했다.\n\n첫 번째 팀은 소년 식당을 열어 요리를 진행했다. 도하 팀은 크림 파스타와 로제 파스타를 만들겠다고 나섰고, 영두 팀은 김밥을 만들기 시작했다.\n\n두 번째 팀은 소년 공방을 열었다. 소년들은 "팬 분들에 너무 받은 것이 많아서 보답하기 위해 팔찌를 만들었다"며 한 땀 한 땀 정성들여 제작하는 모습을 보여줬다. 생일을 맞은 MC 해준은 "\'댓글창에 해준아 생일축하해\'를 남겨주시면 선착순 다섯 분께 CD와 팔찌를 보내드리겠다"고 말했다.\n\n소년 놀이방에서는 소년들의 어린시절 사진이 공개됐다. MC 해준은 멤버들에 돌발 미션을 냈다. 해준은 "제 어린 시절 사진을 맞춰주시면 팬분들에 마술을 보여드리겠다"고 말했고 잘 맞추지 못하는 멤버들에 "어릴 때부터 댄디한 스타일로 입었다"며 힌트를 주기도 했다.\n\n파스타조와 김밥조로 나뉜 첫 번 째 팀은 또 한 번 팬들에 미션을 공지했다. 파스타조 미션문구는 \'도하 & 진섭 & 지형아 파스타를 부탁해\'였고 김밥조의 미션 문구는 \'영두 & 창민 & 용현아 김밥 잘~말아줘\'였다. 앞선 미션과 마찬가지로 오타 없이 댓글을 올려준 팬들에 선착순으로 싸인CD, 팔찌를 전달할 것을 알렸다. V앱 방송 중 하트 백만개가 돌파했고 소년24 멤버들은 이를 기념하기 위해 샤이니의 \'누난 너무 예뻐\'를 불렀다.\n\n그 사이 벌칙방 멤버들은 두 팀으로 나뉘어 요리팀의 부족한 재료, 간식을 사오는 미션을 완료했다.\n\n방송 시간 마무리가 가까워오자 식당, 공방, 놀이방, 벌칙방이 세팅을 마쳤다. 그 때 \'최애부활전\'을 통해 재합류하게 된 다섯 멤버가 깜짝 등장했다. 성호는 "한 번 더 여러분

## 불용어 제거 문장 데이터

단어 리스트는 불용어 제거 명사만 50개

### 변수 정의

In [ ]:
#진짜 기사 문서
real_sen_doc = preprocess_sentence(real, stop_words=stop_words, remove_stopwords=True)
real_sen_doc

'이미지 본보기 생활 달인 간장 새우 밥 달인 화제 떠올랐다 26일 방송 된 생활 달인 에서는 간장 새우 달인 운영 하는 소년 식당 됐다 갖가지 재료 를 넣고 간장 졸여 새우 를 담가 만드는 간장 새우 잘못 조리 하면 비린 맛 먹기 힘들어 쉽지 않다고 한다 그만큼 만드는 노하우 중요하다고 한다 비린 맛 싹 없앤 간장 새우 를 만드는 달인 있다 은 바로 중천 36 세 경력 16년 달인 달인 가게 는 간장 새우 비린 맛 느낄 없어 많이 찾는다고 한다 은 바로 숙성 숙성 비법 은 오이 와 레몬 갈아 청주 를 넣고 밥솥 찌는 이라고 한다 식히고 새우 를 넣어 가다랑어 포의 더해 숙성 시킨다고 간장 새우 핵심 이라고 있는 간장 또한 평범하지 않다 특별한 비법 으로 소고기 를 간장 넣어 끓이고 토마토 와 대파 를 넣어 간장 짠맛 잡아주고 은은한 단맛 내준다고 쌀밥 올라가는 후리카케 또한 특별하다고 한다 은 와우산 29 16 있다 및'

In [ ]:
#진짜 기사 단어
real_noun_doc = preprocess_noun(real, stop_words=stop_words, remove_stopwords=True)
real_noun_doc = real_noun_doc[0]
real_noun_word = get_word_list(real_noun_doc.split(), 40)

In [ ]:
#가짜 기사 문서
fake_sen_doc = preprocess_sentence(fake, stop_words=stop_words, remove_stopwords=True)
fake_sen_doc

'이미지 본보기 소년 24 V 앱 방송 캡처 소년 24 첫 번째 V 앱 방송 됐다 16일 8시 네이버 V 앱 에서는 소년 24 놀러와 마이 소년 하우스 방송 됐다 날 모인 멤버 은 소년 식당 소년 소년 놀이 소년 벌칙 으로 나뉘어 각자 임무 를 수행 했다 첫 번째 팀 은 소년 식당 열어 요리 를 했다 도하 팀 은 크림 파스타 와 로제 파스타 를 만들겠다고 나섰고 팀 은 김밥 했다 번째 팀 은 소년 열었다 소년 은 팬 너무 받은 많아서 보답 하기 위해 팔찌 를 만들었다며 땀 땀 정성 제작 하는 보여줬다 생일 맞은 MC 해준은 댓글 해준아 생일 축하 를 남겨주시면 선착순 다섯 께 CD 와 팔찌 를 보내 드리겠다고 했다 소년 놀이 에서는 소년 어린시절 사진 됐다 MC 해준은 멤버 돌발 미션 냈다 해준은 어린 사진 맞춰주시면 팬 분들 마술 보여 드리겠다고 했고 잘 맞추지 못 하는 멤버 어릴 댄디 스타일 입었다며 힌트 를 주기도 했다 파스타 와 김밥 조로 나뉜 첫 째 팀 은 또 팬 미션 공지 했다 파스타 미션 문구 는 도하 진섭 지형 파스타 를 부탁 해였고 김밥 조의 미션 문구 는 창민 현아 김밥 잘 말아줘였다 앞선 미션 오타 없이 댓글 올려준 팬 선착순 으로 싸인 CD 팔찌 를 전달 알렸다 V 앱 방송 하트 백만 돌파 했고 소년 24 멤버 은 이를 기념 하기 위해 샤이니 너무 예뻐를 불렀다 벌칙 멤버 은 팀 으로 나뉘어 요리 팀 부족한 재료 간식 사오는 미션 완료 했다 방송 시간 가까워오자 식당 놀이 벌칙 방이 세팅 마쳤다 최애 부활 합류 된 다섯 멤버 깜짝 했다 성호 는 여러분 팬 걱정 시키지 않고 열심히 하는 성호 되겠다고 했으며 연태 는 번째 부활 이다 또 만큼 열심히 해서 데뷔 까지 있도록 하겠다는 소감 했다 마지막 으로 소년 24 멤버 은 해준의 생일 축하 했다 하트 200만 개가 돌파 했고 해준은 생일 선물 하트 200만 개를 주신 같다며 팬 고마움 했다 이어 금요일 비하인드 영상 하겠다며 계획 알렸다 popnews heraldcorp com'

In [ ]:
#가짜 기사 단어
fake_noun_doc = preprocess_noun(fake, stop_words=stop_words, remove_stopwords=True)
fake_noun_doc = fake_noun_doc[0]
fake_noun_word = get_word_list(fake_noun_doc.split(), 50)

### 임베딩 (불용어 처리, 명사)

In [ ]:
real_sen_doc_emb = model.encode([real_sen_doc])
print(f'real_sen_doc_emb.shape : {real_sen_doc_emb.shape}')
real_noun_word_emb = model.encode(real_noun_word)
print(f'real_noun_word_emb.shape : {real_noun_word_emb.shape}')
fake_sen_doc_emb = model.encode([fake_sen_doc])
print(f'fake_sen_doc_emb.shape : {fake_sen_doc_emb.shape}')
fake_noun_word_emb = model.encode(fake_noun_word)
print(f'fake_noun_word_emb.shape : {fake_noun_word_emb.shape}')

real_sen_doc_emb.shape : (1, 768)
real_noun_word_emb.shape : (40, 768)
fake_sen_doc_emb.shape : (1, 768)
fake_noun_word_emb.shape : (50, 768)


### 코사인 유사도 (불용어 처리, 문장)

#### real + fake

In [ ]:
# real 문서 + fake 문서
distances = cosine_similarity(real_sen_doc_emb, fake_sen_doc_emb)
distances

array([[0.79657483]], dtype=float32)

In [ ]:
# real 문서 + fake 단어
distances = cosine_similarity(real_sen_doc_emb, fake_noun_word_emb)
print(distances)
print(np.mean(distances))

[[ 0.10943854  0.17549056  0.10409755  0.11753546  0.19283172  0.4262088
   0.07179265  0.18504384  0.17411989  0.19946823  0.16604345  0.16621432
   0.2116246   0.18711269  0.13374582  0.13432822  0.15235442  0.15156323
   0.01799839 -0.00638125  0.16988516  0.15784037  0.0488534   0.12806977
   0.05004213  0.10817692  0.20826003  0.13783562  0.12573594  0.4289006
   0.20026419  0.06316123  0.16627233  0.07579859  0.13335419  0.06931211
   0.12966968  0.12499164  0.15116237  0.1368162   0.20920274  0.08887531
   0.14756191  0.1508715   0.12849006  0.1806768   0.12722611  0.140099
   0.0905388   0.13695954]]
0.1457107


In [ ]:
# real 문서 + fake 단어평균
fake_noun_word_emb_mean = np.mean(fake_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, fake_noun_word_emb_mean)
distances

array([[0.17889291]], dtype=float32)

In [ ]:
# real 단어 + fake 문서
distances = cosine_similarity(real_noun_word_emb, fake_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.25272116


In [ ]:
# real 단어 + fake 단어
distances = cosine_similarity(real_noun_word_emb, fake_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.737159   0.8711393  0.6149226  ... 0.8942684  0.81560814 0.92055047]
 [0.38081753 0.5023731  0.3498181  ... 0.5386984  0.407575   0.5413713 ]
 [0.76245147 0.9123132  0.71214676 ... 0.88417697 0.83928204 0.9209814 ]
 ...
 [0.80436325 0.94215095 0.6495871  ... 0.94562644 0.8781835  0.9382018 ]
 [0.47295114 0.5467932  0.44780362 ... 0.5823052  0.55660504 0.6067578 ]
 [0.51025707 0.63135326 0.54693615 ... 0.5937297  0.60684544 0.639156  ]]
(40, 50)
0.60959643


In [ ]:
# real 단어평균 + fake 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, fake_sen_doc_emb)
distances

array([[0.32546315]], dtype=float32)

#### real + real2

In [ ]:
real2 = real_data['Content'][1]
#진짜 기사 문서
real2_sen_doc = preprocess_sentence(real2, stop_words=stop_words, remove_stopwords=True)
#진짜 기사 단어
real2_noun_doc = preprocess_noun(real2, stop_words=stop_words, remove_stopwords=True)
real2_noun_doc = real2_noun_doc[0]
real2_noun_word = get_word_list(real2_noun_doc.split(), 40)
#임베딩
real2_sen_doc_emb = model.encode([real2_sen_doc])
print(f'real2_sen_doc_emb.shape : {real2_sen_doc_emb.shape}')
real2_noun_word_emb = model.encode(real2_noun_word)
print(f'real2_noun_word_emb.shape : {real2_noun_word_emb.shape}')

real2_sen_doc_emb.shape : (1, 768)
real2_noun_word_emb.shape : (40, 768)


In [ ]:
# real 문서 + real2 문서
distances = cosine_similarity(real_sen_doc_emb, real2_sen_doc_emb)
distances

array([[0.76466453]], dtype=float32)

In [ ]:
# real 문서 + real2 단어
distances = cosine_similarity(real_sen_doc_emb, real2_noun_word_emb)
print(distances)
print(np.mean(distances))

[[ 0.18024842  0.10151167  0.19683757  0.15577254  0.09884672  0.06850202
   0.05203623  0.15596372  0.05004213  0.10817692 -0.00638123  0.20826004
   0.03933421  0.20552608  0.10949478  0.11738224  0.12364757  0.10943855
   0.18504387  0.06071903  0.23384666  0.14065818  0.11734805  0.0703069
   0.04692109  0.3688913   0.13802916  0.11624642  0.10927615  0.13453281
   0.12877429  0.2320099   0.23674554  0.36505002  0.21932709  0.04560141
   0.35640562  0.08294521  0.07935841  0.11492096]]
0.14143994


In [ ]:
# real 문서 + real2 단어평균
real2_noun_word_emb_mean = np.mean(real2_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, real2_noun_word_emb_mean)
distances

array([[0.18663599]], dtype=float32)

In [ ]:
# real 단어 + real2 문서
distances = cosine_similarity(real_noun_word_emb, real2_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.23431304


In [ ]:
# real 단어 + real2 단어
distances = cosine_similarity(real_noun_word_emb, real2_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.9999999  0.56501734 0.8677147  ... 0.51282823 0.6382621  0.913658  ]
 [0.56501734 1.0000002  0.53609437 ... 0.28310558 0.3560412  0.53156877]
 [0.8677147  0.53609437 0.99999964 ... 0.599368   0.72121054 0.89931476]
 ...
 [0.90625846 0.52388054 0.89294875 ... 0.56455696 0.6887381  0.912665  ]
 [0.58103365 0.3285708  0.5906943  ... 0.33889863 0.48009646 0.609683  ]
 [0.57517457 0.39308733 0.6797733  ... 0.52855885 0.5634472  0.6287033 ]]
(40, 40)
0.5700368


In [ ]:
# real 단어평균 + real2 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, real2_sen_doc_emb)
distances

array([[0.3018632]], dtype=float32)

#### real + real3

In [ ]:
real3 = real_data['Content'][2]
#진짜 기사 문서
real3_sen_doc = preprocess_sentence(real3, stop_words=stop_words, remove_stopwords=True)
#진짜 기사 단어
real3_noun_doc = preprocess_noun(real3, stop_words=stop_words, remove_stopwords=True)
real3_noun_doc = real3_noun_doc[0]
real3_noun_word = get_word_list(real3_noun_doc.split(), 50)
#임베딩
real3_sen_doc_emb = model.encode([real3_sen_doc])
print(f'real3_sen_doc_emb.shape : {real3_sen_doc_emb.shape}')
real3_noun_word_emb = model.encode(real3_noun_word)
print(f'real3_noun_word_emb.shape : {real3_noun_word_emb.shape}')

real3_sen_doc_emb.shape : (1, 768)
real3_noun_word_emb.shape : (50, 768)


In [ ]:
# real 문서 + real3 문서
distances = cosine_similarity(real_sen_doc_emb, real3_sen_doc_emb)
distances

array([[0.44675112]], dtype=float32)

In [ ]:
# real 문서 + real3 단어
distances = cosine_similarity(real_sen_doc_emb, real3_noun_word_emb)
print(distances)
print(np.mean(distances))

[[ 0.23854575  0.04973807  0.08134006  0.09635722  0.17985922  0.18504384
   0.10151164  0.10458358  0.16093114  0.12364754  0.07245757  0.1802484
   0.16745713  0.13037969  0.13116422  0.12070265  0.15773185  0.09527606
   0.14100932  0.17731486  0.14607999  0.10943853  0.25519675  0.10768398
   0.15628871  0.1460228   0.08916667  0.1487261   0.05075715  0.16489772
   0.16081733  0.10383753  0.12400323  0.016721    0.14756855  0.22543293
  -0.01232284  0.13910106  0.19283172  0.13139977  0.13310152  0.16722342
   0.205526    0.12450512  0.09979653  0.43192956  0.15204728  0.17849533
   0.12074904  0.19255543]]
0.14209756


In [ ]:
# real 문서 + real3 단어평균
real3_noun_word_emb_mean = np.mean(real3_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, real3_noun_word_emb_mean)
distances

array([[0.17755625]], dtype=float32)

In [ ]:
# real 단어 + real3 문서
distances = cosine_similarity(real_noun_word_emb, real3_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.0844575


In [ ]:
# real 단어 + real3 단어
distances = cosine_similarity(real_noun_word_emb, real3_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.5576047  0.67196846 0.83261347 ... 0.7665353  0.8673779  0.8634747 ]
 [0.47651958 0.36186695 0.82070374 ... 0.47239146 0.5469426  0.43044466]
 [0.5711361  0.7163918  0.7556473  ... 0.8753253  0.864323   0.8588625 ]
 ...
 [0.527141   0.74942136 0.7726966  ... 0.7791547  0.85783803 0.92370045]
 [0.291      0.3172566  0.5538203  ... 0.540928   0.5903452  0.55034006]
 [0.48582825 0.52676046 0.5125721  ... 0.5805893  0.60820407 0.49955863]]
(40, 50)
0.61898226


In [ ]:
# real 단어평균 + real3 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, real3_sen_doc_emb)
distances

array([[0.10590087]], dtype=float32)

#### real + fake2

In [ ]:
fake2 = fake_data['Content'][2]
#가짜 기사 문서
fake2_sen_doc = preprocess_sentence(fake2, stop_words=stop_words, remove_stopwords=True)
#가짜 기사 단어
fake2_noun_doc = preprocess_noun(fake2, stop_words=stop_words, remove_stopwords=True)
fake2_noun_doc = fake2_noun_doc[0]
fake2_noun_word = get_word_list(fake2_noun_doc.split(), 50)
#임베딩
fake2_sen_doc_emb = model.encode([fake2_sen_doc])
print(f'fake2_sen_doc_emb.shape : {fake2_sen_doc_emb.shape}')
fake2_noun_word_emb = model.encode(fake2_noun_word)
print(f'fake2_noun_word_emb.shape : {fake2_noun_word_emb.shape}')

fake2_sen_doc_emb.shape : (1, 768)
fake2_noun_word_emb.shape : (50, 768)


In [ ]:
# real 문서 + fake2 문서
distances = cosine_similarity(real_sen_doc_emb, fake2_sen_doc_emb)
distances

array([[0.45364594]], dtype=float32)

In [ ]:
# real 문서 + fake2 단어
distances = cosine_similarity(real_sen_doc_emb, fake2_noun_word_emb)
print(distances)
print(np.mean(distances))

[[ 0.09635722  0.14272115 -0.01522709  0.21576038  0.16585752  0.14594845
   0.16000523  0.06311636  0.1855616   0.15180606  0.07281734  0.13310155
   0.1269289   0.43192956  0.18305993  0.05902605  0.1259798   0.14515185
  -0.01352105  0.14198472  0.09019913  0.07404239  0.13020822  0.21477368
   0.10857603  0.3068515   0.15407236  0.11380146  0.18402475  0.07817148
   0.18280217  0.17095225  0.08831017  0.14360613  0.09380423  0.15620065
   0.08848649  0.17866033  0.06022337  0.03254095  0.13462847  0.15944095
   0.03486344  0.13736808  0.06678333  0.1218477   0.20286454  0.09436946
   0.17849533  0.07895457]]
0.1315658


In [ ]:
# real 문서 + fake2 단어평균
fake2_noun_word_emb_mean = np.mean(fake2_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, fake2_noun_word_emb_mean)
distances

array([[0.1725595]], dtype=float32)

In [ ]:
# real 단어 + fake2 문서
distances = cosine_similarity(real_noun_word_emb, fake2_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.23150603


In [ ]:
# real 단어 + fake2 단어
distances = cosine_similarity(real_noun_word_emb, fake2_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.6758671  0.8484494  0.46202654 ... 0.8000898  0.7665353  0.7890922 ]
 [0.4587617  0.507185   0.38379097 ... 0.5094266  0.47239146 0.4462431 ]
 [0.75659335 0.7868749  0.48252526 ... 0.81216633 0.8753253  0.73475015]
 ...
 [0.6734233  0.8554228  0.4754923  ... 0.81549567 0.7791547  0.83109295]
 [0.37497193 0.51946974 0.2729228  ... 0.6059015  0.540928   0.5127658 ]
 [0.54128796 0.5310936  0.44662753 ... 0.5286308  0.5805893  0.43173248]]
(40, 50)
0.56896186


In [ ]:
# real 단어평균 + fake2 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, fake2_sen_doc_emb)
distances

array([[0.29669562]], dtype=float32)

## 문장 데이터 (불용어 제거 X)

단어 리스트는 불용어 제거 명사만 50개

### 변수 정의

In [ ]:
#진짜 기사 문서
real_sen_doc = preprocess_sentence(real)
real_sen_doc

'이미지 원 본보기 생활 의 달인 간장 새우 밥 의 달인 이 화제 로 떠올랐다 26일 방송 된 생활 의 달인 에서는 간장 새우 달인 이 운영 하는 소년 식당 이 소개 됐다 갖가지 재료 를 넣고 간장 을 졸여 새우 를 담가 만드는 간장 새우 잘못 조리 하면 비린 맛 이 나 먹기 가 힘들어 만들기 쉽지 않다고 한다 그만큼 만드는 사람 의 노하우 가 중요하다고 한다 여기 그 비린 맛 을 싹 없앤 간장 새우 를 만드는 달인 이 있다 그 주인공 은 바로 박 중천 36 세 경력 16년 달인 달인 의 가게 는 간장 새우 의 비린 맛 을 전혀 느낄 수 없어 사람 들 이 많이 찾는다고 한다 그 비밀 의 시작 은 바로 숙성 숙성 과정 의 비법 은 오이 와 레몬 을 갈아 청주 를 넣고 밥솥 에 찌는 것 이라고 한다 그것 을 식히고 새우 를 넣어 가다랑어 포의 향 을 더해 숙성 시킨다고 간장 새우 의 핵심 이라고 할 수 있는 간장 또한 평범하지 않다 특별한 비법 으로 구 운 소고기 를 간장 에 넣어 끓이고 여기 에 토마토 와 구 운 대파 를 넣어 간장 의 짠맛 을 잡아주고 은은한 단맛 을 내준다고 쌀밥 위 에 올라가는 후리카케 또한 특별하다고 한다 이 곳 은 서울시 마포구 와우산 로 29 라 길 16 에 위치 해 있다 저작권 자 서울 경제 무단 전 재 및 재 배포 금지'

In [ ]:
#진짜 기사 단어
real_noun_doc = preprocess_noun(real, stop_words=stop_words, remove_stopwords=True)
real_noun_doc = real_noun_doc[0]
real_noun_word = get_word_list(real_noun_doc.split(), 40)

In [ ]:
#가짜 기사 문서
fake_sen_doc = preprocess_sentence(fake)
fake_sen_doc

'이미지 원 본보기 소년 24 V 앱 방송 캡처 소년 24 의 첫 번째 V 앱 방송 이 진행 됐다 16일 오후 8시 네이버 V 앱 에서는 소년 24 의 놀러와 마이 소년 하우스 가 방송 됐다 이 날 한 집 에 모인 멤버 들 은 소년 식당 소년 공 방 소년 놀이 방 소년 벌칙 방 으로 나뉘어 각자 임무 를 수행 했다 첫 번째 팀 은 소년 식당 을 열어 요리 를 진행 했다 도하 팀 은 크림 파스타 와 로제 파스타 를 만들겠다고 나섰고 영 두 팀 은 김밥 을 만들기 시작 했다 두 번째 팀 은 소년 공 방 을 열었다 소년 들 은 팬 분 들 에 너무 받은 것 이 많아서 보답 하기 위해 팔찌 를 만들었다며 한 땀 한 땀 정성 들 여 제작 하는 모습 을 보여줬다 생일 을 맞은 MC 해준은 댓글 창 에 해준아 생일 축하 해 를 남겨주시면 선착순 다섯 분 께 CD 와 팔찌 를 보내 드리겠다고 말 했다 소년 놀이 방 에서는 소년 들 의 어린시절 사진 이 공개 됐다 MC 해준은 멤버 들 에 돌발 미션 을 냈다 해준은 제 어린 시절 사진 을 맞춰주시면 팬 분들 에 마술 을 보여 드리겠다고 말 했고 잘 맞추지 못 하는 멤버 들 에 어릴 때 부터 댄디 한 스타일 로 입었다며 힌트 를 주기도 했다 파스타 조 와 김밥 조로 나뉜 첫 번 째 팀 은 또 한 번 팬 들 에 미션 을 공지 했다 파스타 조 미션 문구 는 도하 진섭 지형 아 파스타 를 부탁 해였고 김밥 조의 미션 문구 는 영 두 창민 용 현아 김밥 잘 말아줘였다 앞선 미션 과 마찬가지 로 오타 없이 댓글 을 올려준 팬 들 에 선착순 으로 싸인 CD 팔찌 를 전달 할 것 을 알렸다 V 앱 방송 중 하트 백만 개 가 돌파 했고 소년 24 멤버 들 은 이를 기념 하기 위해 샤이니 의 누 난 너무 예뻐를 불렀다 그 사이 벌칙 방 멤버 들 은 두 팀 으로 나뉘어 요리 팀 의 부족한 재료 간식 을 사오는 미션 을 완료 했다 방송 시간 마무리 가 가까워오자 식당 공 방 놀이 방 벌칙 방이 세팅 을 마쳤다 그 때 최애 부활 전 을 통해 재 합류 

In [ ]:
#가짜 기사 단어
fake_noun_doc = preprocess_noun(fake, stop_words=stop_words, remove_stopwords=True)
fake_noun_doc = fake_noun_doc[0]
fake_noun_word = get_word_list(fake_noun_doc.split(), 50)

### 임베딩 (불용어 처리, 명사)

In [ ]:
real_sen_doc_emb = model.encode([real_sen_doc])
print(f'real_sen_doc_emb.shape : {real_sen_doc_emb.shape}')
real_noun_word_emb = model.encode(real_noun_word)
print(f'real_noun_word_emb.shape : {real_noun_word_emb.shape}')
fake_sen_doc_emb = model.encode([fake_sen_doc])
print(f'fake_sen_doc_emb.shape : {fake_sen_doc_emb.shape}')
fake_noun_word_emb = model.encode(fake_noun_word)
print(f'fake_noun_word_emb.shape : {fake_noun_word_emb.shape}')

real_sen_doc_emb.shape : (1, 768)
real_noun_word_emb.shape : (40, 768)
fake_sen_doc_emb.shape : (1, 768)
fake_noun_word_emb.shape : (50, 768)


### 코사인 유사도 (불용어 처리, 문장)

#### real + fake

In [ ]:
# real 문서 + fake 문서
distances = cosine_similarity(real_sen_doc_emb, fake_sen_doc_emb)
distances

array([[0.4686569]], dtype=float32)

In [ ]:
# real 문서 + fake 단어
distances = cosine_similarity(real_sen_doc_emb, fake_noun_word_emb)
print(distances)
print(np.mean(distances))

[[-0.15897653 -0.19739598 -0.12737617 -0.13536966  0.19630258 -0.03012478
  -0.11467578  0.22215028 -0.07188683 -0.03829678 -0.17419662 -0.19486529
   0.2817642  -0.17408596 -0.18063526 -0.05275441 -0.10816649 -0.14263324
  -0.1440574  -0.18634734 -0.06560585 -0.19509593 -0.1552354  -0.18214272
  -0.17969815 -0.15464473 -0.14887881 -0.17272556 -0.00589596 -0.16943799
  -0.12170868 -0.24084957 -0.16979504 -0.20537648 -0.09554917 -0.20359457
  -0.17127521 -0.20072955  0.02589105 -0.07152741 -0.10181454 -0.12535593
  -0.19108427 -0.00372453 -0.1722775  -0.07373247 -0.1595275  -0.19097258
  -0.1937633  -0.19170946]]
-0.116389304


In [ ]:
# real 문서 + fake 단어평균
fake_noun_word_emb_mean = np.mean(fake_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, fake_noun_word_emb_mean)
distances

array([[-0.14351484]], dtype=float32)

In [ ]:
# real 단어 + fake 문서
distances = cosine_similarity(real_noun_word_emb, fake_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.18364486


In [ ]:
# real 단어 + fake 단어
distances = cosine_similarity(real_noun_word_emb, fake_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.737159   0.8711393  0.6149226  ... 0.8942684  0.81560814 0.92055047]
 [0.38081753 0.5023731  0.3498181  ... 0.5386984  0.407575   0.5413713 ]
 [0.76245147 0.9123132  0.71214676 ... 0.88417697 0.83928204 0.9209814 ]
 ...
 [0.80436325 0.94215095 0.6495871  ... 0.94562644 0.8781835  0.9382018 ]
 [0.47295114 0.5467932  0.44780362 ... 0.5823052  0.55660504 0.6067578 ]
 [0.51025707 0.63135326 0.54693615 ... 0.5937297  0.60684544 0.639156  ]]
(40, 50)
0.60959643


In [ ]:
# real 단어평균 + fake 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, fake_sen_doc_emb)
distances

array([[0.23654202]], dtype=float32)

#### real + real2

In [ ]:
real2 = real_data['Content'][1]
#진짜 기사 문서
real2_sen_doc = preprocess_sentence(real2)
#진짜 기사 단어
real2_noun_doc = preprocess_noun(real2, stop_words=stop_words, remove_stopwords=True)
real2_noun_doc = real2_noun_doc[0]
real2_noun_word = get_word_list(real2_noun_doc.split(), 40)
#임베딩
real2_sen_doc_emb = model.encode([real2_sen_doc])
print(f'real2_sen_doc_emb.shape : {real2_sen_doc_emb.shape}')
real2_noun_word_emb = model.encode(real2_noun_word)
print(f'real2_noun_word_emb.shape : {real2_noun_word_emb.shape}')

real2_sen_doc_emb.shape : (1, 768)
real2_noun_word_emb.shape : (40, 768)


In [ ]:
# real 문서 + real2 문서
distances = cosine_similarity(real_sen_doc_emb, real2_sen_doc_emb)
distances

array([[0.523511]], dtype=float32)

In [ ]:
# real 문서 + real2 단어
distances = cosine_similarity(real_sen_doc_emb, real2_noun_word_emb)
print(distances)
print(np.mean(distances))

[[-0.14863887  0.2172224  -0.10029002  0.01562343 -0.19711441 -0.11228165
  -0.22689816 -0.03746037 -0.17969815 -0.15464473 -0.18634737 -0.14887881
  -0.14026357  0.06226691 -0.15869935 -0.11282379 -0.0721186  -0.15897653
   0.22215031 -0.20851734  0.00498634 -0.17265396 -0.11566339 -0.21877155
  -0.0221128  -0.18240133  0.03222074 -0.15534261  0.12003038  0.09484618
  -0.18147284  0.0528732  -0.0121945   0.10020643 -0.17307505 -0.1701149
  -0.16751073 -0.10523462 -0.1842583  -0.17068672]]
-0.08631797


In [ ]:
# real 문서 + real2 단어평균
real2_noun_word_emb_mean = np.mean(real2_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, real2_noun_word_emb_mean)
distances

array([[-0.12030309]], dtype=float32)

In [ ]:
# real 단어 + real2 문서
distances = cosine_similarity(real_noun_word_emb, real2_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.1942201


In [ ]:
# real 단어 + real2 단어
distances = cosine_similarity(real_noun_word_emb, real2_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.9999999  0.56501734 0.8677147  ... 0.51282823 0.6382621  0.913658  ]
 [0.56501734 1.0000002  0.53609437 ... 0.28310558 0.3560412  0.53156877]
 [0.8677147  0.53609437 0.99999964 ... 0.599368   0.72121054 0.89931476]
 ...
 [0.90625846 0.52388054 0.89294875 ... 0.56455696 0.6887381  0.912665  ]
 [0.58103365 0.3285708  0.5906943  ... 0.33889863 0.48009646 0.609683  ]
 [0.57517457 0.39308733 0.6797733  ... 0.52855885 0.5634472  0.6287033 ]]
(40, 40)
0.5700368


In [ ]:
# real 단어평균 + real2 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, real2_sen_doc_emb)
distances

array([[0.25139713]], dtype=float32)

#### real + real3

In [ ]:
real3 = real_data['Content'][2]
#진짜 기사 문서
real3_sen_doc = preprocess_sentence(real3)
#진짜 기사 단어
real3_noun_doc = preprocess_noun(real3, stop_words=stop_words, remove_stopwords=True)
real3_noun_doc = real3_noun_doc[0]
real3_noun_word = get_word_list(real3_noun_doc.split(), 50)
#임베딩
real3_sen_doc_emb = model.encode([real3_sen_doc])
print(f'real3_sen_doc_emb.shape : {real3_sen_doc_emb.shape}')
real3_noun_word_emb = model.encode(real3_noun_word)
print(f'real3_noun_word_emb.shape : {real3_noun_word_emb.shape}')

real3_sen_doc_emb.shape : (1, 768)
real3_noun_word_emb.shape : (50, 768)


In [ ]:
# real 문서 + real3 문서
distances = cosine_similarity(real_sen_doc_emb, real3_sen_doc_emb)
distances

array([[0.6343499]], dtype=float32)

In [ ]:
# real 문서 + real3 단어
distances = cosine_similarity(real_sen_doc_emb, real3_noun_word_emb)
print(distances)
print(np.mean(distances))

[[ 0.17153692 -0.19212648  0.00259414 -0.09997085 -0.16168326  0.22215031
   0.21722245 -0.20696284 -0.1646709  -0.07211865 -0.0323421  -0.14863881
  -0.12069147 -0.21858816 -0.16905037 -0.12482436 -0.13948819 -0.14309238
  -0.13484493  0.03802302 -0.1374642  -0.15897653  0.03304452 -0.21489468
  -0.09363824 -0.17369163 -0.17373061 -0.16153707 -0.2251095  -0.10432832
  -0.06297941 -0.10531999 -0.14924274 -0.0442906  -0.19686572 -0.16276997
   0.21865468 -0.132907    0.19630258 -0.05594671 -0.19854784 -0.14271434
   0.06226692 -0.19151306 -0.0905458  -0.09906259 -0.1314354  -0.04718224
  -0.16643599 -0.1285899 ]]
-0.090340376


In [ ]:
# real 문서 + real3 단어평균
real3_noun_word_emb_mean = np.mean(real3_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, real3_noun_word_emb_mean)
distances

array([[-0.11012842]], dtype=float32)

In [ ]:
# real 단어 + real3 문서
distances = cosine_similarity(real_noun_word_emb, real3_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.047172282


In [ ]:
# real 단어 + real3 단어
distances = cosine_similarity(real_noun_word_emb, real3_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.5576047  0.67196846 0.83261347 ... 0.7665353  0.8673779  0.8634747 ]
 [0.47651958 0.36186695 0.82070374 ... 0.47239146 0.5469426  0.43044466]
 [0.5711361  0.7163918  0.7556473  ... 0.8753253  0.864323   0.8588625 ]
 ...
 [0.527141   0.74942136 0.7726966  ... 0.7791547  0.85783803 0.92370045]
 [0.291      0.3172566  0.5538203  ... 0.540928   0.5903452  0.55034006]
 [0.48582825 0.52676046 0.5125721  ... 0.5805893  0.60820407 0.49955863]]
(40, 50)
0.61898226


In [ ]:
# real 단어평균 + real3 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, real3_sen_doc_emb)
distances

array([[0.05900887]], dtype=float32)

#### real + fake2

In [ ]:
fake2 = fake_data['Content'][2]
#가짜 기사 문서
fake2_sen_doc = preprocess_sentence(fake2)
#가짜 기사 단어
fake2_noun_doc = preprocess_noun(fake2, stop_words=stop_words, remove_stopwords=True)
fake2_noun_doc = fake2_noun_doc[0]
fake2_noun_word = get_word_list(fake2_noun_doc.split(), 50)
#임베딩
fake2_sen_doc_emb = model.encode([fake2_sen_doc])
print(f'fake2_sen_doc_emb.shape : {fake2_sen_doc_emb.shape}')
fake2_noun_word_emb = model.encode(fake2_noun_word)
print(f'fake2_noun_word_emb.shape : {fake2_noun_word_emb.shape}')

fake2_sen_doc_emb.shape : (1, 768)
fake2_noun_word_emb.shape : (50, 768)


In [ ]:
# real 문서 + fake2 문서
distances = cosine_similarity(real_sen_doc_emb, fake2_sen_doc_emb)
distances

array([[0.3498598]], dtype=float32)

In [ ]:
# real 문서 + fake2 단어
distances = cosine_similarity(real_sen_doc_emb, fake2_noun_word_emb)
print(distances)
print(np.mean(distances))

[[-0.09997085 -0.18657772 -0.14996016 -0.130775   -0.15150782  0.06945073
  -0.15008701 -0.19275793 -0.1200587  -0.06178501 -0.24792571 -0.19854784
  -0.0663608  -0.09906259 -0.1383729  -0.18597284 -0.12075113 -0.05793345
  -0.08228934 -0.18746033 -0.13544121 -0.20861447 -0.17602536  0.01782113
  -0.15209395 -0.03095216  0.01005652 -0.00409429 -0.22802582 -0.10073499
  -0.03310994 -0.1586537  -0.23262703 -0.17847048 -0.15856399 -0.19572704
  -0.11778805 -0.08796841 -0.19045107 -0.12336149 -0.12513095 -0.13310361
  -0.18439788 -0.09239119 -0.16878845 -0.18199745 -0.13037452 -0.14786726
  -0.04718224 -0.20376208]]
-0.12917055


In [ ]:
# real 문서 + fake2 단어평균
fake2_noun_word_emb_mean = np.mean(fake2_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_sen_doc_emb, fake2_noun_word_emb_mean)
distances

array([[-0.16731498]], dtype=float32)

In [ ]:
# real 단어 + fake2 문서
distances = cosine_similarity(real_noun_word_emb, fake2_sen_doc_emb)
# print(distances)
print(np.mean(distances))

0.14844291


In [ ]:
# real 단어 + fake2 단어
distances = cosine_similarity(real_noun_word_emb, fake2_noun_word_emb)
print(distances)
print(distances.shape)
print(np.mean(distances))

[[0.6758671  0.8484494  0.46202654 ... 0.8000898  0.7665353  0.7890922 ]
 [0.4587617  0.507185   0.38379097 ... 0.5094266  0.47239146 0.4462431 ]
 [0.75659335 0.7868749  0.48252526 ... 0.81216633 0.8753253  0.73475015]
 ...
 [0.6734233  0.8554228  0.4754923  ... 0.81549567 0.7791547  0.83109295]
 [0.37497193 0.51946974 0.2729228  ... 0.6059015  0.540928   0.5127658 ]
 [0.54128796 0.5310936  0.44662753 ... 0.5286308  0.5805893  0.43173248]]
(40, 50)
0.56896186


In [ ]:
# real 단어평균 + fake2 문서
real_noun_word_emb_mean = np.mean(real_noun_word_emb, axis=0).reshape(1,-1)
distances = cosine_similarity(real_noun_word_emb_mean, fake2_sen_doc_emb)
distances

array([[0.1895058]], dtype=float32)

# -----다른 기업3 스카이젯-----

In [ ]:
# filtered.groupby('기업명')['Content'].count().sort_values(ascending=False).to_csv('filtered.txt')
# remained.groupby('기업명')['Content'].count().sort_values(ascending=False).to_csv('remained.txt')
# full = pd.concat([filtered, remained], axis=0)
# full.groupby('기업명')['Content'].count().sort_values(ascending=False).to_csv('full.txt')

In [ ]:
# filtered[filtered['c_name']=='정보통신업'].groupby('기업명')['Content'].count().sort_values(ascending=False).to_csv('filtered_ic.txt')

In [40]:
filtered[filtered['기업명']=='스카이젯']

,기업명,Title,Content,c_index,c_name
2351,스카이젯,"룩인사이트 '약올려', 스카이젯-메르시팜 3자간 업무협약 체결",의약품 부동재고 거래 플랫폼 '약올려' 운영사 룩인사이트(대표 가진웅)가 의약품 도...,58,정보통신업
2352,스카이젯,의약품 부동재고 해결 업무협약,룩인사이트-스카이젯-메르시팜\n강원창조경제혁신센터가 보육한 의약품 부동재고 거래 플...,58,정보통신업


In [41]:
remained[remained['기업명']=='스카이젯']

,기업명,Title,Content,c_index,c_name
34386,스카이젯,"렉서스코리아, ‘스카이 젯’ 등장한 영화 발레리안 시사회 초청",렉서스코리아가 렉서스 1인용 우주선이 출연하는 영화 ‘발레리안: 천 개 행성의 도시...,58,정보통신업
34387,스카이젯,"[포토] [렉서스 브랜드 포럼] 렉서스가 디자인한 우주선, 스카이젯",[이데일리 오토in 김하은 기자] 4일 서울 잠실의 커넥트 투에서 렉서스 브랜드 포...,58,정보통신업


In [ ]:
filtered['Content'][2352]

'룩인사이트-스카이젯-메르시팜\n강원창조경제혁신센터가 보육한 의약품 부동재고 거래 플랫폼 기업인 룩인사이트(대표 가진웅)가 최근 스카이젯 본사에서 의약품 도매 전사적자원관리(ERP) 솔루션을 제공하는 스카이젯(대표 정재원), 의약품 전문 공급 업체 메르시팜(대표 박종현)과 \'의약품 산업 내 부동재고 문제 해결\'을 위한 다자간 업무협약을 체결했다.\n\n이번 협약으로 룩인사이트는 스카이젯과 상호 시스템 연동을 통해 전국 3000여개 도매상과 연결망을 확보하고 부동재고 정보를 빠르게 수집해 약국에 제공할 수 있을 것으로 기대된다. 또 메르시팜이 모든 물류 처리를 담당해 보다 안정적인 유통 체계를 구축할 수 있게 됐다. 룩인사이트는 강원창조센터가 \'2021 강원형 뉴딜 특화분야 창업 지원사업\'을 통해 보육한 기업으로 연간 수조원 규모로 반품·폐기되는 의약품 문제를 해결하고자 의약품 도매 거래 플랫폼 \'약올려\'를 개발하고 운영하고 있다. 가진웅 룩인사이트 대표는 "제약사 창고 내 폐기되는 의약품이 연간 1조원을 기록하는 등 심각한 수준이다"며 "부동재고의 처리 서비스는 꼭 필요하고 이번 협약을 통해 의약품 산업에 대한 사회·경제적 문제해결에 역할을 할 수 있을 것으로 기대한다"고 밝혔다.\n정우진 기자 jungwoojin@kado.net\n기자 프로필\n구독\n정우진 기자\n구독자 323\n응원수 483\n강원도내 전세아파트 10곳 중 1곳 깡통전세 \'위험\'\n원주 아파트 매매가 첫 하락…아파트 거래절벽 심화\n\n\nCopyright ⓒ 강원도민일보. All rights reserved. 무단 전재 및 재배포 금지.\n이 기사는 언론사에서 경제 섹션으로 분류했습니다.\n구독\n메인에서 바로 보는 언론사 편집 뉴스 지금 바로 구독해보세요!\n네이버뉴스 강원도민일보 구독 바로가기\n카카오뉴스 강원도민일보 구독 바로가기\n주요뉴스\n해당 언론사에서 선정하며 언론사 페이지(아웃링크)로 이동해 볼 수 있습니다.\n\'아파트가 고등어도 아니고…\' 1월 거래량 

In [44]:
remained['Content'][34386]

'렉서스코리아가 렉서스 1인용 우주선이 출연하는 영화 ‘발레리안: 천 개 행성의 도시’ 시사회에 고객들을 초청한다.\n\n뤽 베송 감독이 메가폰을 잡고, 데인 드한(발레리안 역), 카라 델러비인(로렐린 역)이 출연하는 ‘발레리안: 천 개 행성의 도시’는, 28세기 우주를 배경으로 펼쳐지는 SF영화로, 렉서스의 1인용 우주선 ‘스카이 젯(SKY JET)’이 등장한다.\n\n렉서스에 따르면 스카이 젯은 회사 디자이너들과 영화 크리에이티브팀의 협업을 통해 렉서스의 미래지향적인 디자인과 기술력을 실현했다. 공기역학적 외관 디자인, 목소리나 터치로 작동되는 조종석은 물론, LF-SA 콘셉트의 프론트 그릴, LC500 LED 램프가 스카이 젯에도 구현된다.\n\n이번 특별 시사회는 오는 11일까지 회사 공식 페이스북에서 진행된다. 당첨 결과는 8월 14일 렉서스코리아 페이스북에서 발표된다. 추첨을 통해 총 50팀(1인당 2매)가 선정되면 25일 청담 CGV상영관에서 영화를 관람하면 된다.\n\n동아닷컴 정진수 기자 brjeans@donga.com\n\n▶ 동아일보 단독 / 동아일보 공식 페이스북\n▶ \'아직도 당신만 모르나 VODA\'\n\nⓒ 동아일보 & donga.com, 무단 전재 및 재배포 금지\n동아닷컴 정진수 기자(brjeans@donga.com)\nCopyright ⓒ 동아일보. All rights reserved. 무단 전재 및 재배포 금지.\n이 기사는 언론사에서 생활 섹션으로 분류했습니다.\n구독\n메인에서 바로 보는 언론사 편집 뉴스 지금 바로 구독해보세요!\n주요뉴스\n해당 언론사에서 선정하며 언론사 페이지(아웃링크)로 이동해 볼 수 있습니다.\n\'택시운전사\' 첫 관람자들 후기 극과극 "뭉클, 10점 만점"vs "좌파선동, 0점"\n\'엄지척 논란\' 손혜원 "입이 열 개라도 할 말 없어…제가 잘못한 것"\n여름철 최악의 근무복장 2위 \'男-트레이닝복 女-과다노출 상의\', 1위는?\n\'11만원 때문에\' 같은 일용직 근무자 귀 찌른 40대

In [61]:
real_data = pd.DataFrame({'Content':[filtered['Content'][2351], filtered['Content'][2352]]})
fake_data = pd.DataFrame({'Content':[remained['Content'][34386]]})

In [62]:
data1 = '''식약처가 올해 추진한 '위해약 판매차단시스템' 시범사업이 긍정적인 결과가 나온 것으로 확인됐다. 
식약처는 올해 4월부터 해당 사업에 대한 공고를 통해 관련 프로그램을 개발을 한 후 지난 10월말부터 약국 등의 청구 프로그램업체에 참여 속에서 시범운영을 진행했다. 
이에 최근 시범운영에 대한 결과가 최근 나옴에 따라 현재 내부검토과정을 거치고 있다. 결과적으로는 이번에 개발된 위해약 차단프로그램이 약국 청구프로그램 등에 문제없이 적용된 것으로 알려졌다. 
식약처는 내부검토가 마무리되면 내년 확대사업에 이번 결과를 반영해 보다 개선된 사업으로 추진할 방침이다. 
식약처 관계자는 "이번 사업결과에 대해 아직 검토단계에 들어간 상황인만큼 구체적으로 밝히기는 어렵지만 프로그램 운영에 있어 별다른 문제점은 없었다"면서 "검토결과를 내년에 계획된 확대사업에 적극 반영할 것"이라고 밝혔다.
이어 "이번 사업은 위해약을 약국 등 현장에서 한층 빠르게 차단하기 위해 식약처가 정보를 알리는 형식이기 때문에 내년에 있을 사업도 신속한 정보전달 기능 개선에 초점을 둘 것"이라며 "의무적으로 해당 프로그램을 도입하는 차원이 아닌 현장에서 필요에 의해 자발적으로 사용할 수 있도록 유도할 것"이라고 덧붙였다. 
이와함께 "내년도 확대사업은 프로그램이 개발된 상황이어서 최소 유지관리에 필요한 몇 천만원의 예산이면 충분히 추진할 수 있을 것"이라고 예상했다. 
한편 올해 시범사업에는 4억원의 예산을 투입했으며 케이아이씨시스템즈를 통해 차단프로그램을 개발했다. 
이어 해당 프로그램 적용에는 약국의 경우 약학정보원과 유비케어, 도매업체는 신성아트컴과 대성정보기술, 스카이젯이 시범운영에 참여했다. 
아울러 시스템 개발에는 대한약사회와 한국의약품유통협회, 한국제약협회 등 관련 협회 및 전산업체 등이 참여했다. 
식약처가 개발한 판매차단시스템은 회수대상 의약품에 대한 제품명, 업체명, 제조번호, 회수사유 등의 정보와 회수대상을 조제하거나 공급하는 경우 경고 알림 등이 제공된다.'''

In [63]:
data1_df = pd.DataFrame({'Content':[data1]})

In [64]:
real_data = pd.concat([real_data, data1_df], axis=0)

In [69]:
real_data.reset_index(inplace=True, drop=True)

In [71]:
real_data

,Content
0,의약품 부동재고 거래 플랫폼 '약올려' 운영사 룩인사이트(대표 가진웅)가 의약품 도...
1,룩인사이트-스카이젯-메르시팜\n강원창조경제혁신센터가 보육한 의약품 부동재고 거래 플...
2,식약처가 올해 추진한 '위해약 판매차단시스템' 시범사업이 긍정적인 결과가 나온 것으...


In [56]:
data2 = '''[컨슈머타임스 김종효 기자] 오는 8월 30일 개봉을 앞두고 뤽 베송 감독의 내한 소식으로 더욱 기대를 모으고 있는 영화 '발레리안'이 국내외 다양한 행사 및 브랜드와 프로모션을 펼치며 역대급 컬래보레이션으로 예비 관객들을 사로잡고 있다.
28세기 미래, 우주의 평화를 지키기 위해 시공간을 이동하며 임무를 수행하는 최강 악동 에이전트 발레리안과 섹시 카리스마 에이전트 로렐린의 이야기를 다룬 액션 블록버스터 '발레리안: 천 개 행성의 도시'(수입/배급: 판씨네마㈜)가 유명 브랜드들과의 역대급 콜라보 작업으로 화제다. 
이미 올해 한국에서 처음 개최된 전 세계적인 팝 컬쳐 페스티벌 '코믹콘 서울 2017'에서 예비 관객들을 열광시킨 '발레리안'은 자동차 브랜드부터 건전지, 그리고 문구 브랜드까지 경계를 뛰어넘는 다양한 분야와의 콜라보를 통해 관객들과 미리 만난다. 
'발레리안'은 지난 8월 4일부터 6일까지 3일간 4만2,000여 명의 관람객들이 방문하며 성황리에 막을 내린 '코믹콘 서울 2017'에서 관객들과 만났다. '발레리안'의 두 주연 배우 데인 드한과 카라 델러비인이 코믹콘 서울 팬들을 위해 준비한 깜짝 인사 영상을 시작으로, 국내 최초로 공개된 '발레리안'의 제작기 영상, 그리고 코믹콘 팬들의 취향을 저격한 다양한 이벤트과 굿즈로 현장에서 뜨거운 반응을 불러일으켰다. 
당시 부스에서 사람들의 시선을 사로잡았던 콜라보 노트는 비스타의 감성적인 디자인과 '발레리안'의 28세기 미래 배경이 만나면서 광활한 우주가 펼쳐진 디자인으로 제작되어 관람객들의 소장 욕구를 자극했다.
코믹콘 행사에 이어 이번에는 자동차 피규어를 비롯한 영화의 비주얼을 담은 다양한 굿즈와 특별한 이벤트를 선보인다.
먼저 최고급 기술력과 세련된 디자인으로 전 세계적인 인기를 누리고 있는 세계적인 자동차 브랜드 렉서스와 최고의 시각효과 기술력으로 SF 블록버스터계에 새로운 지평을 여는 '발레리안'의 콜라보가 눈길을 끈다. 
렉서스는 28세기를 배경으로 펼쳐지는 '발레리안'에서 주인공 '발레리안'이 우주를 누비는 빠른 스피드의 1인용 우주선 '스카이젯'을 디자인한 것으로 알려지며 개봉 전부터 화제를 모았다. 이처럼 미래지향적인 디자인과 기술력을 통해 스카이젯을 제작한 렉서스와 '발레리안'은 웰메이드 스카이젯 피규어 증정 이벤트를 진행할 예정으로 이목을 끌고 있다. 
여기에 이미 전 세계 팬들에게 그래픽 노블의 전설로 열광적인 지지를 받고 있는 원작 도서 '발레리안'이 국내 출간돼 영화를 보는 재미를 더한다. 출판사 휴머니스트는 '발레리안' 국내 출간 기념으로 원작팬들을 위한 예매권 증정 이벤트를 진행할 예정이다. 
이와 함께 전국 마트를 비롯해 옥션, G마켓, 11번가, 인터파크 등 온라인 쇼핑 사이트에서는 벡셀과의 콜라보 이벤트가 진행되고 있다. 벡셀 건전지 상품을 구매한 후 벡셀 공식 홈페이지에 인증샷을 업로드하면 추첨을 통해 '발레리안' 예매권을 증정하는 이번 이벤트는 누구나 쉽게 참여할 수 있어 관객들의 열띤 호응이 예상된다.
다양한 브랜드와의 초특급 콜라보로 관객들의 호기심을 자극하는 영화 '발레리안'은 오는 8월 30일 IMAX, 4D, 3D 버전으로 개봉 예정이다.'''

In [72]:
data2_df = pd.DataFrame({'Content':[data2]})
fake_data = pd.concat([fake_data, data2_df], axis=0)
fake_data.reset_index(inplace=True, drop=True)

In [73]:
fake_data

,Content
0,렉서스코리아가 렉서스 1인용 우주선이 출연하는 영화 ‘발레리안: 천 개 행성의 도시...
1,[컨슈머타임스 김종효 기자] 오는 8월 30일 개봉을 앞두고 뤽 베송 감독의 내한 ...


In [74]:
real = real_data['Content'][0]
fake = fake_data['Content'][0]

In [75]:
real

'의약품 부동재고 거래 플랫폼 \'약올려\' 운영사 룩인사이트(대표 가진웅)가 의약품 도매 ERP 솔루션을 제공하는 스카이젯(대표 정재원), 병의원 의약품 전문 공급업체인 메르시팜(대표 박종현)과 함께 3자간 업무협약(MOU)을 체결했다고 밝혔다.\n\n사진제공=룩인사이트\n의약품산업 내 부동재고 문제 해결을 위한 이번 업무협약 체결에 따라 약올려와 스카이젯은 상호 시스템을 연동하고, 제반 상 발생하는 물류 일체를 메르시팜에서 담당하기로 했다. 협약을 통해 전국 3천여 개 도매상이 보유한 부동재고를 수집하여 약국에 노출시키고, \'선입선출 보상제(리워드)\'로 유통시장 내 부동재고 처분을 활성화시킬 계획이다.\n\n약올려는 연간 수조원의 반품·폐기되는 의약품 산업 문제를 해결하기 위해 개발됐다. 약올려를 이용하면, 도매상간 거래 시 주변 최저가 의약품을 쉽게 찾을 수 있고 부동재고를 서로 공유해 빠른 기간 내 처분할 수 있다.\n\n이번 협약을 통해 \'스카이젯\'을 사용하는 600여 개 도매상에게 재고 및 수주 연동으로 더욱 편리한 서비스를 제공할 뿐 아니라, 부동재고 정보 수집 및 약국 참여도 확대될 것으로 기대하고 있다.\n\n정재원 스카이젯 대표는 "전국 도매상에 쌓인 부동재고를 신속하게 처리하는 서비스는 꼭 필요하다고 생각해왔다. 개별 도매상 운영에 도움이 될 뿐 아니라, 사회적·경제적 문제를 해결하는데 큰 역할을 해줄 것으로 기대한다"고 말했다.\n\n박종현 메르시팜 대표는 "제약사 창고 내 폐기 문제 또한 심각한 수준이다. 보고서에 따르면 생산 후 출고조차 되지 못하고 창고에서 폐기되는 의약품이 연간 1조원이 넘는다. 이러한 문제 해결을 위해 중장기적인 노력을 할 것"이라고 말했다. 이에 가진웅 룩인사이트 대표는 "이번 협약으로 의약품 산업 문제 해결을 위한 목표에 한 걸음 더 나아가게 되어 매우 기쁘다"고 답했다.\n허남이 기자 (nyheoo@mt.co.kr)\nCopyright ⓒ 머니투데이. All rights reserved. 무단 전재 및

In [76]:
fake

'렉서스코리아가 렉서스 1인용 우주선이 출연하는 영화 ‘발레리안: 천 개 행성의 도시’ 시사회에 고객들을 초청한다.\n\n뤽 베송 감독이 메가폰을 잡고, 데인 드한(발레리안 역), 카라 델러비인(로렐린 역)이 출연하는 ‘발레리안: 천 개 행성의 도시’는, 28세기 우주를 배경으로 펼쳐지는 SF영화로, 렉서스의 1인용 우주선 ‘스카이 젯(SKY JET)’이 등장한다.\n\n렉서스에 따르면 스카이 젯은 회사 디자이너들과 영화 크리에이티브팀의 협업을 통해 렉서스의 미래지향적인 디자인과 기술력을 실현했다. 공기역학적 외관 디자인, 목소리나 터치로 작동되는 조종석은 물론, LF-SA 콘셉트의 프론트 그릴, LC500 LED 램프가 스카이 젯에도 구현된다.\n\n이번 특별 시사회는 오는 11일까지 회사 공식 페이스북에서 진행된다. 당첨 결과는 8월 14일 렉서스코리아 페이스북에서 발표된다. 추첨을 통해 총 50팀(1인당 2매)가 선정되면 25일 청담 CGV상영관에서 영화를 관람하면 된다.\n\n동아닷컴 정진수 기자 brjeans@donga.com\n\n▶ 동아일보 단독 / 동아일보 공식 페이스북\n▶ \'아직도 당신만 모르나 VODA\'\n\nⓒ 동아일보 & donga.com, 무단 전재 및 재배포 금지\n동아닷컴 정진수 기자(brjeans@donga.com)\nCopyright ⓒ 동아일보. All rights reserved. 무단 전재 및 재배포 금지.\n이 기사는 언론사에서 생활 섹션으로 분류했습니다.\n구독\n메인에서 바로 보는 언론사 편집 뉴스 지금 바로 구독해보세요!\n주요뉴스\n해당 언론사에서 선정하며 언론사 페이지(아웃링크)로 이동해 볼 수 있습니다.\n\'택시운전사\' 첫 관람자들 후기 극과극 "뭉클, 10점 만점"vs "좌파선동, 0점"\n\'엄지척 논란\' 손혜원 "입이 열 개라도 할 말 없어…제가 잘못한 것"\n여름철 최악의 근무복장 2위 \'男-트레이닝복 女-과다노출 상의\', 1위는?\n\'11만원 때문에\' 같은 일용직 근무자 귀 찌른 40대

## 불용어 제거 문장 데이터

### 변수 정의

In [77]:
#진짜 기사 문서
real_sen_doc = preprocess_sentence(real, stop_words=stop_words_ic, remove_stopwords=True)
real_sen_doc

'의약품 부동 재고 거래 플랫폼 약 운영 룩 인사이트 진웅 의약품 도매 ERP 솔루션 을 하는 스카이 젯 정재원 의약품 전문 공급 업체 메르 팜 박종현 함께 3 자간 업무 협약 MOU 을 체결 했다고 밝혔다 룩 인사이트 의약품 산업 부동 재고 문제 해결 을 위 업무 협약 체결 따라 약 와 스카이 젯 시스템 을 연동 하고 제반 발생 하는 물류 일체 를 메르 팜에서 하기로 했다 협약 을 3천 도매 상이 보유 부동 재고 를 수집 하여 약국 노출 시키고 선입선출 보상 워드 유통 부동 재고 처분 을 활성화 시킬 계획 이다 약 는 연간 수조 반품 폐기 되는 의약품 산업 문제 를 해결 하기 위해 개발 됐다 약 를 이용 하면 도매 간 거래 주변 최저 의약품 을 쉽게 찾을 있고 부동 재고 를 공유 빠른 기간 분할 있다 협약 을 스카이 젯 을 하는 600 도매 에게 재고 및 수주 연동 으로 더욱 편리한 서비스 를 뿐 아니라 부동 재고 수집 및 약국 될 으로 기대하고 있다 정재원 스카이 젯 는 도매 쌓인 부동 재고 를 신속하게 처리 하는 서비스 는 꼭 필요하다고 해왔다 개별 도매 운영 도움 될 뿐 아니라 사회 경제 문제 를 해결 하는데 큰 역할 을 해줄 으로 기대한다고 했다 박종현 메르 팜 는 약사 창고 폐기 문제 또한 심각한 수준 이다 보고서 따르면 생산 출고 조차 되지 못 하고 창고 에서 폐기 되는 의약품 연간 1조원 넘는다 이러한 문제 해결 을 위해 중장 기적 을 이라고 했다 진웅 룩 인사이트 는 협약 으로 의약품 산업 문제 해결 을 위 목표 걸음 더 나아가게 되어 기쁘다고 답 했다 허 nyheoo mt co kr'

In [78]:
#가짜 기사 문서
fake_sen_doc = preprocess_sentence(fake, stop_words=stop_words_ic, remove_stopwords=True)
fake_sen_doc

'렉서스 렉서스 1 인용 우주선 출연 하는 영화 발레 리안 행성 시사회 고객 들 을 한다 뤽 베송 감독 메가폰 을 잡고 데인 드 발레 리안 카라 델러비 로렐 린 출연 하는 발레 리안 행성 는 28 세기 우주 를 배경 으로 펼쳐지는 SF 영화로 렉서스 1 인용 우주선 스카이 젯 SKY JET 등장 한다 렉서스 따르면 스카이 젯 회사 디자이너 들 영화 크리에이티브 팀 협업 을 렉서스 미래 지향 디자인 기술 을 실현 했다 공기역학 외관 디자인 목소리 터치 작동 되는 조종석 물론 LF SA 콘셉트 프론트 그릴 LC 500 LED 램프 스카이 젯 에도 구현 된다 시사회 는 오는 11일 까지 회사 공식 페이스북 에서 진행 된다 당첨 결과 는 8월 14일 렉서스 페이스북 에서 된다 추첨 을 총 50 팀 1 인당 2 선정 되면 25일 청담 CGV 상영 에서 영화 를 관람 하면 된다 동아닷컴 정진 brjeans donga com 동아일보 단독 동아일보 공식 페이스북 아직도 당신 만 모르나 VODA 동아일보 donga com 및 동아닷컴 정진 brjeans donga com'

### 임베딩 (불용어 처리, 명사)

In [79]:
real_sen_doc_emb = model.encode([real_sen_doc])
print(f'real_sen_doc_emb.shape : {real_sen_doc_emb.shape}')
fake_sen_doc_emb = model.encode([fake_sen_doc])
print(f'fake_sen_doc_emb.shape : {fake_sen_doc_emb.shape}')

real_sen_doc_emb.shape : (1, 768)
fake_sen_doc_emb.shape : (1, 768)


### 코사인 유사도 (불용어 처리, 문장)

#### real + fake

In [80]:
# real 문서 + fake 문서
distances = cosine_similarity(real_sen_doc_emb, fake_sen_doc_emb)
distances

array([[0.33041406]], dtype=float32)

#### real + real2

In [81]:
real2 = real_data['Content'][1]
#진짜 기사 문서
real2_sen_doc = preprocess_sentence(real2, stop_words=stop_words_ic, remove_stopwords=True)
#임베딩
real2_sen_doc_emb = model.encode([real2_sen_doc])
print(f'real2_sen_doc_emb.shape : {real2_sen_doc_emb.shape}')

real2_sen_doc_emb.shape : (1, 768)


In [82]:
# real 문서 + real2 문서
distances = cosine_similarity(real_sen_doc_emb, real2_sen_doc_emb)
distances

array([[0.866843]], dtype=float32)

#### real + real3

In [83]:
real3 = real_data['Content'][2]
#진짜 기사 문서
real3_sen_doc = preprocess_sentence(real3, stop_words=stop_words_ic, remove_stopwords=True)
#임베딩
real3_sen_doc_emb = model.encode([real3_sen_doc])
print(f'real3_sen_doc_emb.shape : {real3_sen_doc_emb.shape}')

real3_sen_doc_emb.shape : (1, 768)


In [84]:
# real 문서 + real3 문서
distances = cosine_similarity(real_sen_doc_emb, real3_sen_doc_emb)
distances

array([[0.4178484]], dtype=float32)

#### real + fake2

In [85]:
fake2 = fake_data['Content'][1]
#진짜 기사 문서
fake2_sen_doc = preprocess_sentence(fake2, stop_words=stop_words_ic, remove_stopwords=True)
#임베딩
fake2_sen_doc_emb = model.encode([fake2_sen_doc])
print(f'fake2_sen_doc_emb.shape : {fake2_sen_doc_emb.shape}')

fake2_sen_doc_emb.shape : (1, 768)


In [86]:
# real 문서 + fake2 문서
distances = cosine_similarity(real_sen_doc_emb, fake2_sen_doc_emb)
distances

array([[0.50684386]], dtype=float32)

## 문장 데이터 (불용어 제거 X)

단어 리스트는 불용어 제거 명사만 50개

### 변수 정의

In [87]:
#진짜 기사 문서
real_sen_doc = preprocess_sentence(real)
real_sen_doc

'의약품 부동 재고 거래 플랫폼 약 올 려 운영 사 룩 인사이트 대표 가 진웅 가 의약품 도매 ERP 솔루션 을 제공 하는 스카이 젯 대표 정재원 병 의원 의약품 전문 공급 업체 인 메르 시 팜 대표 박종현 과 함께 3 자간 업무 협약 MOU 을 체결 했다고 밝혔다 사진 제공 룩 인사이트 의약품 산업 내 부동 재고 문제 해결 을 위 한 이번 업무 협약 체결 에 따라 약 올 려 와 스카이 젯 은 상호 시스템 을 연동 하고 제반 상 발생 하는 물류 일체 를 메르 시 팜에서 담당 하기로 했다 협약 을 통해 전국 3천 여 개 도매 상이 보유 한 부동 재고 를 수집 하여 약국 에 노출 시키고 선입선출 보상 제 리 워드 로 유통 시장 내 부동 재고 처분 을 활성화 시킬 계획 이다 약 올 려 는 연간 수조 원 의 반품 폐기 되는 의약품 산업 문제 를 해결 하기 위해 개발 됐다 약 올 려 를 이용 하면 도매 상 간 거래 시 주변 최저 가 의약품 을 쉽게 찾을 수 있고 부동 재고 를 서로 공유 해 빠른 기간 내 처 분할 수 있다 이번 협약 을 통해 스카이 젯 을 사용 하는 600 여 개 도매 상 에게 재고 및 수주 연동 으로 더욱 편리한 서비스 를 제공 할 뿐 아니라 부동 재고 정보 수집 및 약국 참여 도 확대 될 것 으로 기대하고 있다 정재원 스카이 젯 대표 는 전국 도매 상 에 쌓인 부동 재고 를 신속하게 처리 하는 서비스 는 꼭 필요하다고 생각 해왔다 개별 도매 상 운영 에 도움 이 될 뿐 아니라 사회 적 경제 적 문제 를 해결 하는데 큰 역할 을 해줄 것 으로 기대한다고 말 했다 박종현 메르 시 팜 대표 는 제 약사 창고 내 폐기 문제 또한 심각한 수준 이다 보고서 에 따르면 생산 후 출고 조차 되지 못 하고 창고 에서 폐기 되는 의약품 이 연간 1조원 이 넘는다 이러한 문제 해결 을 위해 중장 기적 인 노력 을 할 것 이라고 말 했다 이 에 가 진웅 룩 인사이트 대표 는 이번 협약 으로 의약품 산업 문제 해결 을 위 한 목표 에 한 걸음 더 나아가게 되어 매우 기쁘다

In [88]:
#가짜 기사 문서
fake_sen_doc = preprocess_sentence(fake)
fake_sen_doc

'렉서스 코리아 가 렉서스 1 인용 우주선 이 출연 하는 영화 발레 리안 천 개 행성 의 도시 시사회 에 고객 들 을 초청 한다 뤽 베송 감독 이 메가폰 을 잡고 데인 드 한 발레 리안 역 카라 델러비 인 로렐 린 역 이 출연 하는 발레 리안 천 개 행성 의 도시 는 28 세기 우주 를 배경 으로 펼쳐지는 SF 영화로 렉서스 의 1 인용 우주선 스카이 젯 SKY JET 이 등장 한다 렉서스 에 따르면 스카이 젯 은 회사 디자이너 들 과 영화 크리에이티브 팀 의 협업 을 통해 렉서스 의 미래 지향 적 인 디자인 과 기술 력 을 실현 했다 공기역학 적 외관 디자인 목소리 나 터치 로 작동 되는 조종석 은 물론 LF SA 콘셉트 의 프론트 그릴 LC 500 LED 램프 가 스카이 젯 에도 구현 된다 이번 특별 시사회 는 오는 11일 까지 회사 공식 페이스북 에서 진행 된다 당첨 결과 는 8월 14일 렉서스 코리아 페이스북 에서 발표 된다 추첨 을 통해 총 50 팀 1 인당 2 매 가 선정 되면 25일 청담 CGV 상영 관 에서 영화 를 관람 하면 된다 동아닷컴 정진 수 기자 brjeans donga com 동아일보 단독 동아일보 공식 페이스북 아직도 당신 만 모르나 VODA 동아일보 donga com 무단 전 재 및 재 배포 금지 동아닷컴 정진 수 기자 brjeans donga com'

### 임베딩 (불용어 처리, 명사)

In [89]:
real_sen_doc_emb = model.encode([real_sen_doc])
print(f'real_sen_doc_emb.shape : {real_sen_doc_emb.shape}')
fake_sen_doc_emb = model.encode([fake_sen_doc])
print(f'fake_sen_doc_emb.shape : {fake_sen_doc_emb.shape}')

real_sen_doc_emb.shape : (1, 768)
fake_sen_doc_emb.shape : (1, 768)


### 코사인 유사도 (불용어 처리, 문장)

#### real + fake

In [90]:
# real 문서 + fake 문서
distances = cosine_similarity(real_sen_doc_emb, fake_sen_doc_emb)
distances

array([[0.4185794]], dtype=float32)

#### real + real2

In [91]:
real2 = real_data['Content'][1]
#진짜 기사 문서
real2_sen_doc = preprocess_sentence(real2)
#임베딩
real2_sen_doc_emb = model.encode([real2_sen_doc])
print(f'real2_sen_doc_emb.shape : {real2_sen_doc_emb.shape}')

real2_sen_doc_emb.shape : (1, 768)


In [92]:
# real 문서 + real2 문서
distances = cosine_similarity(real_sen_doc_emb, real2_sen_doc_emb)
distances

array([[0.9003694]], dtype=float32)

#### real + real3

In [93]:
real3 = real_data['Content'][2]
#진짜 기사 문서
real3_sen_doc = preprocess_sentence(real3)
#임베딩
real3_sen_doc_emb = model.encode([real3_sen_doc])
print(f'real3_sen_doc_emb.shape : {real3_sen_doc_emb.shape}')

real3_sen_doc_emb.shape : (1, 768)


In [94]:
# real 문서 + real3 문서
distances = cosine_similarity(real_sen_doc_emb, real3_sen_doc_emb)
distances

array([[0.29574272]], dtype=float32)

#### real + fake2

In [95]:
fake2 = fake_data['Content'][1]
#진짜 기사 문서
fake2_sen_doc = preprocess_sentence(fake2)
#임베딩
fake2_sen_doc_emb = model.encode([fake2_sen_doc])
print(f'fake2_sen_doc_emb.shape : {fake2_sen_doc_emb.shape}')

fake2_sen_doc_emb.shape : (1, 768)


In [96]:
# real 문서 + fake2 문서
distances = cosine_similarity(real_sen_doc_emb, fake2_sen_doc_emb)
distances

array([[0.35875413]], dtype=float32)

#### test

In [97]:
test_data1 = '''건강보험심사평가원(원장 김승택) 의약품관리종합정보센터(센터장 이경자, 이하 ‘의약품정보센터’)는 의약품관리 상용소프트웨어업체를 대상으로 실시한 ‘프로그램 적합성 점검 결과’를 의약품정보관리시스템에 공개했다.
의약품정보센터는 ‘2017년 7월 예정된 ‘도매업체의 출하 시 보고제도’의 원활한 시행을 준비하기 위해, 의약품관리 상용소프트웨어업체의 프로그램이 올바른 공급내역 보고서식에 맞게 사용되고 있는지 프로그램 적합성 점검을 실시하고 있다.
프로그램 적합성 점검은 ① 상용 소프트웨어업체가 자율적으로 심사평가원에 점검신청 ② 공급내역 보고서식의 파일 생성 여부 및 기재점검, 전산점검 ③ 점검결과 공개 순서로 진행했다.
프로그램 적합성 점검 결과, 적합성 검사를 신청한 상용소프트웨어업체 7곳 중 6개 업체가 적합성 점검을 통과(4월 14일 기준)했으며, 통과 업체는 스카이젯, 트리플엠, 코스텍, 가나인포시스, 대성정보기술, 앤트소프트 등이다.
한편, 의약품정보센터는 제약사 및 도매업체에게 공급내역 출하 시 보고 유형을 분석하여 다양한 서비스도 제공하고 있다.
Open API(공개용 응용프로그램 인터페이스)를 통해 공급내역 보고 시 필요한 ▲ 표준코드 마스터 정보, RFID tag의 제조번호 및 유효기한 정보 ▲ 묶음번호(Aggregation) 내의 일련번호 정보 ▲ 실시간 입고정보 및 RFID tag 의약품 입고정보 등을 확인 가능하다.
또한 모바일 메신저인 카카오톡 옐로아이디에서 ‘의약품관리종합정보센터’를 친구 추가하면 ‘바코드 오류 등록’ 등 신고 서비스도 이용 가능하다.
건강보험심사평가원 이경자 의약품정보센터장은 “의약품 유통업체가 이번에 공개된 적합성 검사 결과를 참고하여 프로그램을 선택함으로써, 공급내역 보고 시 반송 또는 오류가 발생하여 재보고하는 불편함이 줄어들어 유통정보화가 한 단계 업그레이드 될 것으로 보고 있다”고 말했다.
또 “앞으로도 의약품 공급보고 신고체계가 열악한 도매상들이 원활히 공급내역을 보고하여 제도가 안정적으로 정착될 수 있도록 편리한 정보화 지원을 위해 노력하겠다”고 밝혔다.'''

In [98]:
#진짜 기사 문서
test_sen_doc = preprocess_sentence(test_data1)
#임베딩
test_sen_doc_emb = model.encode([test_sen_doc])
print(f'test_sen_doc_emb.shape : {test_sen_doc_emb.shape}')

test_sen_doc_emb.shape : (1, 768)


In [99]:
# real 문서 + fake2 문서
distances = cosine_similarity(real_sen_doc_emb, test_sen_doc_emb)
distances

array([[0.38341165]], dtype=float32)

# -----다른 기업4 비전정보기술-----

In [ ]:
# filtered.groupby('기업명')['Content'].count().sort_values(ascending=False).to_csv('filtered.txt')
# remained.groupby('기업명')['Content'].count().sort_values(ascending=False).to_csv('remained.txt')
# full = pd.concat([filtered, remained], axis=0)
# full.groupby('기업명')['Content'].count().sort_values(ascending=False).to_csv('full.txt')

In [ ]:
# filtered[filtered['c_name']=='정보통신업'].groupby('기업명')['Content'].count().sort_values(ascending=False).to_csv('filtered_ic.txt')

In [170]:
filtered[filtered['기업명']=='비전정보기술']

,기업명,Title,Content,c_index,c_name
2202,비전정보기술,"비전정보기술, 중국어 교육 프랜차이즈 사업 진출",온라인 중국어 교육 전문업체인 비전정보기술(www.nihao.co.kr 대표 마재영...,58,정보통신업
2203,비전정보기술,"[뉴스] 비전정보, ""중국어/한자교육 프랜차이즈""",온라인과 오프라인을 결합한 어린이 중국어/한자 교육 프랜차이즈 시스템이 국내에서 처...,58,정보통신업
2204,비전정보기술,[표] 2000 대한민국 특허기술대전 수상자 명단,= 상위 4개상 =\n\n -------------------------...,58,정보통신업


In [171]:
remained[remained['기업명']=='비전정보기술']

,기업명,Title,Content,c_index,c_name
30769,비전정보기술,`국내 SW업체 만리장성 본격 노크',(서울=연합뉴스) 김형태기자= 국내 소프트웨어(SW) 업체들이 본격적으로 중국에 진...,58,정보통신업
30770,비전정보기술,"[뉴스] 한국미디어교육 ""어학종합포털 사이트""",한국미디어교육(대표 박종태)은 어학종합 포털 사이트인 e에스오에스(www.esos....,58,정보통신업
30771,비전정보기술,"벤처기업들, 또하나의 벤처특구 '송파밸리' 형성",벤처기업들이 송파구 방이동 석촌동 신천동 등으로 몰리면서 테헤란밸리에 이어 또 하나...,58,정보통신업


In [ ]:
filtered['Content'][2204]

In [161]:
remained['Content'][30770]

'한국미디어교육(대표 박종태)은 어학종합 포털 사이트인 e에스오에스(www.esos.co.kr)를 개설했다고 28일 밝혔다.\n\n한국미디어교육은 지난 2년 동안 오프라인 어학교재 판매에 주력했으나 이번 사이트 오픈으로 어학분야 인터넷 사업에 집중할 예정이다.\n\ne에스오에스는 어학종합 포털 사이트로 영어, 일어, 중국어 분야 외국어 학습에 필요한 학습자료는 물론 사이버 강의, 시험정보, 커뮤니티, 쇼핑몰 등 어학관련 전부문의 정보를 제공할 예정이다.\n\ne에스오에스가 가진 경쟁력은 양질의 콘텐츠 무료이용과 더불어 외국어 학습에 필요한 교재들의 정보들을 수집할 수 있고 저렴한 가격에 어학교재들을 구입할 수 있다는 설명이다.\n\n박종태 대표는 “아직까지 어학교재시장이 오프라인에서만 머물러 있다"며 "다양한 정보와 채널을 가진 온라인 어학 사업이 향후 대세임을 고려해 앞으로 인터넷 서비스를 크게 강화할 예정”이라고 밝혔다.\n\n현재 무료로 진행 중인 사이버강의는 외대닷컴(www.oedae.com)과 제휴를 통해 7월 중순쯤 콘텐츠를 보강하여 유료할 예정인데 외국어 교재 구입시 무료이용 등의 혜택을 줄 예정이다. 그리고 ㈜닷츠코리아(www.mailzine.co.kr)와 제휴를 통해 메일학습을 서비스 할 예정이다.\n\n영어교재의 경우 ㈜태솔, 학문사, ㈜씨네파크, ㈜씨라인, 북맨, 다들미디어 등과 전략적 제휴를 통해 시중에서 구하기 힘든 경쟁력 있는 교재들을 저렴한 가격에 공급할 예정이다.\n\n또한 일어는 일본어 교재의 선두주자인 일본어뱅크를 통해 양질의 교재와 월간지 등을 공급받고 중국어 교재는 ㈜비전정보기술 등과의 제휴로 우수한 교재들을 공급받는다.\n\n머니투데이 홍진석 기자\n\nCopyright ⓒ 머니투데이. All rights reserved. 무단 전재 및 재배포 금지.\n이 기사는 언론사에서 경제 섹션으로 분류했습니다.\n구독\n메인에서 바로 보는 언론사 편집 뉴스 지금 바로 구독해보세요!\n좋아요\n0\n훈훈해요\n0\n슬퍼요\n0\n화나

In [172]:
stand_data = pd.DataFrame({'Content':[filtered['Content'][2203]]})
test_data = pd.DataFrame({'Content':[filtered['Content'][2202], remained['Content'][30769], remained['Content'][30770], remained['Content'][30771]]})

In [173]:
stand_data

,Content
0,온라인과 오프라인을 결합한 어린이 중국어/한자 교육 프랜차이즈 시스템이 국내에서 처...


In [174]:
test_data

,Content
0,온라인 중국어 교육 전문업체인 비전정보기술(www.nihao.co.kr 대표 마재영...
1,(서울=연합뉴스) 김형태기자= 국내 소프트웨어(SW) 업체들이 본격적으로 중국에 진...
2,한국미디어교육(대표 박종태)은 어학종합 포털 사이트인 e에스오에스(www.esos....
3,벤처기업들이 송파구 방이동 석촌동 신천동 등으로 몰리면서 테헤란밸리에 이어 또 하나...


In [176]:
standard = stand_data['Content'][0]

In [177]:
standard

'온라인과 오프라인을 결합한 어린이 중국어/한자 교육 프랜차이즈 시스템이 국내에서 처음 선보였다.\n\n온/오프라인 중국어 교육 전문업체인 비전정보기술(대표 마재영, www.nihao.co.kr)은 최근 송파구청에서 관련 사업설명회를 갖고, 이를 토대로 전국 학원을 하나의 단일 컨소시엄으로 묶는 본격적인 프랜차이즈 교육 시스템을 공급할 예정이라고 밝혔다.\n\n비전정보기술이 추진하고 있는 중국어/한자 교육 프랜차이즈는 이 회사가 지난 4년간 개발해온 멀티미디어 입체형 중국어/한자 동시 학습 시스템을 오프라인 학원에 프랜차이즈 형태로 제공하는 것. 자체 개발한 교육 컨텐츠인 \'니하오중국어\' 뿐 아니라 강사와 PC 등 교육용 인프라, 그림 차트, 놀이기구 등 기타 학습 보조 도구, 브랜드 등도 일괄 지원받을 수 있다.\n\n니하오중국어는 어린이를 위한 중국어 입문 과정에서부터 고급과정까지 전반적인 중국어 및 한자 학습시스템을 포괄하고 있으며, 중국동화 뿐 아니라 각종 놀이문화 역시 비전정보기술이 협력관계를 맺고 있는 중국 현지 출판사, 교육 단체 등의 시스템을 이용하고 있어 입체적인 교육이 가능하다.\n\n사업설명회에 참석한 수원의 혜민외국어학원 정연도 원장은 "서울이나 지방이나 영어에 이은 제 2 외국어 학습 열기를 현실적으로 충족시켜줄 수 있는 시스템이 부족한 게 현실이다. 교육 컨텐츠에서부터 각종 보조도구, 강사, 멀티미디어 학습 시스템까지 원스톱으로 제공되는 이번 프랜차이즈를 통해 큰 고민 없이 어린이를 위한 제 2 외국어 교육이 가능하게 됐다"고 밝혔다.\n\n비전정보기술은 이번 사업설명회를 시작으로 수도권은 각 구별로, 지방은 대도시별로 독점 프랜차이즈 운영권을 제공할 예정이다. 운영 학원들은 단위학급당 월 400만원 정도의 예상 매출이 가능할 것으로 전망하고 있다. 또한 독점권을 제공받은 학원측에는 비전정보기술의 주식도 일부 무상 제공할 계획이다.\n\n한편 비전정보기술은 이번에 첫 출범하는 중국어/한자 교육 시스템 프랜차이즈 사업 외에도 그동안 

## 불용어 제거 문장 데이터

### 변수 정의

In [178]:
#진짜 기사 문서
stand_sen_doc = preprocess_sentence(standard, stop_words=stop_words_ic, remove_stopwords=True)
stand_sen_doc

'온라인과 오프라인 을 결합 어린이 중국어 한자 프랜차이즈 시스템 에서 선보였다 온 오프라인 중국어 업체 비전 정보기술 마재영 www nihao co kr 구청 에서 업설 명 를 갖고 이를 토대 학원 을 단일 컨소시엄 으로 묶는 본격 프랜차이즈 시스템 을 공급 예정 이라고 밝혔다 비전 정보기술 추진 하고 있는 중국어 한자 프랜차이즈 는 회사 4년 간 개발 해온 멀티미디어 입체 형 중국어 한자 동시 학습 시스템 을 오프라인 학원 프랜차이즈 형태 하는 개발 컨텐츠 중국어 뿐 아니라 강사 와 PC 인프라 그림 차트 놀이기구 기타 학습 보조 도구 브랜드 일괄 지원 받을 있다 중국어 는 어린이 를 위 중국어 입문 에서부터 고급 까지 전반 중국어 및 한자 학습 시스템 을 포괄 하고 있으며 동화 뿐 아니라 각종 놀이 문화 비전 정보기술 협력 를 맺고 있는 현지 시스템 을 이용 하고 있어 입체 가능하다 업설 명 참석 혜민 외국어 학원 정연 이나 이나 영어 2 외국어 학습 열기 를 으로 충족 시켜줄 있는 시스템 부족한 이다 컨텐츠 에서부터 각종 보조 도구 강사 멀티미디어 학습 시스템 까지 원스톱 으로 되는 프랜차이즈 를 큰 고민 없이 어린이 를 위 2 외국어 가능하게 됐다고 밝혔다 비전 정보기술 업설 명 를 으로 수도권 구별 대도시 별로 독점 프랜차이즈 운영 을 예정 이다 운영 학원 들 단위 학급 400만원 정도 예상 매출 가능할 으로 전망 하고 있다 또한 독점 을 받은 학원 에는 비전 정보기술 주식 일부 무상 계획 이다 비전 정보기술 첫 출범 하는 중국어 한자 시스템 프랜차이즈 사업 에도 개발 해온 중국어 와 영어 일어 시스템 순차 으로 보급 하면서 한자 문화 서방 문화 을 연결하는 교두보 역할 을 한다는 방침 이다 마재영 비전 정보기술 한자 필요성 누구 절감 하면서도 접근 고심 하고 있는 이다 특히 영어 일반 화가 급속도 진행 되면서 상대 으로 2 외국어 새롭게 부상 하고 있으며 에서도 중국어 는 엄청난 잠재력 으로 인해 학습 욕구 높은 언어 이기도 하다 이런 을 쉽게 중

### 임베딩 (불용어 처리, 명사)

In [179]:
stand_sen_doc_emb = model.encode([stand_sen_doc])
print(f'stand_sen_doc_emb.shape : {stand_sen_doc_emb.shape}')

stand_sen_doc_emb.shape : (1, 768)


### 코사인 유사도 (불용어 처리, 문장)

#### stand + test1

In [180]:
test1 = test_data['Content'][0]
#기사 문서
test1_sen_doc = preprocess_sentence(test1, stop_words=stop_words_ic, remove_stopwords=True)
#임베딩
test1_sen_doc_emb = model.encode([test1_sen_doc])
print(f'test1_sen_doc_emb.shape : {test1_sen_doc_emb.shape}')

test1_sen_doc_emb.shape : (1, 768)


In [181]:
# stand 문서 + test1 문서
distances = cosine_similarity(stand_sen_doc_emb, test1_sen_doc_emb)
distances

array([[0.91002274]], dtype=float32)

#### stand + test2

In [182]:
test2 = test_data['Content'][1]
#기사 문서
test2_sen_doc = preprocess_sentence(test2, stop_words=stop_words_ic, remove_stopwords=True)
#임베딩
test2_sen_doc_emb = model.encode([test2_sen_doc])
print(f'test2_sen_doc_emb.shape : {test2_sen_doc_emb.shape}')

test2_sen_doc_emb.shape : (1, 768)


In [183]:
# stand 문서 + test2 문서
distances = cosine_similarity(stand_sen_doc_emb, test2_sen_doc_emb)
distances

array([[0.41220418]], dtype=float32)

#### stand + test3

In [184]:
test3 = test_data['Content'][2]
#기사 문서
test3_sen_doc = preprocess_sentence(test3, stop_words=stop_words_ic, remove_stopwords=True)
#임베딩
test3_sen_doc_emb = model.encode([test3_sen_doc])
print(f'test3_sen_doc_emb.shape : {test3_sen_doc_emb.shape}')

test3_sen_doc_emb.shape : (1, 768)


In [185]:
# stand 문서 + test3 문서
distances = cosine_similarity(stand_sen_doc_emb, test3_sen_doc_emb)
distances

array([[0.6103313]], dtype=float32)

#### stand + test4

In [186]:
test4 = test_data['Content'][3]
#기사 문서
test4_sen_doc = preprocess_sentence(test4, stop_words=stop_words_ic, remove_stopwords=True)
#임베딩
test4_sen_doc_emb = model.encode([test4_sen_doc])
print(f'test4_sen_doc_emb.shape : {test4_sen_doc_emb.shape}')

test4_sen_doc_emb.shape : (1, 768)


In [187]:
# stand 문서 + test4 문서
distances = cosine_similarity(stand_sen_doc_emb, test4_sen_doc_emb)
distances

array([[0.29805654]], dtype=float32)

## 문장 데이터 (불용어 제거 X)

단어 리스트는 불용어 제거 명사만 50개

### 변수 정의

In [188]:
#진짜 기사 문서
stand_sen_doc = preprocess_sentence(standard)
stand_sen_doc

'온라인과 오프라인 을 결합 한 어린이 중국어 한자 교육 프랜차이즈 시스템 이 국내 에서 처음 선보였다 온 오프라인 중국어 교육 전 문 업체 인 비전 정보기술 대표 마재영 www nihao co kr 은 최근 송파 구청 에서 관련 사 업설 명 회 를 갖고 이를 토대 로 전국 학원 을 하나 의 단일 컨소시엄 으로 묶는 본격 적 인 프랜차이즈 교육 시스템 을 공급 할 예정 이라고 밝혔다 비전 정보기술 이 추진 하고 있는 중국어 한자 교육 프랜차이즈 는 이 회사 가 지난 4년 간 개발 해온 멀티미디어 입체 형 중국어 한자 동시 학습 시스템 을 오프라인 학원 에 프랜차이즈 형태 로 제공 하는 것 자체 개발 한 교육 컨텐츠 인 니 하 오 중국어 뿐 아니라 강사 와 PC 등 교육 용 인프라 그림 차트 놀이기구 등 기타 학습 보조 도구 브랜드 등 도 일괄 지원 받을 수 있다 니 하 오 중국어 는 어린이 를 위 한 중국어 입문 과정 에서부터 고급 과정 까지 전반 적 인 중국어 및 한자 학습 시스템 을 포괄 하고 있으며 중국 동화 뿐 아니라 각종 놀이 문화 역시 비전 정보기술 이 협력 관계 를 맺고 있는 중국 현지 출판사 교육 단체 등 의 시스템 을 이용 하고 있어 입체 적 인 교육 이 가능하다 사 업설 명 회 에 참석 한 수원 의 혜민 외국어 학원 정연 도 원장 은 서울 이나 지방 이나 영어 에 이 은 제 2 외국어 학습 열기 를 현실 적 으로 충족 시켜줄 수 있는 시스템 이 부족한 게 현실 이다 교육 컨텐츠 에서부터 각종 보조 도구 강사 멀티미디어 학습 시스템 까지 원스톱 으로 제공 되는 이번 프랜차이즈 를 통해 큰 고민 없이 어린이 를 위 한 제 2 외국어 교육 이 가능하게 됐다고 밝혔다 비전 정보기술 은 이번 사 업설 명 회 를 시작 으로 수도권 은 각 구별 로 지방 은 대도시 별로 독점 프랜차이즈 운영 권 을 제공 할 예정 이다 운영 학원 들 은 단위 학급 당 월 400만원 정도 의 예상 매출 이 가능할 것 으로 전망 하고 있다 또한 독점 권 을 제공 받은 학원 측 

### 임베딩 (불용어 처리, 명사)

In [189]:
stand_sen_doc_emb = model.encode([stand_sen_doc])
print(f'stand_sen_doc_emb.shape : {stand_sen_doc_emb.shape}')

stand_sen_doc_emb.shape : (1, 768)


### 코사인 유사도 (불용어 처리, 문장)

#### stand + test1

In [190]:
test1 = test_data['Content'][0]
#기사 문서
test1_sen_doc = preprocess_sentence(test1)
#임베딩
test1_sen_doc_emb = model.encode([test1_sen_doc])
print(f'test1_sen_doc_emb.shape : {test1_sen_doc_emb.shape}')

test1_sen_doc_emb.shape : (1, 768)


In [191]:
# stand 문서 + test1 문서
distances = cosine_similarity(stand_sen_doc_emb, test1_sen_doc_emb)
distances

array([[0.8595869]], dtype=float32)

#### stand + test2

In [192]:
test2 = test_data['Content'][1]
#기사 문서
test2_sen_doc = preprocess_sentence(test2)
#임베딩
test2_sen_doc_emb = model.encode([test2_sen_doc])
print(f'test2_sen_doc_emb.shape : {test2_sen_doc_emb.shape}')

test2_sen_doc_emb.shape : (1, 768)


In [193]:
# stand 문서 + test2 문서
distances = cosine_similarity(stand_sen_doc_emb, test2_sen_doc_emb)
distances

array([[0.59255606]], dtype=float32)

#### stand + test3

In [194]:
test3 = test_data['Content'][2]
#기사 문서
test3_sen_doc = preprocess_sentence(test3)
#임베딩
test3_sen_doc_emb = model.encode([test3_sen_doc])
print(f'test3_sen_doc_emb.shape : {test3_sen_doc_emb.shape}')

test3_sen_doc_emb.shape : (1, 768)


In [195]:
# stand 문서 + test3 문서
distances = cosine_similarity(stand_sen_doc_emb, test3_sen_doc_emb)
distances

array([[0.70458376]], dtype=float32)

#### stand + test4

In [196]:
test4 = test_data['Content'][3]
#기사 문서
test4_sen_doc = preprocess_sentence(test4)
#임베딩
test4_sen_doc_emb = model.encode([test4_sen_doc])
print(f'test4_sen_doc_emb.shape : {test4_sen_doc_emb.shape}')

test4_sen_doc_emb.shape : (1, 768)


In [197]:
# stand 문서 + test4 문서
distances = cosine_similarity(stand_sen_doc_emb, test4_sen_doc_emb)
distances

array([[0.42063236]], dtype=float32)